In [1]:
from __future__ import division
import nltk 
import re
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import lxml.html
from nltk.corpus import stopwords
from nltk.stem import *
import networkx as nx
import json
import pickle

In [2]:
'''
This function is used to get the root of the xml tree.
Input- filename
Output- root of xml tree
'''
def getroot_xml(filename):
    root = ET.parse(filename).getroot()
    return root

In [3]:
'''
This function parses xml data
Input- root of xml tree
Output- Title, Question body, Title + Question, Tags
'''
def get_questions_tags(root):
    questions = []
    tags = []
    titles = []
    users = []
    ques_with_title = []
    for row in root.findall('row'):
        post = row.get("PostTypeId")
        user = row.get("OwnerUserId")
        post_type = BeautifulSoup(post, "lxml")
        if post_type.get_text() == "1" and user is not None:
            
            users.append(user)
            
            #Get the Questions
            body = row.get("Body")
            soup = BeautifulSoup(body, "lxml")
            [s.extract() for s in soup('code')] #To remove code from the question posted
            question_s = soup.get_text()
#             q_set = nltk.word_tokenize(question_s)
            q_set = question_s.split()
            question = nltk.Text(q_set)
            questions.append(question)
            
            #Get the Tags
            tag_list = row.get("Tags")
            tag_str = re.sub('[<>]', ' ', tag_list)
#             tag_set = nltk.word_tokenize(tag_str)
            tag_set = tag_str.split()        
            tag_text = nltk.Text(tag_set)
            tags.append(tag_text)

            #Get the Titles
            title_s = row.get("Title")
#             t_set = nltk.word_tokenize(title_s)
            t_set = title_s.split()
            title = nltk.Text(t_set)
            titles.append(title)
            
            q_with_t = title_s + " " + question_s
#             qt_set = nltk.word_tokenize(q_with_t)
            qt_set = q_with_t.split()
            ques_title = nltk.Text(qt_set)
            ques_with_title.append(ques_title)

    return questions, tags, titles, ques_with_title, users

In [4]:
'''
This function removes numbers and special characters from question
'''
def number_removal(ques_list):
    q_with_t_list = []
    for ques in ques_list:
        q_t_text = ""
        for word in ques:
            characters = [".", ",", ":", "(", ")", "[", "]", "{", "}", "?", "'"]
            q_text = ''.join([i for i in word if not (i.isdigit() or [e for e in characters if e in i])])
            if q_text != '':
                q_t_text += q_text + " "
        qt_set = q_t_text.split()
#         qt_set = nltk.word_tokenize(q_t_text)
        title_ques = nltk.Text(qt_set)
        q_with_t_list.append(title_ques)
    return q_with_t_list

In [5]:
'''
This function removes stopwords from the question body
'''
def remove_stopwords(q_list):
    q_with_t_list = []
    for text in q_list:
        stopwords = nltk.corpus.stopwords.words('english')
        st = ""
        for w in text:
            if w.lower() not in stopwords:
                st += w.lower() + " "
        w_set = st.split()
#         w_set = nltk.word_tokenize(st)
        ques_body = nltk.Text(w_set)
        q_with_t_list.append(ques_body)
    return q_with_t_list

In [6]:
'''
This function performs stemming and coverts each word in the question to it's root word
'''
def stemming(q_list):
    stemmer = PorterStemmer()
    post = []
    for q in q_list:
        st = ""
        for word in q:
            st += stemmer.stem(word) + " "
#         w_set = nltk.word_tokenize(st)
        w_set = st.split()
        ques_body = nltk.Text(w_set)
        post.append(ques_body)
    return post

In [7]:
'''
This function will do the final fixes. Last stage of pre-processing
'''
def fixing(text):
    post = []
    for tokens in text:
        for i, t in enumerate(tokens):
            if t == '#' and i > 0:
                left = tokens[:i-1]
                joined = [tokens[i - 1] + t]
                right = tokens[i + 1:]
                tokens = left + joined + right
        post.append(tokens)
    return post

In [8]:
'''
Preprocessor component - Tokenisation, Number removal, Stop-word removal, Stemming
'''
def preprocessor(filename):
    root = getroot_xml(filename)
    questions, tags, titles, ques_with_title, users = get_questions_tags(root)
    ques_with_title_list = number_removal(ques_with_title)
    title_ques = remove_stopwords(ques_with_title_list)
    posts = stemming(title_ques)
    frame = pd.DataFrame({0 : titles,
                          1 : questions,
                          2 : ques_with_title,
                          3 : posts,
                          4 : tags,
                          5 : users})
    return frame

In [9]:
'''
Frequentist Inference Component - POS Tagger
'''
def pos_tag(posts):
    pos_tags = []
    for post in posts:
        tag = []
        for word in post:
            te = nltk.pos_tag([word])
            t = te[0]
            if t[1].startswith('NN') or t[1].startswith('JJ'):
                tag.append(t)
        pos_tags.append(tag)
    return pos_tags

In [10]:
'''
Write POS TAG output to file
'''
def write_to_file(filename, posts):
    o = open(filename,'w')
    pos = pos_tag(posts)
    o.write(str(pos))
    o.close()

In [11]:
'''
Set of all possible tags
'''
def set_of_tags(tag_column):
    tags = []
    for row in tag_column:
        for t in row:
            if t not in tags:
                tags.append(t)
    return tags

In [12]:
'''
Reads the postag file
'''
def readfile(filename):
    f = open(filename,'r')
    software_object = []
    for line in f.readlines():
        print line

In [13]:
'''
Creating post frame
'''
def create_post_frame(pos_tag_list, tag_column):
    posts = []
    tags = []
    for i in range(len(pos_tag_list)):
        post_row = []
        ptl = pos_tag_list[i]
        tc = tag_column[i]
        tags.append(tc)
        for p in ptl:
            post_row.append(p[0])
        posts.append(post_row)
    frame = pd.DataFrame({0 : posts,
                          1 : tags})
    return frame

In [20]:
def bic_output(dictionary, tot_so):
    output = []
    for so in tot_so:
        word_tag = {}
        for word in so:
            for k, v in dictionary.items():
                if word[0] in v:
                    word_tag[k] = v[word[0]]
        output.append(word_tag)
    return output

In [21]:
def getTotalProbability(tot_so, tags, soNtags):
    probs = []
    for so in tot_so:
        probs.append(probability(so, tags, soNtags))
    return probs

In [15]:
'''
Set of all possible tags
'''
def set_of_tags(tag_column):
    tags = []
    for row in tag_column:
        for t in row:
            if t not in tags:
                tags.append(t)
    return tags

In [16]:
# edge_wts = create_weights(soNtags)
# tot_probs = getTotalProbability([tot_so[0]], tags, soNtags)
tags = set_of_tags(p[4])
# soNtags = create_post_frame(tot_so, p[4])
p = preprocessor('Posts_small.xml')

In [52]:
# list(p[3])

In [70]:
def convert_frame(frame, set_tags):
    posts = []
    lengths = []
    tags = []
    for tag in set_tags:
        tag_row = []
        tags.append([tag])
        for i in range(len(frame)):
            t = frame[4][i]
            p = frame[3][i]
            st = ""
            for c in p:
                st = st + " " + c
            if tag in t:
                tag_row.append(st)
        posts.append(tag_row)
        lengths.append(len(tag_row))
    frame = pd.DataFrame({0 : tags,
                          1 : posts,
                          2 : lengths})
    return frame

convert_frame(p, tags)

,0,1,2
0,[c#],[ set form opac use decim doubl want use track...,34
1,[winforms],[ set form opac use decim doubl want use track...,3
2,[type-conversion],[ set form opac use decim doubl want use track...,1
3,[opacity],[ set form opac use decim doubl want use track...,1
4,[html],[ doesnt percentag width child absolut posit p...,7
5,[css],[ doesnt percentag width child absolut posit p...,3
6,[css3],[ doesnt percentag width child absolut posit p...,1
7,[internet-explorer-7],[ doesnt percentag width child absolut posit p...,1
8,[code-generation],[ tool convert visual j# code c# convers tool ...,1
9,[j#],[ tool convert visual j# code c# convers tool ...,1


In [67]:
# tot_so = json.load(open('postags.json', 'r'))
# tags = pickle.load(open('tags.pkl', 'rb'))
# edge_wts = pickle.load(open('edgeWeights.pkl', 'rb'))
# tot_probs = pickle.load(open('probabilities.pkl', 'rb'))
# soNtags = pickle.load(open('soNtags.pkl', 'rb'))

In [23]:
diction = {'bytearray': {u'regex': 0.13360918312867057, u'imagin': 0.00013347570742124932, u'breed': 0.00013347570742124932, u'cross-cultur': 0.00013347570742124932, u'log': 0.26708489054991991, u'sourcesaf': 0.00013347570742124932, u'tester': 0.00013347570742124932, u'internation': 0.00013347570742124932, u'famou': 0.00013347570742124932, u'breakpoint': 0.00013347570742124932, u'set': 0.00013347570742124932, u'claim': 0.00013347570742124932, u'minimum': 0.00013347570742124932, u'attain': 0.13360918312867057, u'aw': 0.00013347570742124932, u'baggag': 0.00013347570742124932, u'good/bett': 0.00013347570742124932, u'air/flex': 0.13360918312867057, u'phpeclips': 0.00013347570742124932, u'grow': 0.00013347570742124932}, 'osx': {u'websit': 0.039630040402439989, u'"form': 0.019824922760041195, u'checkbox': 0.039630040402439989, u'log': 0.079240275687237599, u'technic': 0.019824922760041195, u'me"': 0.059435158044838787, u'author': 0.019824922760041195, u'secret': 0.039630040402439989, u'check': 0.039630040402439989, u'topic': 0.039630040402439989, u'cooki': 0.019824922760041195, u'remain': 0.039630040402439989, u'session': 0.059435158044838787, u'common': 0.039630040402439989, u'persist': 0.039630040402439989, u'answer': 0.039630040402439989, u'cookiesessionstor': 0.019824922760041195, u'"rememb': 0.059435158044838787, u'store': 0.039630040402439989, u'browser': 0.039630040402439989}, 'msbuild': {u'applic': 0.037784991695606215, u'breed': 3.7747244451155068e-05, u'legaci': 0.037784991695606215, u'excus': 3.7747244451155068e-05, u'registri': 0.15102672504907141, u'internation': 3.7747244451155068e-05, u'famou': 3.7747244451155068e-05, u'breakpoint': 3.7747244451155068e-05, u'hive': 0.075532236146761286, u'per': 0.075532236146761286, u'profil': 0.037784991695606215, u'moment': 0.075532236146761286, u'minimum': 3.7747244451155068e-05, u'user': 0.22652121395138156, u'key': 0.075532236146761286, u'isnt': 0.075532236146761286, u'good/bett': 3.7747244451155068e-05, u'unicod': 3.7747244451155068e-05, u'phpeclips': 3.7747244451155068e-05, u'enlighten': 0.037784991695606215}, 'primes': {u'pleas': 0.10545722713864304, u'claim': 0.00010535187526337968, u'set': 0.21080910240202272, u'cross-cultur': 0.00010535187526337968, u'sourcesaf': 0.00010535187526337968, u'/md': 0.10545722713864304, u'internation': 0.00010535187526337968, u'breed': 0.00010535187526337968, u'tester': 0.00010535187526337968, u'famou': 0.00010535187526337968, u'grow': 0.00010535187526337968, u'imagin': 0.00010535187526337968, u'minimum': 0.00010535187526337968, u'build': 0.21080910240202272, u'breakpoint': 0.00010535187526337968, u'good/bett': 0.00010535187526337968, u'unicod': 0.00010535187526337968, u'phpeclips': 0.00010535187526337968, u'imposs': 0.10545722713864304, u'excus': 0.00010535187526337968}, 'graph': {u'node': 0.17031329814404905, u'tester': 4.2567682615358418e-05, u'cross-cultur': 4.2567682615358418e-05, u'sourcesaf': 4.2567682615358418e-05, u'would': 0.1277456155286906, u'minim': 0.042610250297973773, u'internation': 4.2567682615358418e-05, u'graph': 0.1277456155286906, u'metric': 0.042610250297973773, u'random': 0.042610250297973773, u'list': 0.1277456155286906, u'correspond': 0.042610250297973773, u'node/edg': 0.042610250297973773, u'direct': 0.042610250297973773, u'aw': 4.2567682615358418e-05, u'breakpoint': 4.2567682615358418e-05, u'cs': 0.042610250297973773, u'famou': 4.2567682615358418e-05, u'baggag': 4.2567682615358418e-05, u'simpler': 0.042610250297973773}, 'flash': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'datacontext': {u'claim': 2.5321584118302443e-05, u'datacontext': 0.2532411627671427, u'smaller': 0.025346905702420743, u'multipl': 0.075990073939025626, u'expos': 0.025346905702420743, u'relationship': 0.025346905702420743, u'namespac': 0.050668489820723188, u'ever': 0.025346905702420743, u'individu': 0.025346905702420743, u'dispos': 0.025346905702420743, u'correspond': 0.050668489820723188, u'impos': 0.025346905702420743, u'thought': 0.050668489820723188, u'orm': 0.025346905702420743, u'ui': 0.025346905702420743, u'distant': 0.025346905702420743, u'unicod': 2.5321584118302443e-05, u'perspect': 0.025346905702420743, u'class': 0.15195482629393298, u'unnecessari': 0.025346905702420743}, 'resources': {u'front-end': 0.023015727030258438, u'applic': 0.16097213280603331, u'dilemma': 0.023015727030258438, u'insert': 0.023015727030258438, u'row': 0.023015727030258438, u'descript': 0.023015727030258438, u'enumer': 0.069001195622183389, u'correspond': 0.023015727030258438, u'contract': 0.046008461326220912, u'claim': 2.2992734295962477e-05, u'second': 0.023015727030258438, u'integ': 0.023015727030258438, u'uploaddocu': 0.023015727030258438, u'pass': 0.091993929918145881, u'hardcod': 0.023015727030258438, u'invoic': 0.046008461326220912, u'document': 0.13797939851007082, u'back-end': 0.046008461326220912, u'valu': 0.091993929918145881, u'workord': 0.023015727030258438}, 'string': {u'packag': 0.014831387423694657, u'osx': 0.029647958276536476, u'enterpris': 0.014831387423694657, u'python': 0.28152966277484737, u'menu': 0.029647958276536476, u'hobbyist': 0.014831387423694657, u'builtin': 0.014831387423694657, u'per': 0.014831387423694657, u'doc': 0.014831387423694657, u'keystrok': 0.014831387423694657, u'mac': 0.029647958276536476, u'api': 0.059281099982220115, u'decis': 0.029647958276536476, u'docs/tutori': 0.014831387423694657, u'enter': 0.029647958276536476, u'press': 0.029647958276536476, u'onlin': 0.014831387423694657, u'smell': 0.014831387423694657, u'find': 0.059281099982220115, u'select': 0.044464529129378295}, 'http-authentication': {u'cross-cultur': 4.879953152449736e-05, u'breed': 4.879953152449736e-05, u'xampp': 0.097647862580519218, u'recogn': 0.048848331056021856, u'file': 0.19524692562951396, u'claim': 4.879953152449736e-05, u'directori': 0.14644739410501659, u'titl': 0.048848331056021856, u'configur': 0.097647862580519218, u'cross': 4.879953152449736e-05, u'breakpoint': 4.879953152449736e-05, u'up/hav': 0.048848331056021856, u'outsid': 0.097647862580519218, u'famou': 4.879953152449736e-05, u'q/a': 0.048848331056021856, u'"searchable"': 0.048848331056021856, u'phpeclips': 4.879953152449736e-05, u'grow': 4.879953152449736e-05, u'good/bett': 4.879953152449736e-05, u'internation': 4.879953152449736e-05}, 'syntax': {u'despit': 0.064613994319648843, u'breed': 6.4549444874774072e-05, u'cross-cultur': 6.4549444874774072e-05, u'claim': 6.4549444874774072e-05, u'url': 0.12916343919442291, u'famou': 6.4549444874774072e-05, u'lack': 0.064613994319648843, u'excus': 6.4549444874774072e-05, u'state': 0.12916343919442291, u'rest': 0.064613994319648843, u'reload': 0.064613994319648843, u'ajax': 0.19371288406919698, u'minimum': 6.4549444874774072e-05, u'breakpoint': 6.4549444874774072e-05, u'address': 0.12916343919442291, u'ensur': 6.4549444874774072e-05, u'unicod': 6.4549444874774072e-05, u'phpeclips': 6.4549444874774072e-05, u'good/bett': 6.4549444874774072e-05, u'internation': 6.4549444874774072e-05}, 'viewstate': {u'load': 0.064613994319648843, u'claim': 6.4549444874774072e-05, u'set': 6.4549444874774072e-05, u'cross-cultur': 6.4549444874774072e-05, u'appear': 0.064613994319648843, u'scenario': 0.064613994319648843, u'opera': 0.064613994319648843, u'extra-blo': 0.064613994319648843, u'breed': 6.4549444874774072e-05, u'famou': 6.4549444874774072e-05, u'invalid': 0.064613994319648843, u'breakpoint': 6.4549444874774072e-05, u'internation': 6.4549444874774072e-05, u'reli': 0.064613994319648843, u'safari': 0.19371288406919698, u'heavili': 0.064613994319648843, u'good/bett': 6.4549444874774072e-05, u'viewstat': 0.064613994319648843, u'phpeclips': 6.4549444874774072e-05, u'throw': 0.064613994319648843}, 'open-source': {u'claim': 4.4460252534234394e-05, u'term': 0.088964965321003009, u'graphij"property"="value"': 0.044504712786768624, u'faster': 0.044504712786768624, u'phpeclips': 4.4460252534234394e-05, u'memori': 0.088964965321003009, u'data': 0.17788547038947183, u'manipul': 0.044504712786768624, u'breed': 4.4460252534234394e-05, u'intuit': 0.044504712786768624, u'layer': 0.044504712786768624, u'clearer': 0.044504712786768624, u'separ': 0.044504712786768624, u'good/bett': 4.4460252534234394e-05, u'common': 0.044504712786768624, u'breakpoint': 4.4460252534234394e-05, u'hadnt': 0.044504712786768624, u'extract': 0.044504712786768624, u'speed': 0.044504712786768624, u'class': 0.044504712786768624}, 'list': {u'load': 0.10545722713864304, u'ugli': 0.10545722713864304, u'py': 0.00010535187526337968, u'cross-cultur': 0.00010535187526337968, u'sourcesaf': 0.00010535187526337968, u'tester': 0.00010535187526337968, u'internation': 0.00010535187526337968, u'consum': 0.10545722713864304, u'compress': 0.10545722713864304, u'breakpoint': 0.00010535187526337968, u'claim': 0.00010535187526337968, u'imagin': 0.00010535187526337968, u'sleep': 0.10545722713864304, u'aw': 0.00010535187526337968, u'baggag': 0.00010535187526337968, u'good/bett': 0.00010535187526337968, u'disadvantag': 0.21080910240202272, u'phpeclips': 0.00010535187526337968, u'grow': 0.00010535187526337968, u'simpler': 0.00010535187526337968}, 'versioning': {u'play': 0.15695120037658875, u'hit': 0.03926722108896908, u'clip': 0.03926722108896908, u'song': 0.03926722108896908, u'track': 0.19617919347246196, u'famou': 3.9227993095873212e-05, u'pop': 0.03926722108896908, u'grow': 3.9227993095873212e-05, u'distinct': 0.03926722108896908, u'player': 0.11772320728071552, u'internation': 3.9227993095873212e-05, u'mp': 0.11772320728071552, u'breakpoint': 3.9227993095873212e-05, u'good/bett': 3.9227993095873212e-05, u'cross-cultur': 3.9227993095873212e-05, u'unicod': 3.9227993095873212e-05, u'phpeclips': 3.9227993095873212e-05, u'youll': 0.03926722108896908, u'click': 0.03926722108896908, u'plan': 0.03926722108896908}, 'webdav': {u'iceberg': 9.5310712924132673e-05, u'excus': 9.5310712924132673e-05, u'love': 0.095406023637056789, u'cm': 0.095406023637056789, u'english': 9.5310712924132673e-05, u'internation': 9.5310712924132673e-05, u'famou': 9.5310712924132673e-05, u'tip': 9.5310712924132673e-05, u'lean': 0.095406023637056789, u'ensur': 9.5310712924132673e-05, u'rd': 0.095406023637056789, u'minimum': 9.5310712924132673e-05, u'contact': 0.19071673656118945, u'breakpoint': 9.5310712924132673e-05, u'address': 0.095406023637056789, u'good/bett': 9.5310712924132673e-05, u'cross-cultur': 9.5310712924132673e-05, u'unicod': 9.5310712924132673e-05, u'phpeclips': 9.5310712924132673e-05, u'com': 0.095406023637056789}, 'asp-classic': {u'oo': 0.044504712786768624, u'wide': 0.044504712786768624, u'domain': 0.044504712786768624, u'cross-cultur': 4.4460252534234394e-05, u'advantag': 0.044504712786768624, u'offer': 0.044504712786768624, u'mess': 0.044504712786768624, u'index': 0.044504712786768624, u'relat': 0.088964965321003009, u'db': 0.044504712786768624, u'enterpris': 0.044504712786768624, u'worri': 0.044504712786768624, u'internation': 4.4460252534234394e-05, u'orm': 0.044504712786768624, u'arent': 0.044504712786768624, u'famou': 4.4460252534234394e-05, u'good/bett': 4.4460252534234394e-05, u'model': 0.088964965321003009, u'massiv': 0.088964965321003009, u'orient': 0.088964965321003009}, 'cvs': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'aptana': {u'svn': 0.12437300501595987, u'bin': 0.031101023919081372, u'task': 0.02073747046387265, u'develop': 0.031101023919081372, u'goal': 0.010373917008663929, u'ankh': 0.010373917008663929, u'gui': 0.02073747046387265, u'sometim': 0.02073747046387265, u'subvers': 0.16582721883679477, u'better': 0.041464577374290101, u'heard': 0.02073747046387265, u'client': 0.051828130829498817, u'environ': 0.082918791195124977, u'integr': 0.041464577374290101, u'effici': 0.010373917008663929, u'plug-in': 0.010373917008663929, u'love': 0.02073747046387265, u'tortoisesvn': 0.062191684284707539, u'realli': 0.031101023919081372, u'latest': 0.031101023919081372}, 'design': {u'xml': 0.067450328148880703, u'xfdl': 0.1124022296143127, u're-encod': 0.022498426683448709, u'encrypt': 0.022498426683448709, u'sourcesaf': 2.2475950732715994e-05, u'filesave/sav': 0.022498426683448709, u'format': 0.089926278881596711, u'viewer': 0.022498426683448709, u'belong': 0.022498426683448709, u'#': 0.044974377416164703, u'templat': 0.022498426683448709, u'base-gzip': 0.022498426683448709, u'fli': 0.022498426683448709, u'decod': 0.022498426683448709, u'file': 0.22478198327789264, u'identifi': 0.022498426683448709, u'document': 0.022498426683448709, u'modifi': 0.089926278881596711, u'xml-base': 0.022498426683448709, u'updat': 0.044974377416164703}, 'plugins': {u'attach': 0.014614845529404891, u'xampp/apach': 0.014614845529404891, u'major': 0.014614845529404891, u'weve': 0.014614845529404891, u'multipl': 0.058415581381767216, u'secur': 0.029215090813525663, u'apach': 0.07301582666588799, u'larger': 0.014614845529404891, u'db': 0.043815336097646436, u'event': 0.014614845529404891, u'worri': 0.014614845529404891, u'server': 0.13141680780237106, u'hook': 0.029215090813525663, u'session': 0.029215090813525663, u'necessari': 0.014614845529404891, u'letter': 0.014614845529404891, u'team': 0.014614845529404891, u'php': 0.2920195059276996, u'across': 0.043815336097646436, u'land': 0.014614845529404891}, 'download': {u'overrid': 0.069072591774772277, u'claim': 6.9003588186585703e-05, u'imagin': 6.9003588186585703e-05, u'breed': 6.9003588186585703e-05, u'cross-cultur': 6.9003588186585703e-05, u'excus': 6.9003588186585703e-05, u'internation': 6.9003588186585703e-05, u'famou': 6.9003588186585703e-05, u'overlook': 0.069072591774772277, u'minimum': 6.9003588186585703e-05, u'per': 0.069072591774772277, u'host': 0.27608335633452941, u'set': 6.9003588186585703e-05, u'virtual': 0.27608335633452941, u'breakpoint': 6.9003588186585703e-05, u'good/bett': 6.9003588186585703e-05, u'unicod': 6.9003588186585703e-05, u'phpeclips': 6.9003588186585703e-05, u'grow': 6.9003588186585703e-05, u'logfil': 0.069072591774772277}, 'tortoisesvn': {u'excus': 5.7168991538989253e-05, u'usecond': 0.057226160530528231, u'eye': 5.7168991538989253e-05, u'win': 0.11439515206951748, u'howev': 0.057226160530528231, u'resolut': 0.057226160530528231, u'tip': 5.7168991538989253e-05, u'sandcastl': 5.7168991538989253e-05, u'gettimeofday': 0.057226160530528231, u'api': 0.057226160530528231, u'somebodi': 5.7168991538989253e-05, u'frequenc': 0.057226160530528231, u'ensur': 5.7168991538989253e-05, u'guarante': 0.057226160530528231, u'unicod': 5.7168991538989253e-05, u'bridg': 5.7168991538989253e-05, u'baggag': 5.7168991538989253e-05, u'port': 0.22873313514749602, u'microsecond': 0.057226160530528231, u'portabl': 0.057226160530528231}, 'sql-server-2008': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, '64bit': {u'altern': 0.023682715523363155, u'vista': 0.047353595606684659, u'old': 0.047353595606684659, u'hour': 0.035518155565023905, u'remain': 0.011847275481702408, u'colleagu': 0.023682715523363155, u'screen': 0.023682715523363155, u'drive': 0.011847275481702408, u'hang': 0.023682715523363155, u'funki': 0.023682715523363155, u'read-onli': 0.023682715523363155, u'soon': 0.023682715523363155, u'stop': 0.023682715523363155, u'window': 0.15387255598163138, u'html': 0.023682715523363155, u'intern': 0.023682715523363155, u'lightweight': 0.035518155565023905, u'crash': 0.0828599157316669, u'preview': 0.023682715523363155, u'tortois': 0.035518155565023905}, 'cx-oracle': {u'imagin': 9.5310712924132673e-05, u'set': 0.19071673656118945, u'cross-cultur': 9.5310712924132673e-05, u'sourcesaf': 9.5310712924132673e-05, u'claim': 9.5310712924132673e-05, u'internation': 9.5310712924132673e-05, u'famou': 9.5310712924132673e-05, u'breed': 9.5310712924132673e-05, u'tip': 9.5310712924132673e-05, u'iter': 0.19071673656118945, u'minimum': 9.5310712924132673e-05, u'excus': 9.5310712924132673e-05, u'ensur': 9.5310712924132673e-05, u'way': 0.28602744948532211, u'breakpoint': 9.5310712924132673e-05, u'good/bett': 9.5310712924132673e-05, u'unicod': 9.5310712924132673e-05, u'phpeclips': 9.5310712924132673e-05, u'grow': 9.5310712924132673e-05, u'sever': 0.095406023637056789}, 'printing': {u'therefor': 0.026005403720253558, u'locat': 0.077964252312168761, u'faster': 0.026005403720253558, u'regardless': 0.026005403720253558, u'possibl': 0.026005403720253558, u'interact': 0.026005403720253558, u'safe': 0.026005403720253558, u'sync': 0.077964252312168761, u'respons': 0.026005403720253558, u'server': 0.12992310090408399, u'jump': 0.026005403720253558, u'version': 0.051984828016211161, u'branch/build': 0.026005403720253558, u'repositori': 0.077964252312168761, u'b': 0.026005403720253558, u'respect': 0.026005403720253558, u'subvers': 0.051984828016211161, u'local': 0.10394367660812638, u'present': 0.026005403720253558, u'feder': 0.026005403720253558}, 'vb6': {u'imagin': 8.7017055342847191e-05, u'phpeclips': 8.7017055342847191e-05, u'sourcesaf': 8.7017055342847191e-05, u'claim': 8.7017055342847191e-05, u'internation': 8.7017055342847191e-05, u'famou': 8.7017055342847191e-05, u'breed': 8.7017055342847191e-05, u'minimum': 8.7017055342847191e-05, u'tip': 8.7017055342847191e-05, u'ensur': 8.7017055342847191e-05, u'score': 0.34815523842673163, u'lucen': 0.17412112774103722, u'breakpoint': 8.7017055342847191e-05, u'good/bett': 8.7017055342847191e-05, u'unicod': 8.7017055342847191e-05, u'john': 0.087104072398190027, u'ie': 0.087104072398190027, u'grow': 8.7017055342847191e-05, u'match': 0.087104072398190027, u'excus': 8.7017055342847191e-05}, 'nant': {u'imagin': 0.00010535187526337968, u'minimum': 0.00010535187526337968, u'cross-cultur': 0.00010535187526337968, u'sourcesaf': 0.00010535187526337968, u'tester': 0.00010535187526337968, u'internation': 0.00010535187526337968, u'famou': 0.00010535187526337968, u'breed': 0.00010535187526337968, u'reach': 0.10545722713864304, u'grow': 0.00010535187526337968, u'claim': 0.00010535187526337968, u'item': 0.21080910240202272, u'alreadi': 0.10545722713864304, u'obvious': 0.10545722713864304, u'breakpoint': 0.00010535187526337968, u'good/bett': 0.00010535187526337968, u'unicod': 0.00010535187526337968, u'phpeclips': 0.00010535187526337968, u'initialis': 0.10545722713864304, u'expand': 0.10545722713864304}, '3d': {u'imagin': 6.4549444874774072e-05, u'tester': 6.4549444874774072e-05, u'set': 6.4549444874774072e-05, u'd': 0.064613994319648843, u'sourcesaf': 6.4549444874774072e-05, u'opengl': 0.064613994319648843, u'baggag': 6.4549444874774072e-05, u'phpeclips': 6.4549444874774072e-05, u'float': 0.12916343919442291, u'descript': 0.064613994319648843, u'grow': 6.4549444874774072e-05, u'claim': 6.4549444874774072e-05, u'~': 0.064613994319648843, u'sampl': 0.064613994319648843, u'breakpoint': 6.4549444874774072e-05, u'good/bett': 6.4549444874774072e-05, u'comb': 0.064613994319648843, u'forth': 0.064613994319648843, u'display': 0.19371288406919698, u'seri': 0.064613994319648843}, 'winapi': {u'cross-cultur': 5.4077438892494048e-05, u'weve': 0.054131516331386534, u'constant': 0.054131516331386534, u'coupl': 0.054131516331386534, u'excus': 5.4077438892494048e-05, u'editing/view': 0.054131516331386534, u'famou': 5.4077438892494048e-05, u'breed': 5.4077438892494048e-05, u'minimum': 5.4077438892494048e-05, u'cross': 0.054131516331386534, u'non-port': 0.054131516331386534, u'game': 0.054131516331386534, u'set': 5.4077438892494048e-05, u'breakpoint': 5.4077438892494048e-05, u'linux': 0.37859614968635086, u'good/bett': 5.4077438892494048e-05, u'x': 0.054131516331386534, u'unicod': 5.4077438892494048e-05, u'phpeclips': 5.4077438892494048e-05, u'schema': 0.054131516331386534}, 'apache': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'search': {u'claim': 6.4549444874774072e-05, u'tester': 6.4549444874774072e-05, u'excus': 6.4549444874774072e-05, u'cross-cultur': 6.4549444874774072e-05, u'sourcesaf': 6.4549444874774072e-05, u'imagin': 6.4549444874774072e-05, u'internation': 6.4549444874774072e-05, u'famou': 6.4549444874774072e-05, u'gener': 0.064613994319648843, u'breed': 6.4549444874774072e-05, u'warn': 0.19371288406919698, u'cast': 0.19371288406919698, u'minimum': 6.4549444874774072e-05, u'baggag': 6.4549444874774072e-05, u'good/bett': 6.4549444874774072e-05, u'unicod': 6.4549444874774072e-05, u'phpeclips': 6.4549444874774072e-05, u'type': 0.25826232894397111, u'grow': 6.4549444874774072e-05, u'mean': 0.12916343919442291}, 'rockbox': {u'imagin': 0.00013347570742124932, u'phpeclips': 0.00013347570742124932, u'cross-cultur': 0.00013347570742124932, u'rockbox': 0.13360918312867057, u'format': 0.13360918312867057, u'internation': 0.00013347570742124932, u'famou': 0.00013347570742124932, u'breed': 0.00013347570742124932, u'tip': 0.00013347570742124932, u'specifi': 0.13360918312867057, u'claim': 0.00013347570742124932, u'callback': 0.13360918312867057, u'minimum': 0.00013347570742124932, u'ensur': 0.00013347570742124932, u'breakpoint': 0.00013347570742124932, u'good/bett': 0.00013347570742124932, u'unicod': 0.00013347570742124932, u'audio': 0.13360918312867057, u'grow': 0.00013347570742124932, u'excus': 0.00013347570742124932}, 'vb.net': {u'imagin': 6.9003588186585703e-05, u'phpeclips': 6.9003588186585703e-05, u'cross-cultur': 6.9003588186585703e-05, u'sourcesaf': 6.9003588186585703e-05, u'internation': 6.9003588186585703e-05, u'breed': 6.9003588186585703e-05, u'thousand': 0.069072591774772277, u'sp': 0.069072591774772277, u'suppos': 6.9003588186585703e-05, u'breakpoint': 6.9003588186585703e-05, u'second': 0.069072591774772277, u'environ': 0.069072591774772277, u'grow': 6.9003588186585703e-05, u'bolt-on': 0.069072591774772277, u'"chang': 0.069072591774772277, u'x': 0.069072591774772277, u'compani': 0.069072591774772277, u'row': 0.13807617996135799, u'interv': 0.069072591774772277, u'schema': 0.069072591774772277}, 'code-completion': {u'imagin': 0.00022261798753339269, u'set': 0.00022261798753339269, u'cross-cultur': 0.00022261798753339269, u'aw': 0.00022261798753339269, u'sourcesaf': 0.00022261798753339269, u'tester': 0.00022261798753339269, u'internation': 0.00022261798753339269, u'famou': 0.00022261798753339269, u'breed': 0.00022261798753339269, u'discoveri': 0.22284060552092608, u'write': 0.22284060552092608, u'minimum': 0.00022261798753339269, u'breakpoint': 0.00022261798753339269, u'claim': 0.00022261798753339269, u'good/bett': 0.00022261798753339269, u'unicod': 0.00022261798753339269, u'phpeclips': 0.00022261798753339269, u'baggag': 0.00022261798753339269, u'grow': 0.00022261798753339269, u'simpler': 0.00022261798753339269}, 'color-space': {u'origin': 0.035420944558521558, u'followup': 0.035420944558521558, u'given': 0.053122566027048075, u'certainli': 0.017719323089995041, u'word': 0.017719323089995041, u'equival': 0.017719323089995041, u'color': 0.1947355377752602, u'individu': 0.017719323089995041, u'closer': 0.017719323089995041, u'm': 0.017719323089995041, u'say': 0.053122566027048075, u'maxim': 0.035420944558521558, u'distinct': 0.14163067336968066, u'visual': 0.053122566027048075, u'farther': 0.017719323089995041, u'human': 0.035420944558521558, u'bluish': 0.017719323089995041, u'roughli': 0.017719323089995041, u'red': 0.017719323089995041, u'bunch': 0.017719323089995041}, 'queue': {u'queue<t>': 0.19071673656118945, u'set': 9.5310712924132673e-05, u'cross-cultur': 9.5310712924132673e-05, u'imagin': 9.5310712924132673e-05, u'internation': 9.5310712924132673e-05, u'famou': 9.5310712924132673e-05, u'breed': 9.5310712924132673e-05, u'excus': 9.5310712924132673e-05, u'capac': 0.19071673656118945, u'inherit': 0.095406023637056789, u'claim': 9.5310712924132673e-05, u'ensur': 9.5310712924132673e-05, u'minimum': 9.5310712924132673e-05, u'limit': 0.19071673656118945, u'breakpoint': 9.5310712924132673e-05, u'good/bett': 9.5310712924132673e-05, u'unicod': 9.5310712924132673e-05, u'phpeclips': 9.5310712924132673e-05, u'grow': 9.5310712924132673e-05, u'dequeu': 0.095406023637056789}, 'navigation': {u'ftp': 0.014200192929694147, u'domain': 0.028386199852465526, u'via': 0.056758213698008285, u'log': 0.056758213698008285, u'incorrectli': 0.014200192929694147, u'remot': 0.028386199852465526, u'webconfig': 0.028386199852465526, u'base-encod': 0.014200192929694147, u'activ': 0.014200192929694147, u'site': 0.099316234466322414, u'ankhsvn': 0.014200192929694147, u'properli': 0.028386199852465526, u'thank': 0.028386199852465526, u'sent': 0.014200192929694147, u'download': 0.056758213698008285, u'viewstat': 0.056758213698008285, u'clarifi': 0.014200192929694147, u'disabl': 0.014200192929694147, u'valu': 0.028386199852465526, u'aspnet': 0.15606026215740793}, 'sqlite': {u'control': 0.17289835029141562, u'life': 0.024720932529882444, u'sourc': 0.17289835029141562, u'central': 0.07411340511706016, u'distribut': 0.098809641410649032, u'sourcesaf': 2.4696236293588858e-05, u'tester': 2.4696236293588858e-05, u'laptop': 0.024720932529882444, u'hoop': 0.024720932529882444, u'baggag': 2.4696236293588858e-05, u'request': 0.049417168823471302, u'linux"': 0.024720932529882444, u'"run': 0.024720932529882444, u'offic': 0.049417168823471302, u'client': 0.07411340511706016, u'flame': 0.024720932529882444, u'consult': 0.024720932529882444, u'stabl': 0.024720932529882444, u'flexibl': 0.024720932529882444, u'easier': 0.024720932529882444}, 'vim': {u'phpeclips': 4.879953152449736e-05, u'cross-cultur': 4.879953152449736e-05, u'longer': 0.048848331056021856, u'also': 0.048848331056021856, u'syntax': 0.048848331056021856, u'breed': 4.879953152449736e-05, u'express': 0.24404645715401133, u'famou': 4.879953152449736e-05, u'o': 0.048848331056021856, u'repres': 0.048848331056021856, u'binari': 0.24404645715401133, u'minimum': 4.879953152449736e-05, u'ensur': 4.879953152449736e-05, u'breakpoint': 4.879953152449736e-05, u'answer': 0.097647862580519218, u'unicod': 4.879953152449736e-05, u'charact': 0.048848331056021856, u'excus': 4.879953152449736e-05, u'good/bett': 4.879953152449736e-05, u'internation': 4.879953152449736e-05}, 'unix': {u'"distant"': 0.021530585907252861, u'ascend': 0.043039662737675302, u'distanc': 0.043039662737675302, u'resourc': 0.043039662737675302, u'"distance"': 0.021530585907252861, u'c/c++': 0.043039662737675302, u'question': 0.043039662737675302, u'books/tutori': 0.021530585907252861, u'list': 0.15058504688978749, u'metric': 0.043039662737675302, u'space': 0.043039662737675302, u'rubi': 0.043039662737675302, u'three': 0.021530585907252861, u'free': 0.021530585907252861, u'repres': 0.043039662737675302, u'accur': 0.021530585907252861, u'distant': 0.021530585907252861, u'educ': 0.021530585907252861, u'sum': 0.043039662737675302, u'vision': 0.043039662737675302}, 'linear-algebra': {u'imagin': 7.4117995849392229e-05, u'account': 0.074192113845241617, u'set': 7.4117995849392229e-05, u'postgresql': 0.074192113845241617, u'bpgsql': 0.074192113845241617, u'sourcesaf': 7.4117995849392229e-05, u'claim': 7.4117995849392229e-05, u'internation': 7.4117995849392229e-05, u'famou': 7.4117995849392229e-05, u'breed': 7.4117995849392229e-05, u'host': 0.14831010969463385, u'pysql': 0.074192113845241617, u'minimum': 7.4117995849392229e-05, u'abil': 0.074192113845241617, u'breakpoint': 7.4117995849392229e-05, u'instal': 0.2965461013934183, u'unicod': 7.4117995849392229e-05, u'phpeclips': 7.4117995849392229e-05, u'grow': 7.4117995849392229e-05, u'excus': 7.4117995849392229e-05}, 'cloud': {u'imagin': 7.4117995849392229e-05, u'set': 7.4117995849392229e-05, u'cross-cultur': 7.4117995849392229e-05, u'sourcesaf': 7.4117995849392229e-05, u'claim': 7.4117995849392229e-05, u'internation': 7.4117995849392229e-05, u'h': 0.14831010969463385, u'fall': 0.074192113845241617, u'famou': 7.4117995849392229e-05, u'breakpoint': 7.4117995849392229e-05, u'header': 0.074192113845241617, u'minimum': 7.4117995849392229e-05, u'rememb': 0.074192113845241617, u'doc': 0.074192113845241617, u'breed': 7.4117995849392229e-05, u'array': 0.22242810554402606, u'dream': 0.074192113845241617, u'grow': 7.4117995849392229e-05, u'good/bett': 7.4117995849392229e-05, u'attribut': 0.074192113845241617}, 'regex': {u'imagin': 6.9003588186585703e-05, u'set': 6.9003588186585703e-05, u'cross-cultur': 6.9003588186585703e-05, u'sourcesaf': 6.9003588186585703e-05, u'claim': 6.9003588186585703e-05, u'process': 0.27608335633452941, u'anoth': 0.13807617996135799, u'breed': 6.9003588186585703e-05, u'famou': 6.9003588186585703e-05, u'internation': 6.9003588186585703e-05, u'perhap': 0.069072591774772277, u'minimum': 6.9003588186585703e-05, u'suspect': 0.069072591774772277, u'file': 0.27608335633452941, u'breakpoint': 6.9003588186585703e-05, u'good/bett': 6.9003588186585703e-05, u'unicod': 6.9003588186585703e-05, u'phpeclips': 6.9003588186585703e-05, u'grow': 6.9003588186585703e-05, u'excus': 6.9003588186585703e-05}, 'chm': {u'imagin': 0.00010535187526337968, u'set': 0.00010535187526337968, u'cross-cultur': 0.00010535187526337968, u'sourcesaf': 0.00010535187526337968, u'tester': 0.00010535187526337968, u'internation': 0.00010535187526337968, u'famou': 0.00010535187526337968, u'breed': 0.00010535187526337968, u'c++': 0.10545722713864304, u'claim': 0.00010535187526337968, u'good/bett': 0.00010535187526337968, u'unlik': 0.10545722713864304, u'minimum': 0.00010535187526337968, u'ensur': 0.21080910240202272, u'unicod': 0.00010535187526337968, u'phpeclips': 0.00010535187526337968, u'judgment': 0.10545722713864304, u'grow': 0.00010535187526337968, u'try/catch': 0.21080910240202272, u'excus': 0.00010535187526337968}, 'iphone': {u'machin': 0.047091217170290876, u'roi': 0.023557375505977595, u'msdn': 0.047091217170290876, u'boatload': 0.023557375505977595, u'"solution"': 0.023557375505977595, u'third': 0.023557375505977595, u'transit': 0.023557375505977595, u'smooth': 0.023557375505977595, u'@rob': 0.023557375505977595, u'burk': 0.023557375505977595, u'boss': 0.023557375505977595, u'tech': 0.023557375505977595, u'troubl': 0.023557375505977595, u'elev': 0.023557375505977595, u'gave': 0.023557375505977595, u'desktop': 0.023557375505977595, u'strictli': 0.023557375505977595, u'custom': 0.070625058834604168, u'ie': 0.023557375505977595, u'~': 0.023557375505977595}, 'dns': {u'hone': 0.044504712786768624, u'tester': 4.4460252534234394e-05, u'skill': 0.044504712786768624, u'sourc': 0.13342521785523742, u'theoriz': 0.044504712786768624, u'sourcesaf': 4.4460252534234394e-05, u'baggag': 4.4460252534234394e-05, u'commun': 0.044504712786768624, u'weekend': 0.044504712786768624, u'jeff': 0.044504712786768624, u'rubi': 0.13342521785523742, u'non-invit': 4.4460252534234394e-05, u'stop': 0.044504712786768624, u'aw': 4.4460252534234394e-05, u'lieu': 0.044504712786768624, u'mistak': 0.044504712786768624, u'studi': 0.044504712786768624, u'open': 0.13342521785523742, u'cool/interest': 0.044504712786768624, u'simpler': 4.4460252534234394e-05}, 'memory': {u'bandwidth;': 0.01836966894223005, u'project': 0.073423621816046405, u'alloc': 0.01836966894223005, u'disabl': 0.01836966894223005, u'ten': 0.01836966894223005, u'power': 0.055072304191440949, u'memori': 0.055072304191440949, u'horrend': 0.01836966894223005, u'noticed;': 0.01836966894223005, u'ration': 0.01836966894223005, u'rest': 0.01836966894223005, u'compon': 0.036720986566835499, u'caus': 0.091774939440651848, u'rewrit': 0.036720986566835499, u'easier': 0.01836966894223005, u'mung': 0.01836966894223005, u'win;': 0.01836966894223005, u'verifi': 0.01836966894223005, u'ship': 0.036720986566835499, u'minut': 0.01836966894223005}, 'annotations': {u'function': 0.08170014698677118, u'properli': 0.040870488322717613, u'set': 4.0829658664053568e-05, u'thread': 0.08170014698677118, u'internation': 4.0829658664053568e-05, u'famou': 4.0829658664053568e-05, u'servic': 0.12252980565082475, u'minimum': 4.0829658664053568e-05, u'system': 0.040870488322717613, u'yield': 0.040870488322717613, u'tight': 0.040870488322717613, u'service-typ': 0.040870488322717613, u'empti': 0.040870488322717613, u'call': 0.08170014698677118, u'overkil': 0.040870488322717613, u'print': 0.16335946431487833, u'preview': 0.040870488322717613, u'unicod': 4.0829658664053568e-05, u'null': 0.040870488322717613, u'excus': 4.0829658664053568e-05}, 'iis': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'scheduled-tasks': {u'cron': 0.033941407839414073, u'task': 0.10175640851756408, u'feel': 0.033941407839414073, u'schedul': 0.16957140919571409, u'servic': 0.06784890817848907, u'situat': 0.033941407839414073, u'up': 0.033941407839414073, u'unfamiliar': 0.033941407839414073, u'corral': 0.033941407839414073, u'down': 0.033941407839414073, u'schtask': 0.033941407839414073, u'overhead': 0.033941407839414073, u'unaccept': 0.033941407839414073, u'@sasb': 0.033941407839414073, u'away': 0.033941407839414073, u'havent': 0.06784890817848907, u'phpeclips': 3.3907500339075e-05, u'clear': 0.033941407839414073, u'piec': 0.033941407839414073, u'@karl': 0.033941407839414073}, 'windows-nt': {u'case': 0.080131284021773927, u'claim': 8.0051232788984944e-05, u'mfc': 0.080131284021773927, u'cross-cultur': 8.0051232788984944e-05, u'sourcesaf': 8.0051232788984944e-05, u'workaround': 0.080131284021773927, u'nt': 0.16018251681075887, u'breed': 8.0051232788984944e-05, u'c++': 0.080131284021773927, u'caus': 0.16018251681075887, u'ensur': 8.0051232788984944e-05, u'minimum': 8.0051232788984944e-05, u'breakpoint': 8.0051232788984944e-05, u'good/bett': 8.0051232788984944e-05, u'_afxdll': 0.080131284021773927, u'unicod': 8.0051232788984944e-05, u'phpeclips': 8.0051232788984944e-05, u'true': 0.080131284021773927, u'grow': 8.0051232788984944e-05, u'excus': 8.0051232788984944e-05}, 'oop': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'nested-class': {u'claim': 4.4460252534234394e-05, u'sourcesaf': 4.4460252534234394e-05, u'scenario': 0.044504712786768624, u'collect': 0.044504712786768624, u'public': 0.044504712786768624, u'tester': 4.4460252534234394e-05, u'justif': 0.044504712786768624, u'c++': 0.044504712786768624, u'mine': 0.088964965321003009, u'record': 0.044504712786768624, u'imagin': 4.4460252534234394e-05, u'playback': 0.044504712786768624, u'breakpoint': 4.4460252534234394e-05, u'wari': 0.044504712786768624, u'good/bett': 4.4460252534234394e-05, u'grow': 4.4460252534234394e-05, u'phpeclips': 4.4460252534234394e-05, u'baggag': 4.4460252534234394e-05, u'class': 0.40018673306064373, u'conflict': 0.044504712786768624}, 'account': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'phing': {u'web': 0.1539605992201929, u'sourc': 0.10265750051303098, u'cross-cultur': 5.130309870716191e-05, u'minimum': 5.130309870716191e-05, u'confirm': 0.051354401805869067, u'famou': 5.130309870716191e-05, u'tip': 5.130309870716191e-05, u'ensur': 5.130309870716191e-05, u'server': 0.10265750051303098, u'releas': 0.051354401805869067, u'excus': 5.130309870716191e-05, u'snapshot': 0.10265750051303098, u'reiter': 5.130309870716191e-05, u'clean': 0.051354401805869067, u'test': 0.10265750051303098, u'output': 5.130309870716191e-05, u'unicod': 5.130309870716191e-05, u'one-step': 0.051354401805869067, u'config': 0.051354401805869067, u'css': 0.051354401805869067}, 'try-catch': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'embedded-database': {u'perfect': 0.10545722713864304, u'imagin': 0.00010535187526337968, u'breed': 0.00010535187526337968, u'code': 0.42151285292878216, u'cross-cultur': 0.00010535187526337968, u'unicod': 0.00010535187526337968, u'sourcesaf': 0.00010535187526337968, u'tester': 0.00010535187526337968, u'internation': 0.00010535187526337968, u'famou': 0.00010535187526337968, u'properti': 0.10545722713864304, u'claim': 0.00010535187526337968, u'minimum': 0.00010535187526337968, u'breakpoint': 0.00010535187526337968, u'baggag': 0.00010535187526337968, u'good/bett': 0.00010535187526337968, u'ctrl+space': 0.10545722713864304, u'phpeclips': 0.00010535187526337968, u'grow': 0.00010535187526337968, u'excus': 0.00010535187526337968}, 'setup': {u'claim': 5.7168991538989253e-05, u'excus': 5.7168991538989253e-05, u'cross-cultur': 5.7168991538989253e-05, u'possibl': 0.17156414360850672, u'entri': 0.11439515206951748, u'setup': 0.17156414360850672, u'breed': 5.7168991538989253e-05, u'famou': 5.7168991538989253e-05, u'tip': 5.7168991538989253e-05, u'ensur': 5.7168991538989253e-05, u'perhap': 0.057226160530528231, u'minimum': 5.7168991538989253e-05, u'five': 0.057226160530528231, u'breakpoint': 5.7168991538989253e-05, u'good/bett': 5.7168991538989253e-05, u'user/machin': 0.11439515206951748, u'havent': 0.057226160530528231, u'phpeclips': 5.7168991538989253e-05, u'root': 0.11439515206951748, u'unicod': 5.7168991538989253e-05}, 'eclipse': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'data-dictionary': {u'//': 0.051354401805869067, u'pull': 0.051354401805869067, u'breed': 5.130309870716191e-05, u'locat': 0.10265750051303098, u'goal': 0.051354401805869067, u'claim': 5.130309870716191e-05, u'imagin': 5.130309870716191e-05, u'rout': 0.051354401805869067, u'assist': 0.051354401805869067, u'eas': 0.051354401805869067, u'obtain': 0.10265750051303098, u'compon': 0.051354401805869067, u'offic': 0.051354401805869067, u'breakpoint': 5.130309870716191e-05, u'sens': 0.051354401805869067, u'modi': 0.051354401805869067, u'along': 0.051354401805869067, u'phpeclips': 5.130309870716191e-05, u'went': 0.051354401805869067, u'updat': 0.051354401805869067}, 'winforms': {u'redirect': 0.019824922760041195, u'set': 0.059435158044838787, u'somewhat': 0.019824922760041195, u'c#': 0.15846074625683276, u'lock': 0.13865562861443398, u'asmx': 0.059435158044838787, u'systemioioexcept': 0.019824922760041195, u'mis-behaviour': 1.9805117642398797e-05, u'delet': 0.059435158044838787, u'mvc': 0.039630040402439989, u'retrytim': 0.019824922760041195, u'block': 0.019824922760041195, u'way': 0.13865562861443398, u'track': 0.019824922760041195, u'bunch': 0.019824922760041195, u'conveni': 0.039630040402439989, u'paragraph': 1.9805117642398797e-05, u'class': 0.059435158044838787, u'try/catch': 0.039630040402439989, u'network': 0.019824922760041195}, 'scripting': {u'control': 0.17412112774103722, u'info': 0.17412112774103722, u'set': 8.7017055342847191e-05, u'cross-cultur': 8.7017055342847191e-05, u'sourcesaf': 8.7017055342847191e-05, u'form': 0.17412112774103722, u'imagin': 8.7017055342847191e-05, u'remot': 0.087104072398190027, u'queri': 0.087104072398190027, u'gather': 0.087104072398190027, u'famou': 8.7017055342847191e-05, u'claim': 8.7017055342847191e-05, u'internation': 8.7017055342847191e-05, u'minimum': 8.7017055342847191e-05, u'breakpoint': 8.7017055342847191e-05, u'good/bett': 8.7017055342847191e-05, u'unicod': 8.7017055342847191e-05, u'phpeclips': 8.7017055342847191e-05, u'grow': 8.7017055342847191e-05, u'excus': 8.7017055342847191e-05}, 'xfdl': {u'imagin': 0.00011775788977861516, u'breed': 0.00011775788977861516, u'cross-cultur': 0.00011775788977861516, u'aw': 0.00011775788977861516, u'sourcesaf': 0.00011775788977861516, u'restor': 0.11787564766839376, u'internation': 0.00011775788977861516, u'backup': 0.11787564766839376, u'tester': 0.00011775788977861516, u'famou': 0.00011775788977861516, u'empti': 0.11787564766839376, u'claim': 0.00011775788977861516, u'breakpoint': 0.00011775788977861516, u'baggag': 0.00011775788977861516, u'good/bett': 0.00011775788977861516, u'databa': 0.11787564766839376, u'phpeclips': 0.00011775788977861516, u'complic': 0.11787564766839376, u'grow': 0.00011775788977861516, u'prone': 0.11787564766839376}, 'sql-injection': {u'malici': 0.097647862580519218, u'regard': 0.048848331056021856, u'higher': 0.048848331056021856, u'sit': 0.048848331056021856, u'lead': 0.048848331056021856, u'though': 0.048848331056021856, u'prevent': 0.048848331056021856, u'famou': 4.879953152449736e-05, u'phpeclips': 4.879953152449736e-05, u'level': 0.097647862580519218, u'request': 0.14644739410501659, u'cross-platform': 0.048848331056021856, u'minimum': 4.879953152449736e-05, u'internation': 4.879953152449736e-05, u'bloat': 0.048848331056021856, u'treat': 0.048848331056021856, u'unicod': 4.879953152449736e-05, u'analyt': 0.048848331056021856, u'excus': 4.879953152449736e-05, u'web-serv': 0.048848331056021856}, 'apple-touch-icon': {u'dimens': 0.069072591774772277, u'non-invit': 6.9003588186585703e-05, u'repeat': 0.069072591774772277, u'cite': 0.069072591774772277, u'sourcesaf': 6.9003588186585703e-05, u'tester': 6.9003588186585703e-05, u'breed': 6.9003588186585703e-05, u'paragraph': 6.9003588186585703e-05, u'apple-touch-icon': 0.13807617996135799, u'correct': 0.13807617996135799, u'claim': 6.9003588186585703e-05, u'broken': 0.069072591774772277, u'imagin': 6.9003588186585703e-05, u'appl': 0.069072591774772277, u'breakpoint': 6.9003588186585703e-05, u'applecom': 0.069072591774772277, u'grow': 6.9003588186585703e-05, u'reiter': 6.9003588186585703e-05, u'pixel': 0.13807617996135799, u'simpler': 6.9003588186585703e-05}, 'ascii': {u'eventu': 0.048848331056021856, u'importantli': 0.048848331056021856, u'fair': 4.879953152449736e-05, u'older': 0.048848331056021856, u'--': 0.048848331056021856, u'dreamweav': 0.048848331056021856, u'painless': 0.048848331056021856, u'curiou': 0.048848331056021856, u'act': 0.048848331056021856, u'caus': 0.097647862580519218, u'english': 4.879953152449736e-05, u'correl': 0.048848331056021856, u'explorerex': 0.048848331056021856, u'abnorm': 0.048848331056021856, u'varieti': 0.048848331056021856, u'world': 4.879953152449736e-05, u'shell': 0.048848331056021856, u'buggi': 0.048848331056021856, u'strang': 0.048848331056021856, u'editor': 0.048848331056021856}, 'mapi': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'maps': {u'imagin': 9.5310712924132673e-05, u'excus': 9.5310712924132673e-05, u'cross-cultur': 9.5310712924132673e-05, u'sourcesaf': 9.5310712924132673e-05, u'claim': 9.5310712924132673e-05, u'newbi': 0.095406023637056789, u'famou': 9.5310712924132673e-05, u'phpeclips': 9.5310712924132673e-05, u'breed': 9.5310712924132673e-05, u'minimum': 9.5310712924132673e-05, u'internation': 9.5310712924132673e-05, u'realiz': 0.095406023637056789, u'strongly-typ': 0.095406023637056789, u'breakpoint': 9.5310712924132673e-05, u'good/bett': 9.5310712924132673e-05, u'import': 0.19071673656118945, u'unicod': 9.5310712924132673e-05, u'csv': 0.19071673656118945, u'=': 0.095406023637056789, u'grow': 9.5310712924132673e-05}, 'globalization': {u'flaw': 0.02902122231242027, u'algorithm': 0.02902122231242027, u'magic': 0.02902122231242027, u'auto-detect': 0.02902122231242027, u'windows/brows': 0.02902122231242027, u'framework': 0.058013452394758205, u'mm/dd/yyyy': 0.02902122231242027, u'format': 0.02902122231242027, u'number': 0.058013452394758205, u'charact': 0.02902122231242027, u'descriptor': 0.058013452394758205, u'minimum': 2.8992230082337937e-05, u'pars': 0.087005682477096141, u'digit': 0.02902122231242027, u'cultur': 0.11599791255943409, u'date': 0.11599791255943409, u'discard': 0.087005682477096141, u'backward': 0.02902122231242027, u'interpret': 0.02902122231242027, u'appear': 0.02902122231242027}, '.net-3.5': {u'web': 0.2410344162730165, u'breed': 2.4101031524149236e-05, u'homegrown': 0.048226164079822616, u'reproduc': 0.024125132555673383, u'advantag': 0.024125132555673383, u'documenttyp': 0.024125132555673383, u'numer': 0.024125132555673383, u'auto': 0.024125132555673383, u'servic': 0.21693338474886725, u'consum': 0.048226164079822616, u'share': 0.024125132555673383, u'signedworkord': 0.024125132555673383, u'claim': 2.4101031524149236e-05, u'output': 2.4101031524149236e-05, u'oppos': 0.024125132555673383, u'consumpt': 0.024125132555673383, u'type': 0.12052925865227034, u'grow': 2.4101031524149236e-05, u'transport': 0.024125132555673383, u'popul': 0.024125132555673383}, 'nhibernate': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'csv': {u'xml': 0.3663079003719148, u'claim': 2.8175363462188664e-05, u'convert': 0.056378902287839518, u'uncertain': 0.02820353882565085, u'whilst': 0.02820353882565085, u'internation': 2.8175363462188664e-05, u'famou': 2.8175363462188664e-05, u'pro': 0.02820353882565085, u'piec': 0.02820353882565085, u'construct': 0.02820353882565085, u'unfortun': 0.02820353882565085, u'fairli': 0.056378902287839518, u'build': 0.02820353882565085, u'file': 0.11272962921221685, u'good/bett': 2.8175363462188664e-05, u'cross-cultur': 2.8175363462188664e-05, u'model': 0.056378902287839518, u'skill': 0.02820353882565085, u'expand': 0.02820353882565085, u'con': 0.02820353882565085}, 'css': {u'ui-typ': 0.026005403720253558, u'web': 0.15590252520004158, u'bad-guy': 2.5979424295957604e-05, u'set': 0.026005403720253558, u'appear': 0.077964252312168761, u'creat': 0.051984828016211161, u'constraint': 0.026005403720253558, u'url': 0.026005403720253558, u'make': 0.026005403720253558, u'free': 0.026005403720253558, u'small': 0.026005403720253558, u'imagin': 2.5979424295957604e-05, u'html': 0.1818819494959992, u'link': 0.051984828016211161, u'dialogarguments___ie_printtyp': 0.026005403720253558, u'download': 0.051984828016211161, u'environ': 0.026005403720253558, u'res//ieframedll/previewdlg': 0.026005403720253558, u'css': 0.051984828016211161, u'browser': 0.051984828016211161}, 'profile': {u'imagin': 8.7017055342847191e-05, u'breed': 8.7017055342847191e-05, u'set': 8.7017055342847191e-05, u'cross-cultur': 8.7017055342847191e-05, u'sourcesaf': 8.7017055342847191e-05, u'tester': 8.7017055342847191e-05, u'internation': 8.7017055342847191e-05, u'famou': 8.7017055342847191e-05, u'breakpoint': 8.7017055342847191e-05, u'wikipedia': 0.087104072398190027, u'research': 0.17412112774103722, u'claim': 8.7017055342847191e-05, u'calcul': 0.26113818308388442, u'minimum': 8.7017055342847191e-05, u'result': 0.17412112774103722, u'background': 0.087104072398190027, u'baggag': 8.7017055342847191e-05, u'good/bett': 8.7017055342847191e-05, u'phpeclips': 8.7017055342847191e-05, u'grow': 8.7017055342847191e-05}, 'msdn': {u'claim': 4.879953152449736e-05, u'assumpt': 0.048848331056021856, u'cross-cultur': 4.879953152449736e-05, u'linear': 0.097647862580519218, u'point': 0.19524692562951396, u'famou': 4.879953152449736e-05, u'breed': 4.879953152449736e-05, u'disagre': 0.048848331056021856, u'transform': 0.048848331056021856, u'distort': 0.19524692562951396, u'nearbi': 0.048848331056021856, u'creativ': 0.048848331056021856, u'minimum': 4.879953152449736e-05, u'lat/long': 0.048848331056021856, u'breakpoint': 4.879953152449736e-05, u'good/bett': 4.879953152449736e-05, u'phpeclips': 4.879953152449736e-05, u'assum': 0.048848331056021856, u'grow': 4.879953152449736e-05, u'bunch': 0.048848331056021856}, 'mapping': {u'wont': 0.10545722713864304, u'imagin': 0.00010535187526337968, u'vb': 0.10545722713864304, u'legaci': 0.10545722713864304, u'cross-cultur': 0.00010535187526337968, u'oledb': 0.21080910240202272, u'sourcesaf': 0.00010535187526337968, u'claim': 0.00010535187526337968, u'internation': 0.00010535187526337968, u'famou': 0.00010535187526337968, u'breed': 0.00010535187526337968, u'minimum': 0.00010535187526337968, u'research': 0.10545722713864304, u'set': 0.00010535187526337968, u'breakpoint': 0.00010535187526337968, u'good/bett': 0.00010535187526337968, u'unicod': 0.00010535187526337968, u'phpeclips': 0.00010535187526337968, u'grow': 0.00010535187526337968, u'wrote': 0.10545722713864304}, 'ibm-midrange': {u'represent': 0.095406023637056789, u'reiter': 9.5310712924132673e-05, u'openbsd': 0.095406023637056789, u'sourcesaf': 9.5310712924132673e-05, u'organ': 9.5310712924132673e-05, u'tester': 9.5310712924132673e-05, u'breed': 9.5310712924132673e-05, u'ibm_db': 0.095406023637056789, u'wherein': 0.095406023637056789, u'claim': 9.5310712924132673e-05, u'json': 0.095406023637056789, u'paragraph': 9.5310712924132673e-05, u'aw': 9.5310712924132673e-05, u'baggag': 9.5310712924132673e-05, u'grow': 9.5310712924132673e-05, u'simpler': 9.5310712924132673e-05, u'as/': 0.095406023637056789, u'non-invit': 9.5310712924132673e-05, u'store': 0.095406023637056789, u'luck': 0.095406023637056789}, 'date': {u'compile-tim': 0.064613994319648843, u'set': 6.4549444874774072e-05, u'cross-cultur': 6.4549444874774072e-05, u'sourcesaf': 6.4549444874774072e-05, u'tester': 6.4549444874774072e-05, u'may': 0.12916343919442291, u'breed': 6.4549444874774072e-05, u'object': 0.19371288406919698, u'famou': 6.4549444874774072e-05, u'minimum': 6.4549444874774072e-05, u'claim': 6.4549444874774072e-05, u'internation': 6.4549444874774072e-05, u'imagin': 6.4549444874774072e-05, u'instanc': 0.19371288406919698, u'breakpoint': 6.4549444874774072e-05, u'good/bett': 6.4549444874774072e-05, u'unicod': 6.4549444874774072e-05, u'type': 0.25826232894397111, u'grow': 6.4549444874774072e-05, u'excus': 6.4549444874774072e-05}, 'compatibility': {u'"collect': 0.02902122231242027, u'sourc': 0.02902122231242027, u'function/class': 0.02902122231242027, u'mersenn': 0.02902122231242027, u'random': 0.11599791255943409, u'rng': 0.058013452394758205, u'equal': 0.02902122231242027, u'c++': 0.02902122231242027, u'linux/etc': 0.02902122231242027, u'high-ord': 0.02902122231242027, u'mathemat': 0.02902122231242027, u'c-languag': 0.02902122231242027, u'whatev': 0.02902122231242027, u'entropy"': 0.02902122231242027, u'hardwar': 0.02902122231242027, u'robust': 0.058013452394758205, u'bit': 0.11599791255943409, u'twister': 0.02902122231242027, u'special': 0.02902122231242027, u'low-ord': 0.02902122231242027}, 'types': {u'meant': 0.051354401805869067, u'claim': 5.130309870716191e-05, u'breed': 5.130309870716191e-05, u'concept': 0.051354401805869067, u'cross-cultur': 5.130309870716191e-05, u'insid': 0.051354401805869067, u'tester': 5.130309870716191e-05, u'internation': 5.130309870716191e-05, u'curiou': 0.051354401805869067, u'nest': 0.20526369792735483, u'breakpoint': 5.130309870716191e-05, u'famou': 5.130309870716191e-05, u'workhors': 0.10265750051303098, u'imagin': 5.130309870716191e-05, u'book': 0.10265750051303098, u'decod': 0.051354401805869067, u'benefits/drawback': 0.051354401805869067, u'good/bett': 5.130309870716191e-05, u'video': 0.1539605992201929, u'grow': 5.130309870716191e-05}, 'windows-services': {u'imagin': 0.00011775788977861516, u'set': 0.00011775788977861516, u'cross-cultur': 0.00011775788977861516, u'sourcesaf': 0.00011775788977861516, u'launch': 0.23563353744700891, u'internation': 0.00011775788977861516, u'iemobil': 0.11787564766839376, u'tester': 0.00011775788977861516, u'favorit': 0.23563353744700891, u'breakpoint': 0.00011775788977861516, u'claim': 0.00011775788977861516, u'good/bett': 0.00011775788977861516, u'minimum': 0.00011775788977861516, u'aw': 0.00011775788977861516, u'baggag': 0.00011775788977861516, u'breed': 0.00011775788977861516, u'famou': 0.00011775788977861516, u'phpeclips': 0.00011775788977861516, u'mobil': 0.11787564766839376, u'grow': 0.00011775788977861516}, 'ide': {u'precompil': 0.036410592172268288, u'zendcor': 0.036410592172268288, u'via': 0.036410592172268288, u'run': 0.18190746398952423, u'ibm': 0.14553324603521026, u'php/apach': 0.036410592172268288, u'tester': 3.6374217954313981e-05, u'vr': 0.036410592172268288, u'/openbsd': 0.036410592172268288, u'non-invit': 3.6374217954313981e-05, u'roadblock': 0.036410592172268288, u'extens': 0.10915902808089625, u'second': 0.072784810126582278, u'connect': 0.072784810126582278, u'aw': 3.6374217954313981e-05, u'baggag': 3.6374217954313981e-05, u'toddler': 3.6374217954313981e-05, u'back-end': 0.036410592172268288, u'simpler': 3.6374217954313981e-05, u'overkil': 0.036410592172268288}, 'documentation': {u'comment': 0.046575469942304104, u'excus': 4.6528941001302811e-05, u'sandcastl': 0.13963335194490972, u'xslt': 0.046575469942304104, u'process': 0.093104410943606913, u'famou': 4.6528941001302811e-05, u'breakpoint': 4.6528941001302811e-05, u'minimum': 4.6528941001302811e-05, u'cross': 0.093104410943606913, u'reiter': 0.046575469942304104, u'bridg': 0.046575469942304104, u'somebodi': 0.046575469942304104, u'dead-simpl': 0.046575469942304104, u'baggag': 0.046575469942304104, u'ensur': 4.6528941001302811e-05, u'eye': 0.046575469942304104, u'unicod': 4.6528941001302811e-05, u'nice': 0.093104410943606913, u'simpler': 0.046575469942304104, u'gave': 0.046575469942304104}, 'django': {u'underli': 0.042610250297973773, u'map': 0.085177932913332186, u'ajax-i': 0.042610250297973773, u'imagin': 4.2567682615358418e-05, u'color': 0.042610250297973773, u'decent': 0.042610250297973773, u'imag': 0.042610250297973773, u'subscrib': 0.042610250297973773, u'cacheabl': 0.042610250297973773, u'gis/map': 0.042610250297973773, u'compon': 0.085177932913332186, u'state': 0.085177932913332186, u'static': 0.042610250297973773, u'shown': 0.042610250297973773, u'hover/click': 0.042610250297973773, u'wont': 0.042610250297973773, u'presum': 0.042610250297973773, u'imagemap': 0.042610250297973773, u'facil': 0.042610250297973773, u'grow': 4.2567682615358418e-05}, 'vfp': {u'imagin': 0.00015403573629081948, u'tester': 0.00015403573629081948, u'set': 0.00015403573629081948, u'cross-cultur': 0.00015403573629081948, u'sourcesaf': 0.00015403573629081948, u'foxpro': 0.15418977202711026, u'claim': 0.00015403573629081948, u'internation': 0.00015403573629081948, u'famou': 0.00015403573629081948, u'breed': 0.00015403573629081948, u'excus': 0.00015403573629081948, u'messag': 0.15418977202711026, u'minimum': 0.00015403573629081948, u'breakpoint': 0.00015403573629081948, u'good/bett': 0.00015403573629081948, u'unicod': 0.00015403573629081948, u'phpeclips': 0.00015403573629081948, u'baggag': 0.00015403573629081948, u'grow': 0.00015403573629081948, u'vfp': 0.30822550831792972}, 'photoshop': {u'photoshop': 0.075532236146761286, u'guess': 0.037784991695606215, u'detect': 0.11327948059791634, u'cross-cultur': 3.7747244451155068e-05, u'phpeclips': 3.7747244451155068e-05, u'imagin': 3.7747244451155068e-05, u'internation': 3.7747244451155068e-05, u'famou': 3.7747244451155068e-05, u'breed': 3.7747244451155068e-05, u'minimum': 3.7747244451155068e-05, u'rule': 0.037784991695606215, u'grow': 3.7747244451155068e-05, u'claim': 3.7747244451155068e-05, u'set': 3.7747244451155068e-05, u'suppos': 0.037784991695606215, u'breakpoint': 3.7747244451155068e-05, u'unicod': 3.7747244451155068e-05, u'font': 0.41525743620715683, u'display': 0.15102672504907141, u'glyph': 0.037784991695606215}, 'code-generation': {u'absolut': 0.14831010969463385, u'set': 7.4117995849392229e-05, u'excus': 7.4117995849392229e-05, u'claim': 7.4117995849392229e-05, u'internation': 7.4117995849392229e-05, u'famou': 7.4117995849392229e-05, u'breed': 7.4117995849392229e-05, u'tip': 7.4117995849392229e-05, u'ensur': 7.4117995849392229e-05, u'percentag': 0.14831010969463385, u'good/bett': 7.4117995849392229e-05, u'minimum': 7.4117995849392229e-05, u'besid': 0.074192113845241617, u'breakpoint': 7.4117995849392229e-05, u'posit': 0.2965461013934183, u'iceberg': 7.4117995849392229e-05, u'unicod': 7.4117995849392229e-05, u'phpeclips': 7.4117995849392229e-05, u'pixel': 0.074192113845241617, u'collaps': 0.074192113845241617}, 'directx': {u'claim': 7.4117995849392229e-05, u'tester': 7.4117995849392229e-05, u'cube': 0.22242810554402606, u'aw': 7.4117995849392229e-05, u'sourcesaf': 7.4117995849392229e-05, u'guru': 0.074192113845241617, u'primit': 0.14831010969463385, u'paragraph': 7.4117995849392229e-05, u'excel': 7.4117995849392229e-05, u'reiter': 7.4117995849392229e-05, u'tutori': 0.074192113845241617, u'accomplish': 0.074192113845241617, u'although': 0.074192113845241617, u'baggag': 7.4117995849392229e-05, u'breed': 7.4117995849392229e-05, u'grow': 7.4117995849392229e-05, u'simpler': 7.4117995849392229e-05, u'non-invit': 7.4117995849392229e-05, u'directx': 0.074192113845241617, u'either': 0.074192113845241617}, 'gtk': {u'assembl': 0.11772320728071552, u'bonu': 0.03926722108896908, u'breed': 3.9227993095873212e-05, u'cross-cultur': 3.9227993095873212e-05, u'extra': 0.03926722108896908, u'internation': 3.9227993095873212e-05, u'famou': 3.9227993095873212e-05, u'hundr': 0.03926722108896908, u'maintainable/modd': 0.03926722108896908, u'per': 0.03926722108896908, u'back-end': 0.03926722108896908, u'game': 0.11772320728071552, u'minimum': 3.9227993095873212e-05, u'refresh': 0.03926722108896908, u'breakpoint': 3.9227993095873212e-05, u'good/bett': 3.9227993095873212e-05, u'add/chang': 0.03926722108896908, u'trade': 0.03926722108896908, u'class': 0.078495214184842302, u'card': 0.23540718656833517}, 'forms': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'multithreading': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'flex': {u'bytearray': 0.15418977202711026, u'imagin': 0.00015403573629081948, u'set': 0.00015403573629081948, u'cross-cultur': 0.00015403573629081948, u'success': 0.15418977202711026, u'sourcesaf': 0.00015403573629081948, u'tester': 0.00015403573629081948, u'internation': 0.00015403573629081948, u'famou': 0.00015403573629081948, u'breed': 0.00015403573629081948, u'claim': 0.00015403573629081948, u'aw': 0.00015403573629081948, u'minimum': 0.00015403573629081948, u'breakpoint': 0.00015403573629081948, u'actionscript': 0.30822550831792972, u'good/bett': 0.00015403573629081948, u'unicod': 0.00015403573629081948, u'phpeclips': 0.00015403573629081948, u'baggag': 0.00015403573629081948, u'grow': 0.00015403573629081948}, 'html-form': {u'imagin': 6.9003588186585703e-05, u'breed': 6.9003588186585703e-05, u'cross-cultur': 6.9003588186585703e-05, u'form': 0.27608335633452941, u'sourcesaf': 6.9003588186585703e-05, u'claim': 6.9003588186585703e-05, u'goe': 0.13807617996135799, u'famou': 6.9003588186585703e-05, u'wizard': 0.13807617996135799, u'excus': 6.9003588186585703e-05, u'ensur': 6.9003588186585703e-05, u'forward': 0.069072591774772277, u'minimum': 6.9003588186585703e-05, u'breakpoint': 6.9003588186585703e-05, u'press': 0.20707976814794368, u'unicod': 6.9003588186585703e-05, u'phpeclips': 6.9003588186585703e-05, u'grow': 6.9003588186585703e-05, u'good/bett': 6.9003588186585703e-05, u'internation': 6.9003588186585703e-05}, 'crash': {u'spawn': 0.087104072398190027, u'imagin': 8.7017055342847191e-05, u'sip': 0.087104072398190027, u'breakpoint': 8.7017055342847191e-05, u'sourcesaf': 8.7017055342847191e-05, u'tester': 8.7017055342847191e-05, u'breed': 8.7017055342847191e-05, u'baggag': 8.7017055342847191e-05, u'jpeg': 0.17412112774103722, u'non-invit': 8.7017055342847191e-05, u'claim': 8.7017055342847191e-05, u'mac': 0.087104072398190027, u'cross-platform': 0.087104072398190027, u'aw': 8.7017055342847191e-05, u'similarli': 0.087104072398190027, u'pdf': 0.17412112774103722, u'grow': 8.7017055342847191e-05, u'phpeclips': 8.7017055342847191e-05, u'unnecessari': 8.7017055342847191e-05, u'simpler': 8.7017055342847191e-05}, 'error-handling': {u'cross-cultur': 4.4460252534234394e-05, u'breed': 4.4460252534234394e-05, u'object': 0.088964965321003009, u'group': 0.088964965321003009, u'defaultweight': 0.044504712786768624, u'weight': 0.088964965321003009, u'internation': 4.4460252534234394e-05, u'famou': 4.4460252534234394e-05, u'properti': 0.088964965321003009, u'minimum': 4.4460252534234394e-05, u'good/bett': 4.4460252534234394e-05, u't_variabl': 0.044504712786768624, u'unexpect': 0.044504712786768624, u'shipment': 0.17788547038947183, u'breakpoint': 4.4460252534234394e-05, u'put': 0.088964965321003009, u'magic': 0.044504712786768624, u'"defaultweight"': 0.044504712786768624, u'phpeclips': 4.4460252534234394e-05, u'produc': 0.044504712786768624}, 'views': {u'imagin': 6.9003588186585703e-05, u'breed': 6.9003588186585703e-05, u'group': 0.13807617996135799, u'unicod': 6.9003588186585703e-05, u'claim': 6.9003588186585703e-05, u'internation': 6.9003588186585703e-05, u'famou': 6.9003588186585703e-05, u'million': 0.069072591774772277, u'ye': 0.069072591774772277, u'django': 0.13807617996135799, u'member': 0.069072591774772277, u'minimum': 6.9003588186585703e-05, u'statement': 0.069072591774772277, u'breakpoint': 6.9003588186585703e-05, u'good/bett': 6.9003588186585703e-05, u'cross-cultur': 6.9003588186585703e-05, u'grow': 6.9003588186585703e-05, u'phpeclips': 6.9003588186585703e-05, u'approach': 0.13807617996135799, u'display': 0.13807617996135799}, 'python': {u'imagin': 6.0635459616783893e-05, u'tester': 6.0635459616783893e-05, u'java': 0.060696095076400669, u'consensu': 0.060696095076400669, u'claim': 6.0635459616783893e-05, u'internation': 6.0635459616783893e-05, u'sun': 0.060696095076400669, u'breed': 6.0635459616783893e-05, u'microsystem': 0.060696095076400669, u'sign': 0.12133155469318456, u'conciou': 0.060696095076400669, u'decis': 0.060696095076400669, u'breakpoint': 6.0635459616783893e-05, u'resembl': 0.060696095076400669, u'good/bett': 6.0635459616783893e-05, u'opinion': 0.060696095076400669, u'phpeclips': 6.0635459616783893e-05, u'unsign': 0.18196701430996845, u'grow': 6.0635459616783893e-05, u'loop': 0.060696095076400669}, 'pass-by-value': {u'variabl': 0.19071673656118945, u'imagin': 9.5310712924132673e-05, u'set': 9.5310712924132673e-05, u'cross-cultur': 9.5310712924132673e-05, u'sourcesaf': 9.5310712924132673e-05, u'claim': 9.5310712924132673e-05, u'internation': 9.5310712924132673e-05, u'famou': 9.5310712924132673e-05, u'breed': 9.5310712924132673e-05, u'excus': 9.5310712924132673e-05, u'good/bett': 9.5310712924132673e-05, u'minimum': 9.5310712924132673e-05, u'breakpoint': 9.5310712924132673e-05, u'pass': 0.19071673656118945, u'ensur': 9.5310712924132673e-05, u'unicod': 9.5310712924132673e-05, u'phpeclips': 9.5310712924132673e-05, u'grow': 9.5310712924132673e-05, u'valu': 0.19071673656118945, u'refer': 0.19071673656118945}, 'indexing': {u'regex': 0.12704813920995808, u'pull': 0.031785850374698332, u'ace': 0.031785850374698332, u'evalu': 0.031785850374698332, u'imagin': 3.1754096278419917e-05, u'boost': 0.031785850374698332, u'express': 0.063539946653118254, u'c++': 0.063539946653118254, u'easy-to-us': 0.063539946653118254, u'claim': 3.1754096278419917e-05, u'standalon': 0.031785850374698332, u'tag': 0.063539946653118254, u'windows-specif': 0.031785850374698332, u'platform-independ': 0.031785850374698332, u'robust': 0.031785850374698332, u'sub-str': 0.031785850374698332, u'data': 0.15880223548837802, u'grow': 3.1754096278419917e-05, u'regular': 0.063539946653118254, u'unfortun': 0.031785850374698332}, 'safari': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'application-integration': {u'xml': 0.029887734384330584, u'non-invit': 2.9857876507822766e-05, u'old': 0.029887734384330584, u'practic': 0.029887734384330584, u'zip': 0.089603487399976117, u'whatev': 0.029887734384330584, u'class/tabl': 0.029887734384330584, u'built-in': 0.029887734384330584, u'user;': 0.029887734384330584, u'rout': 0.029887734384330584, u'profil': 0.20903499343126719, u'pros/con': 0.029887734384330584, u'stile': 0.029887734384330584, u'mechan': 0.029887734384330584, u'user': 0.17917711692344443, u'cross-cultur': 2.9857876507822766e-05, u'kept': 0.029887734384330584, u'probabl': 0.029887734384330584, u'guidanc': 0.029887734384330584, u'matrix': 0.029887734384330584}, 'permissions': {u'imagin': 8.0051232788984944e-05, u'lan': 0.080131284021773927, u'set': 8.0051232788984944e-05, u'cross-cultur': 8.0051232788984944e-05, u'consider': 0.080131284021773927, u'sourcesaf': 8.0051232788984944e-05, u'tester': 8.0051232788984944e-05, u'internation': 8.0051232788984944e-05, u'plain': 0.080131284021773927, u'phpeclips': 8.0051232788984944e-05, u'breed': 8.0051232788984944e-05, u'apach': 0.16018251681075887, u'one-off': 0.080131284021773927, u'claim': 8.0051232788984944e-05, u'non-internet': 0.080131284021773927, u'baggag': 8.0051232788984944e-05, u'good/bett': 8.0051232788984944e-05, u'famou': 8.0051232788984944e-05, u'php': 0.24023374959974381, u'grow': 8.0051232788984944e-05}, '.net': {u'claim': 8.0051232788984944e-05, u'set': 8.0051232788984944e-05, u'cross-cultur': 8.0051232788984944e-05, u'convers': 0.080131284021773927, u'sourcesaf': 8.0051232788984944e-05, u'imagin': 8.0051232788984944e-05, u'c#': 0.16018251681075887, u'famou': 8.0051232788984944e-05, u'tool': 0.16018251681075887, u'breed': 8.0051232788984944e-05, u'grow': 8.0051232788984944e-05, u'internation': 8.0051232788984944e-05, u'visual': 0.16018251681075887, u'minimum': 8.0051232788984944e-05, u'breakpoint': 8.0051232788984944e-05, u'good/bett': 8.0051232788984944e-05, u'unicod': 8.0051232788984944e-05, u'phpeclips': 8.0051232788984944e-05, u'port': 0.080131284021773927, u'j#': 0.16018251681075887}, 'odbc': {u'tri': 0.088964965321003009, u'explan': 0.044504712786768624, u'group': 0.088964965321003009, u'divid': 0.044504712786768624, u'python': 0.088964965321003009, u'-': 0.044504712786768624, u'review': 0.044504712786768624, u'itertoolsgroupbi': 0.044504712786768624, u'"prerequisite"': 0.044504712786768624, u'cross': 4.4460252534234394e-05, u'iter': 0.044504712786768624, u'element': 0.044504712786768624, u'tip': 4.4460252534234394e-05, u'criteria': 0.044504712786768624, u'ensur': 4.4460252534234394e-05, u'objectifi': 0.044504712786768624, u'havent': 0.044504712786768624, u'beyond': 0.044504712786768624, u'pointer': 0.044504712786768624, u'children': 0.044504712786768624}, 'x11': {u'imagin': 7.4117995849392229e-05, u'experienc': 0.074192113845241617, u'breed': 7.4117995849392229e-05, u'cross-cultur': 7.4117995849392229e-05, u'breakpoint': 7.4117995849392229e-05, u'sourcesaf': 7.4117995849392229e-05, u'tester': 7.4117995849392229e-05, u'internation': 7.4117995849392229e-05, u'sdl/opengl': 0.074192113845241617, u'phpeclips': 7.4117995849392229e-05, u'gtk': 0.2965461013934183, u'initialis': 0.074192113845241617, u'claim': 7.4117995849392229e-05, u'grow': 7.4117995849392229e-05, u'aw': 7.4117995849392229e-05, u'baggag': 7.4117995849392229e-05, u'good/bett': 7.4117995849392229e-05, u'implement': 0.14831010969463385, u'stay': 0.074192113845241617, u'realis': 0.074192113845241617}, 'adobe': {u'imagin': 9.5310712924132673e-05, u'tester': 9.5310712924132673e-05, u'breed': 9.5310712924132673e-05, u'cross-cultur': 9.5310712924132673e-05, u'sourcesaf': 9.5310712924132673e-05, u'stream': 0.095406023637056789, u'claim': 9.5310712924132673e-05, u'internation': 9.5310712924132673e-05, u'phpeclips': 9.5310712924132673e-05, u'imag': 0.095406023637056789, u'flash': 0.28602744948532211, u'famou': 9.5310712924132673e-05, u'grow': 9.5310712924132673e-05, u'minimum': 9.5310712924132673e-05, u'set': 9.5310712924132673e-05, u'breakpoint': 9.5310712924132673e-05, u'good/bett': 9.5310712924132673e-05, u'disk': 0.095406023637056789, u'baggag': 9.5310712924132673e-05, u'display': 0.19071673656118945}, 'stored-procedures': {u'huge': 0.027430669735832511, u'inclin': 0.027430669735832511, u'resourc': 0.027430669735832511, u'chain': 0.027430669735832511, u'navig': 0.027430669735832511, u'lose': 0.027430669735832511, u'databas': 0.16444700208264829, u'section': 0.082237202674558807, u'relat': 0.10964046914392198, u'grow': 2.7403266469363151e-05, u'imposit': 0.027430669735832511, u'easili': 0.027430669735832511, u'interrel': 0.027430669735832511, u'design': 0.054833936205195659, u'sens': 0.027430669735832511, u'linqtosql': 0.13704373561328512, u'big': 0.027430669735832511, u'necessari': 0.027430669735832511, u'easier': 0.027430669735832511, u'practic': 0.027430669735832511}, 'image-processing': {u'teeni': 0.087104072398190027, u'trainer': 0.087104072398190027, u'tester': 8.7017055342847191e-05, u'wiggl': 0.087104072398190027, u'sourcesaf': 8.7017055342847191e-05, u'imagin': 8.7017055342847191e-05, u'internation': 8.7017055342847191e-05, u'breed': 8.7017055342847191e-05, u'grow': 8.7017055342847191e-05, u'mous': 0.17412112774103722, u'paragraph': 8.7017055342847191e-05, u'aw': 8.7017055342847191e-05, u'breakpoint': 8.7017055342847191e-05, u'good/bett': 8.7017055342847191e-05, u'taskbar': 0.087104072398190027, u'phpeclips': 8.7017055342847191e-05, u'baggag': 8.7017055342847191e-05, u'pixel': 0.087104072398190027, u'today': 0.087104072398190027, u'icon': 0.087104072398190027}, 'loops': {u'claim': 8.0051232788984944e-05, u'tester': 8.0051232788984944e-05, u'purchas': 0.080131284021773927, u'honestli': 0.080131284021773927, u'sourcesaf': 8.0051232788984944e-05, u'imagin': 8.0051232788984944e-05, u'internation': 8.0051232788984944e-05, u'pay': 0.080131284021773927, u'breed': 8.0051232788984944e-05, u'mss': 0.080131284021773927, u'upgrad': 0.080131284021773927, u'famou': 8.0051232788984944e-05, u'host': 0.080131284021773927, u'suspect': 0.080131284021773927, u'cross-cultur': 8.0051232788984944e-05, u'breakpoint': 8.0051232788984944e-05, u'good/bett': 8.0051232788984944e-05, u'upsel': 0.080131284021773927, u'collat': 0.16018251681075887, u'baggag': 8.0051232788984944e-05}, 'email-spam': {u'blatantli': 0.048848331056021856, u'unicod': 4.879953152449736e-05, u'shuffl': 0.048848331056021856, u'spamish': 0.048848331056021856, u'imagin': 4.879953152449736e-05, u'big': 0.048848331056021856, u'human': 0.048848331056021856, u'licens': 0.048848331056021856, u'famou': 4.879953152449736e-05, u'terminolog': 0.048848331056021856, u'grow': 4.879953152449736e-05, u'tricki': 0.048848331056021856, u'reli': 0.048848331056021856, u'minimum': 4.879953152449736e-05, u'key': 0.048848331056021856, u'money': 0.048848331056021856, u'simpleton': 0.048848331056021856, u'cross-cultur': 4.879953152449736e-05, u'folder': 0.048848331056021856, u'email': 0.24404645715401133}, 'command-line': {u'claim': 9.5310712924132673e-05, u'shell': 0.28602744948532211, u'cross-cultur': 9.5310712924132673e-05, u'sourcesaf': 9.5310712924132673e-05, u'equival': 0.19071673656118945, u'imagin': 9.5310712924132673e-05, u'internation': 9.5310712924132673e-05, u'breed': 9.5310712924132673e-05, u'lack': 0.095406023637056789, u'famou': 9.5310712924132673e-05, u'ensur': 9.5310712924132673e-05, u'ii': 0.19071673656118945, u'minimum': 9.5310712924132673e-05, u'set': 9.5310712924132673e-05, u'breakpoint': 9.5310712924132673e-05, u'good/bett': 9.5310712924132673e-05, u'unicod': 9.5310712924132673e-05, u'phpeclips': 9.5310712924132673e-05, u'grow': 9.5310712924132673e-05, u'excus': 9.5310712924132673e-05}, 'preg-replace': {u'imagin': 9.5310712924132673e-05, u'capit': 0.095406023637056789, u'set': 9.5310712924132673e-05, u'word': 0.095406023637056789, u'cross-cultur': 9.5310712924132673e-05, u'sourcesaf': 9.5310712924132673e-05, u'tester': 9.5310712924132673e-05, u'internation': 9.5310712924132673e-05, u'preg_replac': 0.19071673656118945, u'breakpoint': 9.5310712924132673e-05, u'claim': 9.5310712924132673e-05, u'good/bett': 9.5310712924132673e-05, u'aw': 9.5310712924132673e-05, u'baggag': 9.5310712924132673e-05, u'wrap': 0.19071673656118945, u'span': 0.095406023637056789, u'phpeclips': 9.5310712924132673e-05, u'extract': 0.095406023637056789, u'grow': 9.5310712924132673e-05, u'simpler': 9.5310712924132673e-05}, 'generics': {u'late': 0.035132668819317696, u'permission-bas': 0.035132668819317696, u'good': 0.035132668819317696, u'essenti': 0.035132668819317696, u'spam': 0.21062052505966589, u'peopl': 0.10532781131545696, u'breed': 3.5097571248069633e-05, u'send': 0.1404253825635266, u'mark': 0.070230240067387331, u'topic': 0.035132668819317696, u'imagin': 3.5097571248069633e-05, u'set': 3.5097571248069633e-05, u'ignor': 0.035132668819317696, u'paid': 0.035132668819317696, u'clean': 0.035132668819317696, u'breakpoint': 3.5097571248069633e-05, u'good/bett': 3.5097571248069633e-05, u'implement': 0.035132668819317696, u'phpeclips': 3.5097571248069633e-05, u'wonder': 0.070230240067387331}, 'math': {u'altern': 0.019439912996193583, u'controlnet': 0.019439912996193583, u'strength': 0.019439912996193583, u'cross-cultur': 1.9420492503689893e-05, u'execut': 0.019439912996193583, u'primarili': 0.019439912996193583, u'internation': 1.9420492503689893e-05, u'stay': 0.019439912996193583, u'research': 0.038860405499883476, u'project': 0.13596286801833296, u'extens': 0.019439912996193583, u'visual': 0.19422434552940263, u'frontpag': 0.019439912996193583, u'studio': 0.19422434552940263, u'integr': 0.097121883010953175, u'repositori': 0.038860405499883476, u'team': 0.019439912996193583, u'svnex': 0.019439912996193583, u'tracking/comparison': 0.019439912996193583, u'ide': 0.038860405499883476}, 'build-process': {u'real': 0.046575469942304104, u'head': 0.046575469942304104, u'set': 4.6528941001302811e-05, u'excus': 4.6528941001302811e-05, u'you/should': 0.093104410943606913, u'internation': 4.6528941001302811e-05, u'databas': 0.093104410943606913, u'breed': 4.6528941001302811e-05, u'compress': 0.046575469942304104, u'autom': 0.093104410943606913, u'famou': 4.6528941001302811e-05, u'minimum': 4.6528941001302811e-05, u'much': 0.093104410943606913, u'build': 0.18616229294621256, u'breakpoint': 4.6528941001302811e-05, u'good/bett': 4.6528941001302811e-05, u'follow': 0.046575469942304104, u'unicod': 4.6528941001302811e-05, u'phpeclips': 4.6528941001302811e-05, u'take': 0.13963335194490972}, 'theory': {u'orient': 0.023557375505977595, u'oo': 0.023557375505977595, u'properti': 0.094158900498917453, u'iceberg': 2.3533841664313284e-05, u'appear': 0.047091217170290876, u'english': 2.3533841664313284e-05, u'classpatch': 0.023557375505977595, u'breed': 2.3533841664313284e-05, u'self': 0.047091217170290876, u'within': 0.094158900498917453, u'tip': 2.3533841664313284e-05, u'argument': 0.070625058834604168, u'class': 0.16476042549185732, u'outsid': 0.023557375505977595, u'breakpoint': 2.3533841664313284e-05, u'good/bett': 2.3533841664313284e-05, u'phpeclips': 2.3533841664313284e-05, u'true': 0.047091217170290876, u'method': 0.2588957921491104, u'variables/properti': 0.023557375505977595}, 'dictionary': {u'sort': 0.13807617996135799, u'imagin': 6.9003588186585703e-05, u'tester': 6.9003588186585703e-05, u'set': 6.9003588186585703e-05, u'cross-cultur': 6.9003588186585703e-05, u'sourcesaf': 6.9003588186585703e-05, u'claim': 6.9003588186585703e-05, u'internation': 6.9003588186585703e-05, u'frequenc': 0.20707976814794368, u'breed': 6.9003588186585703e-05, u'famou': 6.9003588186585703e-05, u'resort': 0.069072591774772277, u'minimum': 6.9003588186585703e-05, u'breakpoint': 6.9003588186585703e-05, u'cleanest': 0.069072591774772277, u'good/bett': 6.9003588186585703e-05, u'unicod': 6.9003588186585703e-05, u'phpeclips': 6.9003588186585703e-05, u'grow': 6.9003588186585703e-05, u'valu': 0.3450869445211151}, 'localization': {u'claim': 5.130309870716191e-05, u'excus': 0.051354401805869067, u'cross-cultur': 0.051354401805869067, u'sourcesaf': 5.130309870716191e-05, u'advantag': 0.051354401805869067, u'imagin': 5.130309870716191e-05, u'internation': 0.10265750051303098, u'famou': 0.051354401805869067, u'joel': 0.051354401805869067, u'breed': 5.130309870716191e-05, u'tip': 0.10265750051303098, u'us': 0.051354401805869067, u'grow': 5.130309870716191e-05, u'minimum': 0.051354401805869067, u'breakpoint': 5.130309870716191e-05, u'ensur': 0.051354401805869067, u'unicod': 0.1539605992201929, u'phpeclips': 5.130309870716191e-05, u'war': 0.051354401805869067, u'speak': 0.051354401805869067}, 'subclipse': {u'imagin': 5.130309870716191e-05, u'set': 5.130309870716191e-05, u'newest': 0.1539605992201929, u'sourcesaf': 5.130309870716191e-05, u'plugin': 0.051354401805869067, u'internation': 5.130309870716191e-05, u'famou': 5.130309870716191e-05, u'breed': 5.130309870716191e-05, u'minimum': 5.130309870716191e-05, u'detail': 0.051354401805869067, u'grow': 5.130309870716191e-05, u'claim': 5.130309870716191e-05, u'aptana': 0.20526369792735483, u'cross-cultur': 5.130309870716191e-05, u'good/bett': 5.130309870716191e-05, u'new': 0.10265750051303098, u'unicod': 5.130309870716191e-05, u'told': 0.10265750051303098, u'manag': 0.1539605992201929, u'highli': 0.051354401805869067}, 'ajax': {u'unwind': 0.057226160530528231, u'deleg': 0.11439515206951748, u'detect': 0.057226160530528231, u'phpeclips': 5.7168991538989253e-05, u'claim': 5.7168991538989253e-05, u'try-catch': 0.057226160530528231, u'breed': 5.7168991538989253e-05, u'happi': 0.057226160530528231, u'snippet': 0.057226160530528231, u'bring': 0.057226160530528231, u'imagin': 5.7168991538989253e-05, u'unc': 0.057226160530528231, u'sleep': 0.057226160530528231, u'grow': 5.7168991538989253e-05, u'breakpoint': 5.7168991538989253e-05, u'alreadi': 0.057226160530528231, u'anonym': 0.057226160530528231, u'throughout': 0.057226160530528231, u'method': 0.11439515206951748, u'good/bett': 5.7168991538989253e-05}, 'orm': {u'claim': 8.0051232788984944e-05, u'tester': 8.0051232788984944e-05, u'phpeclips': 8.0051232788984944e-05, u'cross-cultur': 8.0051232788984944e-05, u'longer': 0.080131284021773927, u'sourcesaf': 8.0051232788984944e-05, u'imagin': 8.0051232788984944e-05, u'internation': 8.0051232788984944e-05, u'breed': 8.0051232788984944e-05, u'nhibern': 0.16018251681075887, u'tend': 0.080131284021773927, u'poco': 0.080131284021773927, u'aw': 8.0051232788984944e-05, u'concensu': 0.080131284021773927, u'lightweight': 0.080131284021773927, u'good/bett': 8.0051232788984944e-05, u'model': 0.24023374959974381, u'breakpoint': 8.0051232788984944e-05, u'baggag': 8.0051232788984944e-05, u'grow': 8.0051232788984944e-05}, 'submit-button': {u'imagin': 0.00018208302986161691, u'set': 0.00018208302986161691, u'cross-cultur': 0.00018208302986161691, u'sourcesaf': 0.00018208302986161691, u'tester': 0.00018208302986161691, u'internation': 0.00018208302986161691, u'famou': 0.00018208302986161691, u'breed': 0.00018208302986161691, u'submit': 0.5464311726147123, u'claim': 0.00018208302986161691, u'minimum': 0.00018208302986161691, u'aw': 0.00018208302986161691, u'breakpoint': 0.00018208302986161691, u'good/bett': 0.00018208302986161691, u'unicod': 0.00018208302986161691, u'phpeclips': 0.00018208302986161691, u'baggag': 0.00018208302986161691, u'grow': 0.00018208302986161691, u'simpler': 0.00018208302986161691, u'excus': 0.00018208302986161691}, 'bash': {u'comment': 0.038860405499883476, u'sort': 0.038860405499883476, u'art': 0.019439912996193583, u'code': 0.077701390507263268, u'toward': 0.019439912996193583, u'secur': 0.077701390507263268, u'critiqu': 0.038860405499883476, u'review': 0.038860405499883476, u'team': 0.038860405499883476, u'flush': 0.019439912996193583, u'obscur': 0.019439912996193583, u'non-fre': 0.019439912996193583, u'resources/method': 0.019439912996193583, u'worst': 0.019439912996193583, u'plan': 0.038860405499883476, u'learn': 0.038860405499883476, u'peer': 0.038860405499883476, u'onlin': 0.058280898003573368, u'post': 0.058280898003573368, u'php': 0.019439912996193583}, 'ie-mobile': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'cruisecontrol.net': {u'claim': 2.2992734295962477e-05, u'use': 0.22995033569392073, u'breed': 2.2992734295962477e-05, u'old': 0.023015727030258438, u'assemblyinfoc': 0.023015727030258438, u'internation': 2.2992734295962477e-05, u'famou': 2.2992734295962477e-05, u'bit': 0.023015727030258438, u'configur': 0.046008461326220912, u'minimum': 2.2992734295962477e-05, u'ccnet': 0.18396486710199578, u'final': 0.023015727030258438, u'cruisecontrol': 0.069001195622183389, u'set': 2.2992734295962477e-05, u'overlap': 0.023015727030258438, u'breakpoint': 2.2992734295962477e-05, u'cross-cultur': 2.2992734295962477e-05, u'revis': 0.069001195622183389, u'label': 0.023015727030258438, u'nant': 0.20695760139795824}, 'membership': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'timer': {u'incorrect': 0.046575469942304104, u'portion': 0.046575469942304104, u'convert': 0.046575469942304104, u'excus': 4.6528941001302811e-05, u'okay': 0.046575469942304104, u'string': 0.13963335194490972, u'internation': 4.6528941001302811e-05, u'famou': 4.6528941001302811e-05, u'reliabl': 0.093104410943606913, u'good/bett': 4.6528941001302811e-05, u'detail': 0.046575469942304104, u'cast': 0.046575469942304104, u'"direct': 0.046575469942304104, u'minimum': 4.6528941001302811e-05, u'translat': 0.093104410943606913, u'cast"': 0.046575469942304104, u'cross-cultur': 4.6528941001302811e-05, u't-sql': 0.093104410943606913, u'unicod': 4.6528941001302811e-05, u'loop': 0.093104410943606913}, 'lambda': {u'prime': 0.12916343919442291, u'claim': 6.4549444874774072e-05, u'code': 0.32281177381874515, u'cross-cultur': 6.4549444874774072e-05, u'minimum': 6.4549444874774072e-05, u'internation': 6.4549444874774072e-05, u'famou': 6.4549444874774072e-05, u'breed': 6.4549444874774072e-05, u'tip': 6.4549444874774072e-05, u'ensur': 6.4549444874774072e-05, u'ineffici': 0.064613994319648843, u'matter': 0.12916343919442291, u'cannot': 0.064613994319648843, u'manner': 0.064613994319648843, u'breakpoint': 6.4549444874774072e-05, u'print': 0.064613994319648843, u'unicod': 6.4549444874774072e-05, u'phpeclips': 6.4549444874774072e-05, u'grow': 6.4549444874774072e-05, u'excus': 6.4549444874774072e-05}, 'windows-mobile': {u'control': 0.18186777661036127, u'load': 0.040430776691182412, u'vb': 0.2020730623131011, u'legaci': 0.020225490988442577, u'enabl': 0.020225490988442577, u'directli': 0.020225490988442577, u'unload': 0.060636062393922253, u'memori': 0.020225490988442577, u'createobject': 0.020225490988442577, u'dll': 0.020225490988442577, u'asynchron': 0.020225490988442577, u'recompil': 0.020225490988442577, u'vc': 0.020225490988442577, u'full': 0.020225490988442577, u'common': 0.020225490988442577, u'c/c++': 0.020225490988442577, u'ide': 0.060636062393922253, u'com': 0.040430776691182412, u'everyday': 0.020225490988442577, u'unabl': 0.020225490988442577}, 'variables': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'hook': {u'imagin': 0.00018208302986161691, u'set': 0.00018208302986161691, u'cross-cultur': 0.00018208302986161691, u'aw': 0.00018208302986161691, u'sourcesaf': 0.00018208302986161691, u'tester': 0.00018208302986161691, u'internation': 0.00018208302986161691, u'breed': 0.00018208302986161691, u'plugin': 0.5464311726147123, u'famou': 0.00018208302986161691, u'claim': 0.00018208302986161691, u'minimum': 0.00018208302986161691, u'breakpoint': 0.00018208302986161691, u'baggag': 0.00018208302986161691, u'good/bett': 0.00018208302986161691, u'unicod': 0.00018208302986161691, u'phpeclips': 0.00018208302986161691, u'grow': 0.00018208302986161691, u'simpler': 0.00018208302986161691, u'excus': 0.00018208302986161691}, 'sockets': {u'rundown': 0.033941407839414073, u'claim': 3.3907500339075e-05, u'c': 0.10175640851756408, u'properli': 0.033941407839414073, u'cross-cultur': 3.3907500339075e-05, u'socket': 0.16957140919571409, u'spit': 0.033941407839414073, u'internation': 3.3907500339075e-05, u'replac': 0.16957140919571409, u'phpeclips': 3.3907500339075e-05, u'famou': 3.3907500339075e-05, u'kind': 0.033941407839414073, u'malloc': 0.033941407839414073, u'z/o': 0.13566390885663909, u'perhap': 0.06784890817848907, u'breakpoint': 3.3907500339075e-05, u'good/bett': 3.3907500339075e-05, u'happi': 0.033941407839414073, u'complic': 0.033941407839414073, u'longer': 0.033941407839414073}, 'size': {u'box': 0.040430776691182412, u'exhibit': 0.020225490988442577, u'fast': 0.020225490988442577, u'insan': 0.020225490988442577, u'socket': 0.020225490988442577, u'environment;': 0.020225490988442577, u'goe': 0.040430776691182412, u'replac': 0.020225490988442577, u'toward': 0.020225490988442577, u'sign': 0.020225490988442577, u'long': 0.040430776691182412, u'mark': 0.020225490988442577, u'environ': 0.040430776691182412, u'grunti': 0.020225490988442577, u'heap': 0.16166249090762144, u'hell': 0.020225490988442577, u'seem': 0.040430776691182412, u'purifi': 0.060636062393922253, u'reproduc': 0.080841348096662102, u'inapplic': 0.020225490988442577}, 'system': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'sandcastle': {u'msdn-style': 0.074192113845241617, u'imagin': 7.4117995849392229e-05, u'set': 7.4117995849392229e-05, u'unless': 0.14831010969463385, u'unicod': 7.4117995849392229e-05, u'sourcesaf': 7.4117995849392229e-05, u'tester': 0.074192113845241617, u'internation': 7.4117995849392229e-05, u'famou': 7.4117995849392229e-05, u'paragraph': 0.074192113845241617, u'minimum': 7.4117995849392229e-05, u'grow': 7.4117995849392229e-05, u'claim': 7.4117995849392229e-05, u'api': 0.074192113845241617, u'good/bett': 7.4117995849392229e-05, u'cross-cultur': 7.4117995849392229e-05, u'along': 0.074192113845241617, u'phpeclips': 7.4117995849392229e-05, u'produc': 0.22242810554402606, u'awesom': 0.074192113845241617}, 'monkeypatching': {u'imagin': 4.879953152449736e-05, u'unfriendli': 0.048848331056021856, u'task': 0.048848331056021856, u'set': 4.879953152449736e-05, u'sourcesaf': 4.879953152449736e-05, u'jeff': 0.097647862580519218, u'breed': 4.879953152449736e-05, u'notif': 0.048848331056021856, u'friendli': 0.048848331056021856, u'mark': 0.048848331056021856, u'blog': 0.048848331056021856, u'unhandl': 0.048848331056021856, u'handler': 0.14644739410501659, u'dialog': 0.097647862580519218, u'claim': 4.879953152449736e-05, u'good/bett': 4.879953152449736e-05, u'uncaught': 0.048848331056021856, u'grow': 4.879953152449736e-05, u'phpeclips': 4.879953152449736e-05, u'except': 0.14644739410501659}, 'release-management': {u'given': 0.12252980565082475, u'lack': 0.040870488322717613, u'iceberg': 4.0829658664053568e-05, u'manager/manag': 0.08170014698677118, u'attach': 0.040870488322717613, u'internation': 4.0829658664053568e-05, u'hobbi': 0.040870488322717613, u'price': 0.040870488322717613, u'express': 0.08170014698677118, u'mdf': 0.040870488322717613, u'complex': 0.040870488322717613, u'"explain': 0.040870488322717613, u'binari': 0.040870488322717613, u'alreadi': 0.040870488322717613, u'steep': 0.040870488322717613, u'batch': 0.040870488322717613, u'buildscript': 0.040870488322717613, u'compatibl': 0.040870488322717613, u'alter': 0.040870488322717613, u'target': 0.040870488322717613}, 'rdoc': {u'comment': 0.077964252312168761, u'unzip': 0.026005403720253558, u'unprocess': 0.026005403720253558, u'belief': 0.051984828016211161, u'incred': 0.026005403720253558, u'rubi': 0.23384079808791439, u'offici': 0.051984828016211161, u'work': 0.051984828016211161, u'rdoc': 0.077964252312168761, u'batch': 0.026005403720253558, u'`request_get': 0.026005403720253558, u'extens': 0.026005403720253558, u'ruby-lik': 0.026005403720253558, u'breakpoint': 2.5979424295957604e-05, u'exceptionnotifi': 0.051984828016211161, u'path': 0.026005403720253558, u'havent': 0.026005403720253558, u'choos': 0.026005403720253558, u'grow': 2.5979424295957604e-05, u'view': 0.077964252312168761}, 'filesystems': {u'gig': 0.051354401805869067, u'copi': 0.2565667966345167, u'cross-cultur': 5.130309870716191e-05, u'zip': 0.051354401805869067, u'claim': 5.130309870716191e-05, u'internation': 5.130309870716191e-05, u'breed': 5.130309870716191e-05, u'away': 0.051354401805869067, u'random': 0.051354401805869067, u'manag': 0.051354401805869067, u'pata': 0.051354401805869067, u'vanilla': 0.051354401805869067, u'"even': 0.051354401805869067, u'"are': 0.10265750051303098, u'breakpoint': 5.130309870716191e-05, u'cancel': 0.051354401805869067, u'grow': 5.130309870716191e-05, u'phpeclips': 5.130309870716191e-05, u'hundr': 0.051354401805869067, u'good/bett': 5.130309870716191e-05}, 'github': {u'imagin': 0.080131284021773927, u'tester': 8.0051232788984944e-05, u'git': 0.32028498238872877, u'cross-cultur': 8.0051232788984944e-05, u'distribut': 0.080131284021773927, u'sourcesaf': 0.080131284021773927, u'claim': 8.0051232788984944e-05, u'internation': 8.0051232788984944e-05, u'famou': 8.0051232788984944e-05, u'breed': 0.080131284021773927, u'set': 8.0051232788984944e-05, u'minimum': 8.0051232788984944e-05, u'combin': 0.080131284021773927, u'aw': 8.0051232788984944e-05, u'breakpoint': 8.0051232788984944e-05, u'good/bett': 8.0051232788984944e-05, u'unicod': 8.0051232788984944e-05, u'phpeclips': 8.0051232788984944e-05, u'baggag': 8.0051232788984944e-05, u'grow': 0.080131284021773927}, 'throttle': {u'imagin': 6.4549444874774072e-05, u'tester': 6.4549444874774072e-05, u'smaller': 0.064613994319648843, u'%': 0.064613994319648843, u'sourcesaf': 6.4549444874774072e-05, u'simpli': 0.064613994319648843, u'less': 0.064613994319648843, u'certain': 0.12916343919442291, u'phpeclips': 6.4549444874774072e-05, u'minim': 0.064613994319648843, u'claim': 6.4549444874774072e-05, u'good/bett': 6.4549444874774072e-05, u'intens': 0.064613994319648843, u'breakpoint': 6.4549444874774072e-05, u'baggag': 6.4549444874774072e-05, u'breed': 6.4549444874774072e-05, u'throttl': 0.064613994319648843, u'locat': 0.064613994319648843, u'loop': 0.12916343919442291, u'depend': 0.064613994319648843}, 'visualj#': {u'imagin': 0.00018208302986161691, u'convert': 0.18226511289147848, u'code': 0.36434814275309541, u'cross-cultur': 0.00018208302986161691, u'sourcesaf': 0.00018208302986161691, u'tester': 0.00018208302986161691, u'internation': 0.00018208302986161691, u'famou': 0.00018208302986161691, u'breed': 0.00018208302986161691, u'minimum': 0.00018208302986161691, u'claim': 0.00018208302986161691, u'set': 0.00018208302986161691, u'aw': 0.00018208302986161691, u'breakpoint': 0.00018208302986161691, u'good/bett': 0.00018208302986161691, u'unicod': 0.00018208302986161691, u'phpeclips': 0.00018208302986161691, u'baggag': 0.00018208302986161691, u'grow': 0.00018208302986161691, u'excus': 0.00018208302986161691}, 'iteration': {u'imagin': 9.5310712924132673e-05, u'set': 9.5310712924132673e-05, u'depend': 0.095406023637056789, u'workth': 0.095406023637056789, u'sourcesaf': 9.5310712924132673e-05, u'secur': 0.095406023637056789, u'internation': 9.5310712924132673e-05, u'famou': 9.5310712924132673e-05, u'breed': 9.5310712924132673e-05, u'portablein': 0.095406023637056789, u'grow': 9.5310712924132673e-05, u'claim': 9.5310712924132673e-05, u'minimum': 9.5310712924132673e-05, u'cross-cultur': 9.5310712924132673e-05, u'breakpoint': 9.5310712924132673e-05, u'good/bett': 9.5310712924132673e-05, u'encrypt': 0.19071673656118945, u'unicod': 9.5310712924132673e-05, u'phpeclips': 9.5310712924132673e-05, u'method': 0.19071673656118945}, 'lucene': {u'//': 0.036410592172268288, u'index': 0.072784810126582278, u'differ': 0.10915902808089625, u'phpeclips': 3.6374217954313981e-05, u'return': 0.10915902808089625, u'name': 0.072784810126582278, u'cross-cultur': 3.6374217954313981e-05, u'dob': 0.072784810126582278, u'famou': 3.6374217954313981e-05, u'breed': 3.6374217954313981e-05, u'smith': 0.036410592172268288, u'search': 0.072784810126582278, u'internation': 3.6374217954313981e-05, u'minimum': 3.6374217954313981e-05, u'result': 0.10915902808089625, u'partit': 0.10915902808089625, u'breakpoint': 3.6374217954313981e-05, u'good/bett': 3.6374217954313981e-05, u'multipl': 0.036410592172268288, u'exactli': 0.072784810126582278}, 'linear-equation': {u'c': 0.10820895522388059, u'b': 0.054131516331386534, u'cross-cultur': 5.4077438892494048e-05, u'linear': 0.10820895522388059, u'tx': 0.054131516331386534, u'internation': 5.4077438892494048e-05, u'approxim': 0.054131516331386534, u'breed': 5.4077438892494048e-05, u'c++': 0.054131516331386534, u'here': 0.054131516331386534, u'need': 0.10820895522388059, u'minimum': 5.4077438892494048e-05, u'equat': 0.16228639411637463, u'breakpoint': 5.4077438892494048e-05, u'solv': 0.10820895522388059, u'unicod': 5.4077438892494048e-05, u'phpeclips': 5.4077438892494048e-05, u'grow': 5.4077438892494048e-05, u'good/bett': 5.4077438892494048e-05, u'excus': 5.4077438892494048e-05}, 'binary-data': {u'prevent': 0.018968262996246158, u'sqlite': 0.028447654798468128, u'situat': 0.0094888711940241902, u'practic': 0.018968262996246158, u'creat': 0.047406438402912067, u'edit': 0.094803397414021912, u'databas': 0.22751488264512948, u'technolog': 0.0094888711940241902, u'two': 0.047406438402912067, u'uniqu': 0.018968262996246158, u'flow': 0.0094888711940241902, u'column': 0.047406438402912067, u'drag': 0.0094888711940241902, u'user': 0.05688583020513404, u'ms': 0.018968262996246158, u'big': 0.018968262996246158, u'car': 0.0094888711940241902, u'xsd': 0.018968262996246158, u'store': 0.037927046600690101, u'updat': 0.066365222007356006}, 'browser': {u'ago': 0.20707976814794368, u'cetera': 0.069072591774772277, u'set': 6.9003588186585703e-05, u'cross-cultur': 6.9003588186585703e-05, u'breakpoint': 6.9003588186585703e-05, u'hour': 0.069072591774772277, u'imagin': 6.9003588186585703e-05, u'internation': 6.9003588186585703e-05, u'famou': 6.9003588186585703e-05, u'breed': 6.9003588186585703e-05, u'month': 0.069072591774772277, u'claim': 6.9003588186585703e-05, u'calcul': 0.069072591774772277, u'minimum': 6.9003588186585703e-05, u'rel': 0.13807617996135799, u'time': 0.13807617996135799, u'et': 0.069072591774772277, u'unicod': 6.9003588186585703e-05, u'phpeclips': 6.9003588186585703e-05, u'grow': 6.9003588186585703e-05}, 'debugging': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'protocol-buffers': {u'wide': 0.057226160530528231, u'search': 0.17156414360850672, u'set': 5.7168991538989253e-05, u'influenti': 0.057226160530528231, u'upper-right': 0.057226160530528231, u'tester': 5.7168991538989253e-05, u'career': 0.057226160530528231, u'could': 0.057226160530528231, u'breed': 5.7168991538989253e-05, u'rang': 0.057226160530528231, u'read': 0.057226160530528231, u'imagin': 5.7168991538989253e-05, u'book': 0.17156414360850672, u'sourcesaf': 5.7168991538989253e-05, u'breakpoint': 5.7168991538989253e-05, u'good/bett': 5.7168991538989253e-05, u'corner': 0.057226160530528231, u'phpeclips': 5.7168991538989253e-05, u'vari': 0.057226160530528231, u'grow': 5.7168991538989253e-05}, 'type-safety': {u'imagin': 0.00015403573629081948, u'claim': 0.00015403573629081948, u'actual': 0.15418977202711026, u'cross-cultur': 0.00015403573629081948, u'sourcesaf': 0.00015403573629081948, u'tester': 0.00015403573629081948, u'internation': 0.00015403573629081948, u'famou': 0.00015403573629081948, u'breed': 0.00015403573629081948, u'hobbyist': 0.00015403573629081948, u'aw': 0.00015403573629081948, u'minimum': 0.00015403573629081948, u'eras': 0.15418977202711026, u'breakpoint': 0.00015403573629081948, u'baggag': 0.00015403573629081948, u'good/bett': 0.00015403573629081948, u'safeti': 0.30822550831792972, u'unicod': 0.00015403573629081948, u'phpeclips': 0.00015403573629081948, u'grow': 0.00015403573629081948}, 'favicon': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'ant': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'shell': {u'claim': 4.2567682615358418e-05, u'eventu': 0.042610250297973773, u'set': 4.2567682615358418e-05, u'hold': 0.042610250297973773, u'navig': 0.042610250297973773, u'decent': 0.042610250297973773, u'phpeclips': 4.2567682615358418e-05, u'breed': 4.2567682615358418e-05, u'unawar': 0.042610250297973773, u'vim': 0.042610250297973773, u'access': 0.042610250297973773, u'host': 0.085177932913332186, u'shell': 0.042610250297973773, u'command': 0.1277456155286906, u'know': 0.17031329814404905, u'good/bett': 4.2567682615358418e-05, u'effect': 0.042610250297973773, u'longtim': 0.042610250297973773, u'final': 0.042610250297973773, u'man': 0.042610250297973773}, 'form-submit': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'asp.net': {u'attempt': 0.0984192575101666, u'execut': 0.032828282828282825, u'bot': 0.032828282828282825, u'proper': 0.032828282828282825, u'english': 3.2795487340941887e-05, u'encod': 0.13121474485110848, u'"c"': 0.032828282828282825, u'getstr': 0.032828282828282825, u'snippet': 0.032828282828282825, u'thing': 0.065623770169224716, u'attack': 0.065623770169224716, u'paragraph': 3.2795487340941887e-05, u'decod': 0.19680571953299228, u'inject': 0.032828282828282825, u'wari': 0.032828282828282825, u'resurg': 0.032828282828282825, u'iceberg': 3.2795487340941887e-05, u'interestingli': 0.032828282828282825, u'exact': 0.032828282828282825, u'speak': 3.2795487340941887e-05}, 'zos': {u'msdn-style': 0.00011775788977861516, u'excus': 0.00011775788977861516, u'cross-cultur': 0.00011775788977861516, u'minimum': 0.00011775788977861516, u'internation': 0.00011775788977861516, u'tip': 0.00011775788977861516, u'oe': 0.11787564766839376, u'phpeclips': 0.00011775788977861516, u'upon': 0.11787564766839376, u'af_inet': 0.11787564766839376, u'ensur': 0.00011775788977861516, u'much': 0.11787564766839376, u'breakpoint': 0.00011775788977861516, u'famou': 0.00011775788977861516, u'dear': 0.11787564766839376, u'output': 0.00011775788977861516, u';': 0.11787564766839376, u'bridg': 0.00011775788977861516, u'unicod': 0.00011775788977861516, u'good/bett': 0.00011775788977861516}, 'performance': {u'claim': 0.00010535187526337968, u'tester': 0.00010535187526337968, u'set': 0.00010535187526337968, u'cross-cultur': 0.00010535187526337968, u'sourcesaf': 0.00010535187526337968, u'imagin': 0.00010535187526337968, u'internation': 0.00010535187526337968, u'famou': 0.00010535187526337968, u'breed': 0.00010535187526337968, u'binari': 0.21080910240202272, u'minimum': 0.00010535187526337968, u'breakpoint': 0.00010535187526337968, u'mysql': 0.21080910240202272, u'good/bett': 0.00010535187526337968, u'unicod': 0.00010535187526337968, u'phpeclips': 0.00010535187526337968, u'data': 0.21080910240202272, u'grow': 0.00010535187526337968, u'store': 0.10545722713864304, u'excus': 0.00010535187526337968}, 'j#': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'visual-studio': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'artificial-intelligence': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'collaboration': {u'kb': 0.051354401805869067, u'berkeleydb': 0.10265750051303098, u'resourc': 0.051354401805869067, u'record': 0.051354401805869067, u'optim': 0.051354401805869067, u'real-world': 0.051354401805869067, u'away': 0.051354401805869067, u'bdb': 0.051354401805869067, u'cursor': 0.051354401805869067, u'delet': 0.051354401805869067, u'locker': 0.051354401805869067, u'concurr': 0.1539605992201929, u'internation': 5.130309870716191e-05, u'aw': 5.130309870716191e-05, u'baggag': 5.130309870716191e-05, u'breed': 5.130309870716191e-05, u'cross-cultur': 5.130309870716191e-05, u'suffer': 0.051354401805869067, u'hammer': 0.051354401805869067, u'simpler': 5.130309870716191e-05}, 'software-engineering': {u'imagin': 5.130309870716191e-05, u'claim': 5.130309870716191e-05, u'hide': 0.20526369792735483, u'ridicul': 0.051354401805869067, u'sourcesaf': 5.130309870716191e-05, u'tester': 5.130309870716191e-05, u'baggag': 5.130309870716191e-05, u'vestigi': 0.051354401805869067, u'inherit': 0.20526369792735483, u'reiter': 5.130309870716191e-05, u'member': 0.1539605992201929, u'paragraph': 5.130309870716191e-05, u'design': 0.10265750051303098, u'descend': 0.051354401805869067, u'aw': 5.130309870716191e-05, u'intellisens': 0.051354401805869067, u'ship': 5.130309870716191e-05, u'grow': 5.130309870716191e-05, u'non-invit': 5.130309870716191e-05, u'simpler': 5.130309870716191e-05}, 'statistics': {u'hacker': 0.0083204430916620931, u'video': 0.0083204430916620931, u'develop': 0.024953012990037094, u'secur': 0.016636728040849594, u'rubi': 0.012478585566255843, u'iphon': 0.012478585566255843, u'rail': 0.012478585566255843, u'coder': 0.0083204430916620931, u'/': 0.054060010312193339, u'weekli': 0.012478585566255843, u'-': 0.012478585566255843, u'area': 0.012478585566255843, u'mac': 0.012478585566255843, u'radio': 0.029111155464630844, u'linux': 0.020794870515443344, u'tech': 0.012478585566255843, u'inact': 0.029111155464630844, u'podcast': 0.091483292583537087, u'haystack': 0.0083204430916620931, u'listen': 0.012478585566255843}, 'asp.net-mvc': {u'nearest': 0.12133155469318456, u'north': 0.060696095076400669, u'mercat': 0.060696095076400669, u'claim': 6.0635459616783893e-05, u'internation': 6.0635459616783893e-05, u'lat/long->x/i': 0.060696095076400669, u'breed': 6.0635459616783893e-05, u'readabl': 0.060696095076400669, u'geograph': 0.12133155469318456, u'worri': 0.060696095076400669, u'three': 0.060696095076400669, u'coordin': 0.060696095076400669, u'imagin': 6.0635459616783893e-05, u'subway': 0.060696095076400669, u'breakpoint': 6.0635459616783893e-05, u'known': 0.060696095076400669, u'phpeclips': 6.0635459616783893e-05, u'grow': 6.0635459616783893e-05, u'good/bett': 6.0635459616783893e-05, u'x/i': 0.060696095076400669}, 'class': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'url': {u'claim': 7.4117995849392229e-05, u'breed': 7.4117995849392229e-05, u'bar': 0.14831010969463385, u'imagin': 7.4117995849392229e-05, u'bookmark': 0.074192113845241617, u'famou': 7.4117995849392229e-05, u'app': 0.2965461013934183, u'move': 0.074192113845241617, u'minimum': 7.4117995849392229e-05, u'current': 0.14831010969463385, u'internation': 7.4117995849392229e-05, u'set': 7.4117995849392229e-05, u'therebi': 0.074192113845241617, u'breakpoint': 7.4117995849392229e-05, u'good/bett': 7.4117995849392229e-05, u'cross-cultur': 7.4117995849392229e-05, u'unicod': 7.4117995849392229e-05, u'phpeclips': 7.4117995849392229e-05, u'grow': 7.4117995849392229e-05, u'excus': 7.4117995849392229e-05}, 'flat-file': {u'key': 0.020642580219417634, u'index': 0.12375237152520005, u'full': 0.041264538480574119, u'regardless': 0.020642580219417634, u'outlin': 0.020642580219417634, u'scenario': 0.041264538480574119, u'replic': 0.020642580219417634, u'column': 0.14437432978635653, u're-insert': 0.020642580219417634, u'work': 0.041264538480574119, u'ugh': 0.020642580219417634, u'swap': 0.061886496741730593, u'rememb': 0.061886496741730593, u'sql': 0.082508455002887088, u'hack': 0.020642580219417634, u'multipl': 0.020642580219417634, u'http//stackoverflowcom/questions//how-does-database-indexing-work': 0.020642580219417634, u'primari': 0.020642580219417634, u'special': 0.061886496741730593, u'row': 0.061886496741730593}, 'language-agnostic': {u'sort': 0.14499014264177201, u'subset': 0.02902122231242027, u'clarif': 0.058013452394758205, u'effici': 0.11599791255943409, u'algorithm': 0.058013452394758205, u'color': 0.087005682477096141, u'famou': 2.8992230082337937e-05, u'random': 0.02902122231242027, u'tip': 2.8992230082337937e-05, u'number': 0.087005682477096141, u'n': 0.11599791255943409, u'stuck': 0.02902122231242027, u'assur': 0.02902122231242027, u'wheel': 0.02902122231242027, u'pseudo-solv': 0.02902122231242027, u'english': 2.8992230082337937e-05, u'distinguish': 0.02902122231242027, u'excus': 2.8992230082337937e-05, u'>': 0.02902122231242027, u'duplic': 0.02902122231242027}, 'file-locking': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'xampp': {u'firefox->view': 0.080131284021773927, u'imagin': 8.0051232788984944e-05, u'fee': 0.16018251681075887, u'breakpoint': 8.0051232788984944e-05, u'directli': 0.080131284021773927, u'tester': 8.0051232788984944e-05, u'breed': 8.0051232788984944e-05, u'baggag': 8.0051232788984944e-05, u'paid': 0.080131284021773927, u'grow': 8.0051232788984944e-05, u'claim': 8.0051232788984944e-05, u'pageclientscriptregisterclientscriptblock': 0.080131284021773927, u'paragraph': 8.0051232788984944e-05, u'master': 0.080131284021773927, u'aw': 8.0051232788984944e-05, u'excess': 0.080131284021773927, u'taken': 0.080131284021773927, u'client-sid': 0.080131284021773927, u'non-invit': 8.0051232788984944e-05, u'simpler': 8.0051232788984944e-05}, 'text': {u'real': 0.093104410943606913, u'absolut': 0.093104410943606913, u'excus': 4.6528941001302811e-05, u'iceberg': 0.046575469942304104, u'string': 0.093104410943606913, u'output': 4.6528941001302811e-05, u'famou': 4.6528941001302811e-05, u'work': 0.13963335194490972, u'project': 0.18616229294621256, u'ensur': 4.6528941001302811e-05, u'stori': 0.046575469942304104, u'minimum': 4.6528941001302811e-05, u'besid': 0.046575469942304104, u'msdn-style': 4.6528941001302811e-05, u'cross-cultur': 4.6528941001302811e-05, u'english': 0.046575469942304104, u'posit': 0.046575469942304104, u'youv': 0.046575469942304104, u'unicod': 4.6528941001302811e-05, u'glyph': 4.6528941001302811e-05}, 'random': {u'imagin': 0.00011775788977861516, u'tester': 0.00011775788977861516, u'set': 0.00011775788977861516, u'cross-cultur': 0.00011775788977861516, u'cx_oracl': 0.11787564766839376, u'sourcesaf': 0.00011775788977861516, u'claim': 0.00011775788977861516, u'internation': 0.00011775788977861516, u'famou': 0.00011775788977861516, u'breed': 0.00011775788977861516, u'-': 0.11787564766839376, u'minimum': 0.00011775788977861516, u'result': 0.23563353744700891, u'breakpoint': 0.00011775788977861516, u'good/bett': 0.00011775788977861516, u'unicod': 0.00011775788977861516, u'phpeclips': 0.00011775788977861516, u'grow': 0.00011775788977861516, u'best': 0.23563353744700891, u'excus': 0.00011775788977861516}, 'html-parsing': {u'cross-cultur': 6.0635459616783893e-05, u'phpeclips': 6.0635459616783893e-05, u'breviti': 0.060696095076400669, u'buffer': 0.18196701430996845, u'text': 0.060696095076400669, u'breed': 6.0635459616783893e-05, u'famou': 6.0635459616783893e-05, u'internation': 6.0635459616783893e-05, u'benefit': 0.060696095076400669, u'minimum': 6.0635459616783893e-05, u'user-fac': 0.060696095076400669, u'editor': 0.060696095076400669, u'breakpoint': 6.0635459616783893e-05, u'factor': 0.060696095076400669, u'good/bett': 6.0635459616783893e-05, u'protocol': 0.18196701430996845, u'unicod': 6.0635459616783893e-05, u'opinion': 0.060696095076400669, u'speed': 0.060696095076400669, u'excus': 6.0635459616783893e-05}, 'datetime': {u'oper': 0.017113451412158927, u'insert': 0.017113451412158927, u'featur': 0.034209806469260751, u'interconnect': 0.017113451412158927, u'depart': 0.017113451412158927, u'though': 0.034209806469260751, u'object': 0.034209806469260751, u'drive': 0.017113451412158927, u'one': 0.11969158175476989, u'framework': 0.11969158175476989, u'theori': 0.017113451412158927, u'integr': 0.034209806469260751, u'connect': 0.051306161526362581, u'key': 0.11969158175476989, u'fulli': 0.017113451412158927, u'winform': 0.017113451412158927, u'foreign': 0.11969158175476989, u'except': 0.017113451412158927, u'piec': 0.017113451412158927, u'addit': 0.085498871640566243}, 'colors': {u'error_log': 0.13360918312867057, u'tester': 0.00013347570742124932, u'claim': 0.00013347570742124932, u'cross-cultur': 0.00013347570742124932, u'sourcesaf': 0.00013347570742124932, u'imagin': 0.00013347570742124932, u'internation': 0.00013347570742124932, u'famou': 0.00013347570742124932, u'breed': 0.00013347570742124932, u'minimum': 0.00013347570742124932, u'separ': 0.40056059797116922, u'good/bett': 0.00013347570742124932, u'set': 0.00013347570742124932, u'breakpoint': 0.00013347570742124932, u'linux': 0.13360918312867057, u'ensur': 0.00013347570742124932, u'unicod': 0.00013347570742124932, u'phpeclips': 0.00013347570742124932, u'grow': 0.00013347570742124932, u'excus': 0.00013347570742124932}, 'xml': {u'assumpt': 0.037784991695606215, u'breed': 3.7747244451155068e-05, u'java': 0.22652121395138156, u'minimum': 3.7747244451155068e-05, u'fals': 0.037784991695606215, u'famou': 3.7747244451155068e-05, u'object': 0.30201570285369167, u'list': 0.075532236146761286, u'dramat': 3.7747244451155068e-05, u'getter/sett': 0.075532236146761286, u'internation': 3.7747244451155068e-05, u'getter': 0.037784991695606215, u'cross-cultur': 3.7747244451155068e-05, u'breakpoint': 3.7747244451155068e-05, u'good/bett': 3.7747244451155068e-05, u'"purist"': 0.037784991695606215, u'unicod': 3.7747244451155068e-05, u'phpeclips': 3.7747244451155068e-05, u'assum': 0.037784991695606215, u'discuss': 0.037784991695606215}, 'parameters': {u'encount': 0.040870488322717613, u'typeid': 0.040870488322717613, u'depart': 0.040870488322717613, u'cross-cultur': 4.0829658664053568e-05, u'primarili': 0.040870488322717613, u'offer': 0.08170014698677118, u'chosen': 0.040870488322717613, u'join': 0.040870488322717613, u'make': 0.040870488322717613, u'pretend': 0.040870488322717613, u'famou': 4.0829658664053568e-05, u'typeauxid': 0.040870488322717613, u'internation': 4.0829658664053568e-05, u'permiss': 0.20418912297893191, u'pay': 4.0829658664053568e-05, u'less': 0.040870488322717613, u'sens': 0.040870488322717613, u'neither': 0.040870488322717613, u'grow': 0.040870488322717613, u'paramet': 0.08170014698677118}, 'cross-platform': {u'imagin': 5.7168991538989253e-05, u'minimum': 5.7168991538989253e-05, u'cross-cultur': 5.7168991538989253e-05, u'string': 0.22873313514749602, u'sourcesaf': 5.7168991538989253e-05, u'claim': 5.7168991538989253e-05, u'famou': 5.7168991538989253e-05, u'breed': 5.7168991538989253e-05, u'list': 0.17156414360850672, u'charact': 0.11439515206951748, u'good/bett': 5.7168991538989253e-05, u'length': 0.057226160530528231, u'permut': 0.11439515206951748, u'breakpoint': 5.7168991538989253e-05, u'y': 0.057226160530528231, u'x': 0.057226160530528231, u'phpeclips': 5.7168991538989253e-05, u'associ': 5.7168991538989253e-05, u'grow': 5.7168991538989253e-05, u'portabl': 0.057226160530528231}, 'terminal': {u'loss': 0.11787564766839376, u'breed': 0.00011775788977861516, u'set': 0.00011775788977861516, u'cross-cultur': 0.00011775788977861516, u'sourcesaf': 0.00011775788977861516, u'tester': 0.00011775788977861516, u'imagin': 0.00011775788977861516, u'famou': 0.00011775788977861516, u'linux': 0.23563353744700891, u'due': 0.11787564766839376, u'claim': 0.00011775788977861516, u'internation': 0.00011775788977861516, u'lamp': 0.11787564766839376, u'minimum': 0.00011775788977861516, u'breakpoint': 0.00011775788977861516, u'baggag': 0.00011775788977861516, u'good/bett': 0.00011775788977861516, u'els': 0.11787564766839376, u'phpeclips': 0.00011775788977861516, u'grow': 0.00011775788977861516}, 'title-case': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'ip': {u'imagin': 0.00015403573629081948, u'tester': 0.00015403573629081948, u'set': 0.00015403573629081948, u'cross-cultur': 0.00015403573629081948, u'sourcesaf': 0.00015403573629081948, u'claim': 0.00015403573629081948, u'internation': 0.00015403573629081948, u'root': 0.15418977202711026, u'sudo': 0.15418977202711026, u'famou': 0.00015403573629081948, u'baggag': 0.00015403573629081948, u'good/bett': 0.00015403573629081948, u'minimum': 0.00015403573629081948, u'privileg': 0.15418977202711026, u'breakpoint': 0.00015403573629081948, u'breed': 0.00015403573629081948, u'unicod': 0.00015403573629081948, u'phpeclips': 0.00015403573629081948, u'config': 0.15418977202711026, u'grow': 0.00015403573629081948}, 'pass-by-reference': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'db2': {u'imagin': 6.4549444874774072e-05, u'iseri': 0.064613994319648843, u'cross-cultur': 6.4549444874774072e-05, u'sourcesaf': 6.4549444874774072e-05, u'upcom': 0.064613994319648843, u'internation': 6.4549444874774072e-05, u'driver': 0.064613994319648843, u'breed': 6.4549444874774072e-05, u'db': 0.25826232894397111, u'claim': 6.4549444874774072e-05, u'emul': 0.064613994319648843, u'minimum': 6.4549444874774072e-05, u'cento': 0.064613994319648843, u'breakpoint': 6.4549444874774072e-05, u'linux': 0.19371288406919698, u'good/bett': 6.4549444874774072e-05, u'famou': 6.4549444874774072e-05, u'unicod': 6.4549444874774072e-05, u'turn': 0.064613994319648843, u'grow': 6.4549444874774072e-05}, 'compiler': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'continuous-integration': {u'/usr/local/lib/ruby/gems//gems/rails-/lib/fcgi_handlerrbin': 0.046008461326220912, u'`request': 0.023015727030258438, u'/usr/local/lib/ruby//net/protocolrbin': 0.13797939851007082, u'version': 0.023015727030258438, u'call': 0.069001195622183389, u'servic': 0.046008461326220912, u'`sysread': 0.023015727030258438, u'`to_proc': 0.023015727030258438, u'later': 0.023015727030258438, u'moment': 0.023015727030258438, u'`start': 0.023015727030258438, u'stacktrac': 0.046008461326220912, u'/usr/local/lib/ruby//net/httprbin': 0.16097213280603331, u'`exit': 0.023015727030258438, u'`timeout': 0.046008461326220912, u'went': 0.023015727030258438, u'`rbuf_fil': 0.046008461326220912, u'`read_status_lin': 0.023015727030258438, u'frequent': 0.023015727030258438, u'luck': 0.023015727030258438}, 'c++': {u'prevent': 0.069072591774772277, u'excus': 6.9003588186585703e-05, u'set': 6.9003588186585703e-05, u'cross-cultur': 6.9003588186585703e-05, u'claim': 6.9003588186585703e-05, u'internation': 6.9003588186585703e-05, u'breed': 6.9003588186585703e-05, u'error': 0.13807617996135799, u'famou': 6.9003588186585703e-05, u'grow': 6.9003588186585703e-05, u'minimum': 6.9003588186585703e-05, u'trigger': 0.13807617996135799, u'tabl': 0.13807617996135799, u'breakpoint': 6.9003588186585703e-05, u'mysql': 0.069072591774772277, u'good/bett': 6.9003588186585703e-05, u'unicod': 6.9003588186585703e-05, u'phpeclips': 6.9003588186585703e-05, u'throw': 0.13807617996135799, u'updat': 0.13807617996135799}, 'architecture': {u'imagin': 0.00011775788977861516, u'breed': 0.00011775788977861516, u'set': 0.00011775788977861516, u'cross-cultur': 0.00011775788977861516, u'longer': 0.11787564766839376, u'sourcesaf': 0.00011775788977861516, u'plugin': 0.23563353744700891, u'internation': 0.00011775788977861516, u'famou': 0.00011775788977861516, u'tester': 0.00011775788977861516, u'upgrad': 0.11787564766839376, u'grow': 0.00011775788977861516, u'claim': 0.00011775788977861516, u'aw': 0.00011775788977861516, u'breakpoint': 0.00011775788977861516, u'good/bett': 0.00011775788977861516, u'found': 0.11787564766839376, u'phpeclips': 0.00011775788977861516, u'baggag': 0.00011775788977861516, u'complain': 0.11787564766839376}, 'server-configuration': {u'serv': 0.52686472819216179, u'imagin': 0.00010535187526337968, u'set': 0.00010535187526337968, u'cross-cultur': 0.00010535187526337968, u'sourcesaf': 0.00010535187526337968, u'tester': 0.00010535187526337968, u'internation': 0.00010535187526337968, u'breed': 0.00010535187526337968, u'htdoc': 0.10545722713864304, u'famou': 0.00010535187526337968, u'claim': 0.00010535187526337968, u'minimum': 0.00010535187526337968, u'instanc': 0.10545722713864304, u'aw': 0.00010535187526337968, u'breakpoint': 0.00010535187526337968, u'good/bett': 0.00010535187526337968, u'unicod': 0.00010535187526337968, u'phpeclips': 0.00010535187526337968, u'baggag': 0.00010535187526337968, u'grow': 0.00010535187526337968}, 'testing': {u'ago': 0.018713078591191205, u'nunit': 0.093490615419128106, u'neck': 0.018713078591191205, u'packag': 0.037407462798175431, u'weak': 0.018713078591191205, u'ironrubi': 0.018713078591191205, u'question': 0.056101847005159654, u'rspec': 0.018713078591191205, u'ask': 0.056101847005159654, u'briefli': 0.018713078591191205, u'greatli': 0.018713078591191205, u'introduc': 0.018713078591191205, u'misconcept': 0.018713078591191205, u'test': 0.14957376804008077, u'youv': 0.018713078591191205, u'pitfal': 0.037407462798175431, u'saw': 0.018713078591191205, u'day': 0.018713078591191205, u'unit': 0.13087938383309655, u'goal': 0.018713078591191205}, 'web-services': {u'dataset': 0.019069572506286672, u'given': 0.019069572506286672, u'join': 0.019069572506286672, u'name': 0.038120094490589042, u'result': 0.019069572506286672, u'get': 0.038120094490589042, u'queri': 0.15242322639640327, u'elegantli': 0.019069572506286672, u'express': 0.019069572506286672, u'resultset': 0.019069572506286672, u'linq': 0.19052427036500799, u'order': 0.057170616474891411, u'collect': 0.057170616474891411, u'ignor': 0.019069572506286672, u'aggreg': 0.019069572506286672, u'sql': 0.038120094490589042, u'datat': 0.038120094490589042, u'usual': 0.019069572506286672, u'page': 0.038120094490589042, u'left': 0.019069572506286672}, 'analytics': {u'excus': 5.7168991538989253e-05, u'cross-cultur': 5.7168991538989253e-05, u'log': 0.11439515206951748, u'internation': 5.7168991538989253e-05, u'famou': 5.7168991538989253e-05, u'latter': 0.057226160530528231, u'flash': 0.11439515206951748, u'minimum': 5.7168991538989253e-05, u'cross': 5.7168991538989253e-05, u'ensur': 5.7168991538989253e-05, u'binari': 0.057226160530528231, u'host': 0.11439515206951748, u'paragraph': 5.7168991538989253e-05, u'download': 0.22873313514749602, u'unicod': 5.7168991538989253e-05, u'phpeclips': 5.7168991538989253e-05, u'webal': 0.057226160530528231, u'former': 0.057226160530528231, u'good/bett': 5.7168991538989253e-05, u'nice': 0.057226160530528231}, 'legacy': {u'insert': 0.18226511289147848, u'-bit': 0.36434814275309541, u'set': 0.00018208302986161691, u'cross-cultur': 0.00018208302986161691, u'sourcesaf': 0.00018208302986161691, u'imagin': 0.00018208302986161691, u'internation': 0.00018208302986161691, u'famou': 0.00018208302986161691, u'breed': 0.00018208302986161691, u'tester': 0.00018208302986161691, u'claim': 0.00018208302986161691, u'minimum': 0.00018208302986161691, u'aw': 0.00018208302986161691, u'breakpoint': 0.00018208302986161691, u'good/bett': 0.00018208302986161691, u'unicod': 0.00018208302986161691, u'phpeclips': 0.00018208302986161691, u'baggag': 0.00018208302986161691, u'grow': 0.00018208302986161691, u'excus': 0.00018208302986161691}, 'migration': {u'decim': 0.10915902808089625, u'solution"': 0.036410592172268288, u'mythic': 0.036410592172268288, u'"best': 0.036410592172268288, u'float': 0.072784810126582278, u'parser': 0.036410592172268288, u'french': 0.036410592172268288, u'separ': 0.14553324603521026, u'german': 0.036410592172268288, u'american': 0.036410592172268288, u'other': 0.036410592172268288, u'black': 0.036410592172268288, u'multi-cultur': 0.036410592172268288, u'breakpoint': 3.6374217954313981e-05, u'fun': 0.036410592172268288, u'otherwis': 0.036410592172268288, u'en-u': 0.036410592172268288, u'frenchman': 0.036410592172268288, u'non-numer': 0.036410592172268288, u'browser': 0.036410592172268288}, 'web-applications': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'reverse-engineering': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'actionscript-3': {u'hit': 0.074192113845241617, u'investig': 0.074192113845241617, u'unless': 0.074192113845241617, u'xslt': 7.4117995849392229e-05, u'sys/socketh': 0.074192113845241617, u'famou': 7.4117995849392229e-05, u'minimum': 7.4117995849392229e-05, u'thingi': 0.074192113845241617, u'c++': 0.14831010969463385, u'reiter': 7.4117995849392229e-05, u'internation': 7.4117995849392229e-05, u'api': 0.14831010969463385, u'patient': 7.4117995849392229e-05, u'#ifdef': 0.074192113845241617, u'cross-cultur': 7.4117995849392229e-05, u'appar': 0.074192113845241617, u'unicod': 7.4117995849392229e-05, u'bridg': 7.4117995849392229e-05, u'cross': 7.4117995849392229e-05, u'excus': 7.4117995849392229e-05}, 'object-database': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'import': {u'imagin': 5.7168991538989253e-05, u'tripl': 0.11439515206951748, u'set': 5.7168991538989253e-05, u'cross-cultur': 5.7168991538989253e-05, u'sourcesaf': 5.7168991538989253e-05, u'advanc': 0.057226160530528231, u'internation': 5.7168991538989253e-05, u'breed': 5.7168991538989253e-05, u'drop': 0.057226160530528231, u'tester': 5.7168991538989253e-05, u'claim': 5.7168991538989253e-05, u'minimum': 5.7168991538989253e-05, u'breakpoint': 5.7168991538989253e-05, u'famou': 5.7168991538989253e-05, u'good/bett': 5.7168991538989253e-05, u'databound': 0.17156414360850672, u'funciton': 0.057226160530528231, u'phpeclips': 5.7168991538989253e-05, u'grow': 5.7168991538989253e-05, u'paramet': 0.40024010976446378}, 'email': {u'imagin': 0.00013347570742124932, u'tester': 0.00013347570742124932, u'set': 0.00013347570742124932, u'cross-cultur': 0.00013347570742124932, u'sourcesaf': 0.00013347570742124932, u'claim': 0.00013347570742124932, u'process': 0.26708489054991991, u'famou': 0.00013347570742124932, u'top': 0.13360918312867057, u'breed': 0.00013347570742124932, u'internation': 0.00013347570742124932, u'step': 0.26708489054991991, u'minimum': 0.00013347570742124932, u'breakpoint': 0.00013347570742124932, u'good/bett': 0.00013347570742124932, u'unicod': 0.00013347570742124932, u'phpeclips': 0.00013347570742124932, u'baggag': 0.00013347570742124932, u'grow': 0.00013347570742124932, u'excus': 0.00013347570742124932}, 'xslt': {u'function': 0.19759458658500445, u'claim': 2.4696236293588858e-05, u'remedi': 0.024720932529882444, u'doubl': 0.024720932529882444, u'set': 2.4696236293588858e-05, u'ascx': 0.024720932529882444, u'defined"': 0.024720932529882444, u'"feeamountcheck': 0.024720932529882444, u'javascript': 0.24698705917218217, u'due': 0.024720932529882444, u'integ': 0.049417168823471302, u'custom': 0.024720932529882444, u'imagin': 2.4696236293588858e-05, u'valid': 0.098809641410649032, u'breakpoint': 2.4696236293588858e-05, u'ensur': 0.024720932529882444, u'containerdataitem': 0.024720932529882444, u'grow': 2.4696236293588858e-05, u'singl': 0.049417168823471302, u'aspnet': 0.07411340511706016}, 'javascript': {u'unabl': 0.022003868812098829, u'nunit': 0.022003868812098829, u'spars': 0.022003868812098829, u'task': 0.043985755737272487, u'cruisecontrolnet': 0.065967642662446146, u'offer': 0.022003868812098829, u'mainli': 0.022003868812098829, u'/poutputpath': 0.022003868812098829, u'compon': 0.022003868812098829, u'work;': 0.022003868812098829, u'instruct': 0.022003868812098829, u'msbuild': 0.10993141651279348, u'adequ': 0.022003868812098829, u'build': 0.13191330343796712, u'unknown': 0.022003868812098829, u'cruis': 0.043985755737272487, u'document': 0.043985755737272487, u'ccnetartifactdirectori': 0.065967642662446146, u'mstest': 0.022003868812098829, u'googl': 0.022003868812098829}, 'article': {u'subtop': 0.018713078591191205, u'strength': 0.018713078591191205, u'http': 0.056101847005159654, u'url': 0.037407462798175431, u'checklist': 0.037407462798175431, u'username/password': 0.018713078591191205, u'question': 0.037407462798175431, u'authent': 0.22435130486801766, u'public': 0.018713078591191205, u'connect': 0.018713078591191205, u'intern': 0.018713078591191205, u'base': 0.074796231212143877, u'subject': 0.018713078591191205, u'user': 0.13087938383309655, u'forgotten': 0.018713078591191205, u'websites"': 0.018713078591191205, u'password': 0.074796231212143877, u'valid': 0.037407462798175431, u'digest': 0.018713078591191205, u'includ': 0.037407462798175431}, 'c': {u'core': 0.041976724668494371, u'memory-bound': 0.014001566608851337, u'thread': 0.083939461757958939, u'eleg': 0.014001566608851337, u'mis-behaviour': 0.014001566608851337, u'gotten': 0.014001566608851337, u'period': 0.027989145638672854, u'c++': 0.055964303698315898, u'except': 0.027989145638672854, u'behaviour': 0.027989145638672854, u'written': 0.027989145638672854, u'calcul': 0.014001566608851337, u'throughput': 0.027989145638672854, u'send': 0.083939461757958939, u'time': 0.11191461981760195, u'corrupt': 0.097927040787780445, u'guarante': 0.014001566608851337, u'disk': 0.014001566608851337, u'fifteen': 0.014001566608851337, u'cpu': 0.083939461757958939}, 'exchange-server': {u'count': 0.085177932913332186, u'powershel': 0.042610250297973773, u'iceberg': 4.2567682615358418e-05, u'dump': 0.042610250297973773, u'deploy': 0.17031329814404905, u'lastli': 0.042610250297973773, u'tip': 4.2567682615358418e-05, u'sandcastl': 4.2567682615358418e-05, u'non-invit': 4.2567682615358418e-05, u'somebodi': 4.2567682615358418e-05, u'ensur': 4.2567682615358418e-05, u'live': 0.25544866337476591, u'bridg': 4.2567682615358418e-05, u'export': 0.042610250297973773, u'sql-script': 0.042610250297973773, u'eye': 4.2567682615358418e-05, u'gate': 0.042610250297973773, u'guy': 0.042610250297973773, u'verification/convers': 0.042610250297973773, u'red': 0.042610250297973773}, 'triggers': {u'\u03c0': 0.097647862580519218, u'gcc': 0.048848331056021856, u'constant': 0.097647862580519218, u'auto-fold': 0.048848331056021856, u'compar': 0.097647862580519218, u'internation': 4.879953152449736e-05, u'configur': 0.048848331056021856, u'tip': 4.879953152449736e-05, u'clearli': 0.048848331056021856, u'here': 0.048848331056021856, u'd': 0.048848331056021856, u'person': 0.048848331056021856, u'stori': 4.879953152449736e-05, u'portable;': 0.048848331056021856, u'cross-cultur': 4.879953152449736e-05, u'world': 4.879953152449736e-05, u'apart': 0.048848331056021856, u'inlin': 0.097647862580519218, u'war': 4.879953152449736e-05, u'theori': 0.048848331056021856}, 'country': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'connection': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'berkeley-db': {u'imagin': 0.00010535187526337968, u'messagebox': 0.10545722713864304, u'excus': 0.00010535187526337968, u'cross-cultur': 0.00010535187526337968, u'sourcesaf': 0.00010535187526337968, u'claim': 0.00010535187526337968, u'internation': 0.00010535187526337968, u'win': 0.10545722713864304, u'breed': 0.00010535187526337968, u'famou': 0.00010535187526337968, u'pop': 0.10545722713864304, u'wrong': 0.10545722713864304, u'minimum': 0.00010535187526337968, u'exit': 0.10545722713864304, u'breakpoint': 0.00010535187526337968, u'horribl': 0.10545722713864304, u'unicod': 0.00010535187526337968, u'phpeclips': 0.00010535187526337968, u'grow': 0.00010535187526337968, u'probabl': 0.10545722713864304}, 'pdf': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'com': {u'claim': 7.4117995849392229e-05, u'regardless': 0.074192113845241617, u'sourcesaf': 7.4117995849392229e-05, u'tester': 7.4117995849392229e-05, u'internation': 7.4117995849392229e-05, u'famou': 7.4117995849392229e-05, u'builder': 0.074192113845241617, u'breed': 7.4117995849392229e-05, u'devic': 0.074192113845241617, u'histor': 0.074192113845241617, u'landscap': 0.14831010969463385, u'good/bett': 7.4117995849392229e-05, u'imagin': 7.4117995849392229e-05, u'minimum': 7.4117995849392229e-05, u'mode': 0.14831010969463385, u'iphon': 0.14831010969463385, u'posit': 0.074192113845241617, u'cross-cultur': 7.4117995849392229e-05, u'phpeclips': 7.4117995849392229e-05, u'grow': 7.4117995849392229e-05}, 'datediff': {u'imagin': 8.7017055342847191e-05, u'given': 0.087104072398190027, u'cross-cultur': 8.7017055342847191e-05, u'sourcesaf': 8.7017055342847191e-05, u'tester': 8.7017055342847191e-05, u'internation': 8.7017055342847191e-05, u'famou': 8.7017055342847191e-05, u'age': 0.17412112774103722, u'breed': 8.7017055342847191e-05, u'minimum': 8.7017055342847191e-05, u'someon': 0.087104072398190027, u'repres': 0.087104072398190027, u'person': 0.087104072398190027, u'calcul': 0.17412112774103722, u'birthday': 0.087104072398190027, u'breakpoint': 8.7017055342847191e-05, u'unicod': 8.7017055342847191e-05, u'phpeclips': 8.7017055342847191e-05, u'baggag': 8.7017055342847191e-05, u'grow': 8.7017055342847191e-05}, 'version-control': {u'silverlight': 0.064613994319648843, u'wmp': 0.064613994319648843, u'firefox': 0.19371288406919698, u'english': 6.4549444874774072e-05, u'univers': 0.064613994319648843, u'tip': 6.4549444874774072e-05, u'site': 0.12916343919442291, u'ensur': 6.4549444874774072e-05, u'wmv': 0.064613994319648843, u'player': 0.12916343919442291, u'sandcastl': 6.4549444874774072e-05, u'bridg': 6.4549444874774072e-05, u'deliv': 0.064613994319648843, u'eye': 6.4549444874774072e-05, u'iceberg': 6.4549444874774072e-05, u'xp': 0.064613994319648843, u'reiter': 6.4549444874774072e-05, u'cross': 6.4549444874774072e-05, u'xslt': 6.4549444874774072e-05, u'excus': 6.4549444874774072e-05}, 'linux': {u'flat': 0.22242810554402606, u'matur': 0.074192113845241617, u'claim': 7.4117995849392229e-05, u'cross-cultur': 7.4117995849392229e-05, u'eleg': 0.074192113845241617, u'sql-like': 0.074192113845241617, u'queri': 0.074192113845241617, u'top': 0.074192113845241617, u'breed': 7.4117995849392229e-05, u'famou': 7.4117995849392229e-05, u'syntax': 0.074192113845241617, u'trick': 0.074192113845241617, u'internation': 7.4117995849392229e-05, u'minimum': 7.4117995849392229e-05, u'overhead': 0.074192113845241617, u'breakpoint': 7.4117995849392229e-05, u'unicod': 7.4117995849392229e-05, u'phpeclips': 7.4117995849392229e-05, u'grow': 7.4117995849392229e-05, u'excus': 7.4117995849392229e-05}, 'timezoneoffset': {u'msdn-style': 7.4117995849392229e-05, u'imagin': 7.4117995849392229e-05, u'breed': 7.4117995849392229e-05, u'cross-cultur': 7.4117995849392229e-05, u'zone': 0.14831010969463385, u'header': 0.074192113845241617, u'internation': 7.4117995849392229e-05, u'famou': 7.4117995849392229e-05, u'time': 0.14831010969463385, u'descript': 0.074192113845241617, u'breakpoint': 7.4117995849392229e-05, u'claim': 7.4117995849392229e-05, u'user-ag': 0.074192113845241617, u'perhap': 0.074192113845241617, u'minimum': 7.4117995849392229e-05, u'determin': 0.14831010969463385, u'offset': 0.074192113845241617, u'unicod': 7.4117995849392229e-05, u'phpeclips': 7.4117995849392229e-05, u'grow': 7.4117995849392229e-05}, 'mailto': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'medium-trust': {u'excus': 4.4460252534234394e-05, u'triangl': 0.044504712786768624, u'valid': 0.044504712786768624, u'whenev': 0.044504712786768624, u'cross': 4.4460252534234394e-05, u'tupl': 0.044504712786768624, u'normal': 0.044504712786768624, u'minimum': 4.4460252534234394e-05, u'three': 0.044504712786768624, u'etc;': 0.044504712786768624, u'aim': 0.044504712786768624, u'other': 0.044504712786768624, u'combin': 0.088964965321003009, u'unicod': 4.4460252534234394e-05, u'neg': 0.044504712786768624, u'test': 0.17788547038947183, u'--will': 0.044504712786768624, u'close-to': 0.044504712786768624, u'repetit': 0.044504712786768624, u'fring': 0.044504712786768624}, 'latitude-longitude': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'java': {u'claim': 9.5310712924132673e-05, u'sql-base': 0.095406023637056789, u'excus': 9.5310712924132673e-05, u'iceberg': 9.5310712924132673e-05, u'internation': 9.5310712924132673e-05, u'famou': 9.5310712924132673e-05, u'breakpoint': 9.5310712924132673e-05, u'tip': 9.5310712924132673e-05, u'good/bett': 9.5310712924132673e-05, u'membership': 0.095406023637056789, u'minimum': 9.5310712924132673e-05, u'video': 0.095406023637056789, u'cross-cultur': 9.5310712924132673e-05, u'english': 9.5310712924132673e-05, u'ensur': 9.5310712924132673e-05, u'tie': 0.095406023637056789, u'phpeclips': 9.5310712924132673e-05, u'site-map': 0.095406023637056789, u'page': 0.19071673656118945, u'view': 0.095406023637056789}, 'rdbms': {u'imagin': 5.7168991538989253e-05, u'ultim': 0.057226160530528231, u'set': 5.7168991538989253e-05, u'cross-cultur': 5.7168991538989253e-05, u'monitor': 0.057226160530528231, u'sourcesaf': 5.7168991538989253e-05, u'claim': 5.7168991538989253e-05, u'internation': 5.7168991538989253e-05, u'checksum': 0.057226160530528231, u'phpeclips': 5.7168991538989253e-05, u'breed': 5.7168991538989253e-05, u'minimum': 5.7168991538989253e-05, u'binary_checksum': 0.057226160530528231, u'plan': 0.057226160530528231, u'famou': 5.7168991538989253e-05, u'good/bett': 5.7168991538989253e-05, u'chang': 0.34307111822547448, u'third-parti': 0.057226160530528231, u'grow': 5.7168991538989253e-05, u'valu': 0.17156414360850672}, 'encryption': {u'claim': 5.4077438892494048e-05, u'domain\\aus': 0.054131516331386534, u'network': 0.054131516331386534, u'iprincipalnam': 0.054131516331386534, u'internation': 5.4077438892494048e-05, u'visitor': 0.054131516331386534, u'configur': 0.10820895522388059, u'imperson': 0.054131516331386534, u'friction': 0.054131516331386534, u'breakpoint': 5.4077438892494048e-05, u'record': 0.054131516331386534, u'experiment': 0.054131516331386534, u'rememb': 0.054131516331386534, u'zeu': 0.054131516331386534, u'debug': 0.054131516331386534, u'login': 0.054131516331386534, u'phpeclips': 5.4077438892494048e-05, u'annoyingli': 0.054131516331386534, u'anonym': 0.054131516331386534, u'good/bett': 5.4077438892494048e-05}, 'tag-cloud': {u'lowest': 0.074192113845241617, u'claim': 7.4117995849392229e-05, u'tester': 7.4117995849392229e-05, u'set': 7.4117995849392229e-05, u'cross-cultur': 7.4117995849392229e-05, u'sourcesaf': 7.4117995849392229e-05, u'artist': 0.074192113845241617, u'internation': 7.4117995849392229e-05, u'famou': 7.4117995849392229e-05, u'breed': 7.4117995849392229e-05, u'minimum': 7.4117995849392229e-05, u'imagin': 7.4117995849392229e-05, u'tag': 0.2965461013934183, u'enclos': 0.14831010969463385, u'breakpoint': 7.4117995849392229e-05, u'good/bett': 7.4117995849392229e-05, u'phpeclips': 7.4117995849392229e-05, u'grow': 7.4117995849392229e-05, u'cloud': 0.14831010969463385, u'higher': 0.074192113845241617}, 'projects-and-solutions': {u'imagin': 0.00010535187526337968, u'layer': 0.10545722713864304, u'set': 0.00010535187526337968, u'group': 0.10545722713864304, u'unicod': 0.00010535187526337968, u'sourcesaf': 0.00010535187526337968, u'claim': 0.00010535187526337968, u'internation': 0.00010535187526337968, u'famou': 0.00010535187526337968, u'breed': 0.00010535187526337968, u'solut': 0.21080910240202272, u'tester': 0.00010535187526337968, u'minimum': 0.00010535187526337968, u'splite': 0.10545722713864304, u'breakpoint': 0.00010535187526337968, u'good/bett': 0.00010535187526337968, u'cross-cultur': 0.00010535187526337968, u'folder': 0.21080910240202272, u'grow': 0.00010535187526337968, u'excus': 0.00010535187526337968}, 'collections': {u'accordingli': 0.037784991695606215, u'feedback': 0.037784991695606215, u'lamp/net': 0.037784991695606215, u'flesh': 0.037784991695606215, u'ip': 0.037784991695606215, u'safe': 0.037784991695606215, u'bandwidth': 0.037784991695606215, u'ideal': 0.075532236146761286, u'tester': 3.7747244451155068e-05, u'blacklist': 0.037784991695606215, u'hardwar': 0.037784991695606215, u'perhap': 0.037784991695606215, u'requester/add': 0.037784991695606215, u'inject': 0.11327948059791634, u'sql': 0.11327948059791634, u'ban': 0.037784991695606215, u'technolog': 0.037784991695606215, u'tip': 3.7747244451155068e-05, u'attack': 0.075532236146761286, u'stack;': 0.037784991695606215}, 'pi': {u'assembl': 0.053371385895657743, u'ago': 0.026699029126213591, u'-bit': 0.053371385895657743, u'welcom': 0.026699029126213591, u'optimis': 0.026699029126213591, u'sourcesaf': 2.6672356769444152e-05, u'vi': 0.10671609943454605, u'top': 0.026699029126213591, u'built-in': 0.026699029126213591, u'obtain': 0.026699029126213591, u'flag': 0.026699029126213591, u'version': 0.1333884562039902, u'somebodi': 2.6672356769444152e-05, u'fastest': 0.1333884562039902, u'world': 2.6672356769444152e-05, u'test': 0.16006081297343436, u'x': 0.053371385895657743, u'stori': 2.6672356769444152e-05, u'hard-cod': 0.026699029126213591, u'baselin': 0.026699029126213591}, 'data-binding': {u'imagin': 0.00011775788977861516, u'set': 0.00011775788977861516, u'cross-cultur': 0.00011775788977861516, u'name': 0.23563353744700891, u'sourcesaf': 0.00011775788977861516, u'claim': 0.00011775788977861516, u'internation': 0.00011775788977861516, u'famou': 0.00011775788977861516, u'breed': 0.00011775788977861516, u'minimum': 0.00011775788977861516, u'correspond': 0.11787564766839376, u'psd': 0.11787564766839376, u'disc': 0.11787564766839376, u'breakpoint': 0.00011775788977861516, u'good/bett': 0.00011775788977861516, u'unicod': 0.00011775788977861516, u'phpeclips': 0.00011775788977861516, u'grow': 0.00011775788977861516, u'physic': 0.11787564766839376, u'excus': 0.00011775788977861516}, 'pocketpc': {u'imagin': 0.00011775788977861516, u'set': 0.00011775788977861516, u'cross-cultur': 0.00011775788977861516, u'sourcesaf': 0.00011775788977861516, u'directli': 0.11787564766839376, u'claim': 0.00011775788977861516, u'internation': 0.00011775788977861516, u'famou': 0.00011775788977861516, u'screen': 0.23563353744700891, u'breed': 0.00011775788977861516, u'excus': 0.00011775788977861516, u'specifi': 0.11787564766839376, u'minimum': 0.00011775788977861516, u'unicod': 0.00011775788977861516, u'breakpoint': 0.00011775788977861516, u'good/bett': 0.00011775788977861516, u'line': 0.11787564766839376, u'phpeclips': 0.00011775788977861516, u'grow': 0.00011775788977861516, u'paramet': 0.11787564766839376}, 'extreme-programming': {u'excus': 6.9003588186585703e-05, u'iceberg': 6.9003588186585703e-05, u'nag': 0.069072591774772277, u'minimum': 6.9003588186585703e-05, u'internation': 6.9003588186585703e-05, u'hobbi': 0.069072591774772277, u'user-friendli': 0.069072591774772277, u'breed': 6.9003588186585703e-05, u'tip': 6.9003588186585703e-05, u'continu': 0.20707976814794368, u'vcse': 0.069072591774772277, u'open-sourc': 0.069072591774772277, u'check-in': 0.069072591774772277, u'suitabl': 0.069072591774772277, u'cross-cultur': 6.9003588186585703e-05, u'english': 6.9003588186585703e-05, u'test': 0.13807617996135799, u'famou': 6.9003588186585703e-05, u'phpeclips': 6.9003588186585703e-05, u'good/bett': 6.9003588186585703e-05}, 'c#': {u'applic': 0.0081281966007235289, u'ive': 0.0088055322062920403, u'use': 0.040809639569404206, u'code': 0.0084668644035077855, u'like': 0.015917556064761411, u'would': 0.015748222163369286, u'get': 0.013377547543879494, u'make': 0.0072815270937628901, u'work': 0.014393550952232262, u'know': 0.0093135339104684235, u'one': 0.010837539022997576, u'look': 0.0082975305021156572, u'im': 0.017102893374506309, u'user': 0.0093135339104684235, u'way': 0.01456288485362439, u'need': 0.0099908695160369366, u'file': 0.0089748661076841686, u'seem': 0.0069428592909786344, u'someth': 0.0071121931923707618, u'want': 0.011006872924389705}, 'bayesian': {u'oo': 0.030807583405145882, u'anoth': 0.030807583405145882, u'classif': 0.030807583405145882, u'purpos': 0.030807583405145882, u'manipul': 0.030807583405145882, u'claim': 3.0776806598547338e-05, u'text': 0.21546842299642993, u'imag': 0.18469161639788259, u'within': 0.061584390003693215, u'breed': 3.0776806598547338e-05, u'bound': 0.030807583405145882, u'filter': 0.061584390003693215, u'imagin': 3.0776806598547338e-05, u'intent': 0.030807583405145882, u'phpeclips': 3.0776806598547338e-05, u'learn': 0.030807583405145882, u'bayesian': 0.061584390003693215, u'ocr': 0.061584390003693215, u'grow': 3.0776806598547338e-05, u'aforgenet': 0.030807583405145882}, 'ruby': {u'imagin': 6.0635459616783893e-05, u'cannot': 0.060696095076400669, u'set': 6.0635459616783893e-05, u'cross-cultur': 6.0635459616783893e-05, u'unicod': 6.0635459616783893e-05, u'claim': 6.0635459616783893e-05, u'internation': 6.0635459616783893e-05, u'famou': 6.0635459616783893e-05, u'breed': 6.0635459616783893e-05, u'list': 0.060696095076400669, u'minimum': 6.0635459616783893e-05, u'offic': 0.18196701430996845, u'base': 0.060696095076400669, u'mime': 0.18196701430996845, u'breakpoint': 6.0635459616783893e-05, u'good/bett': 6.0635459616783893e-05, u'seem': 0.060696095076400669, u'phpeclips': 6.0635459616783893e-05, u'type': 0.24260247392675238, u'grow': 6.0635459616783893e-05}, 'datatable': {u'enterpris': 0.032042363107116056, u'vendor': 0.013735056661113451, u'structur': 0.013735056661113451, u'column': 0.032042363107116056, u'databas': 0.073233802610621915, u'copi': 0.041196016330117352, u'-': 0.022888709884114754, u'procedur': 0.013735056661113451, u'server': 0.12357889533712905, u'record': 0.027465536495615402, u'trigger': 0.022888709884114754, u'studio': 0.027465536495615402, u'connect': 0.018311883272614102, u'sql': 0.1281557219486297, u'tabl': 0.086964282445123858, u'queri': 0.022888709884114754, u'data': 0.041196016330117352, u'manag': 0.027465536495615402, u'store': 0.022888709884114754, u'logic': 0.013735056661113451}, 'floating-point': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'exception': {u'imagin': 8.0051232788984944e-05, u'what': 0.080131284021773927, u'cross-cultur': 8.0051232788984944e-05, u'sourcesaf': 8.0051232788984944e-05, u'claim': 8.0051232788984944e-05, u'recordset': 0.080131284021773927, u'databas': 0.16018251681075887, u'breakpoint': 8.0051232788984944e-05, u'famou': 8.0051232788984944e-05, u'tester': 8.0051232788984944e-05, u'record': 0.080131284021773927, u'minimum': 8.0051232788984944e-05, u'queri': 0.080131284021773927, u'connect': 0.16018251681075887, u'simplest': 0.080131284021773927, u'good/bett': 8.0051232788984944e-05, u'phpeclips': 8.0051232788984944e-05, u'grow': 8.0051232788984944e-05, u'loop': 0.080131284021773927, u'internation': 8.0051232788984944e-05}, 'glossary': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'hyperlink': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'objective-c': {u'comment': 0.03926722108896908, u'playgroundbluess': 0.03926722108896908, u'tester': 3.9227993095873212e-05, u'breed': 3.9227993095873212e-05, u'cross-cultur': 3.9227993095873212e-05, u'sourcesaf': 3.9227993095873212e-05, u'give': 0.078495214184842302, u'internation': 3.9227993095873212e-05, u'hanselman': 0.03926722108896908, u'breakpoint': 3.9227993095873212e-05, u'site': 0.15695120037658875, u'claim': 3.9227993095873212e-05, u'imagin': 3.9227993095873212e-05, u'link': 0.03926722108896908, u'iphon': 0.11772320728071552, u'good/bett': 3.9227993095873212e-05, u'x': 0.15695120037658875, u'icon': 0.15695120037658875, u'grow': 3.9227993095873212e-05, u'size': 0.078495214184842302}, 'air': {u'claim': 5.130309870716191e-05, u'set': 5.130309870716191e-05, u'cross-cultur': 5.130309870716191e-05, u'unicod': 5.130309870716191e-05, u'equival': 0.051354401805869067, u'orgeclipseui': 0.051354401805869067, u'remot': 0.10265750051303098, u'famou': 5.130309870716191e-05, u'breed': 5.130309870716191e-05, u'minimum': 5.130309870716191e-05, u'upgrad': 0.051354401805869067, u'subvers': 0.1539605992201929, u'internation': 5.130309870716191e-05, u'imagin': 5.130309870716191e-05, u'version': 0.20526369792735483, u'good/bett': 5.130309870716191e-05, u'grow': 5.130309870716191e-05, u'phpeclips': 5.130309870716191e-05, u'relev': 0.051354401805869067, u'subclips': 0.20526369792735483}, 'dvcs': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'mainframe': {u'c#': 0.019069572506286672, u'socket': 0.057170616474891411, u'receiv': 0.057170616474891411, u'acknowledg': 0.019069572506286672, u'requestsjust': 0.019069572506286672, u'request': 0.057170616474891411, u'messag': 0.057170616474891411, u'server': 0.17147374838070564, u'ever': 0.019069572506286672, u'buffer': 0.019069572506286672, u'client': 0.13337270441210092, u'mobil': 0.019069572506286672, u'devic': 0.019069572506286672, u'alreadi': 0.019069572506286672, u'client-serv': 0.019069572506286672, u'affect': 0.019069572506286672, u'listen': 0.019069572506286672, u'data': 0.076221138459193788, u'respect': 0.019069572506286672, u'wait': 0.019069572506286672}, 'anchor': {u'data-typ': 0.057226160530528231, u'imagin': 5.7168991538989253e-05, u'tester': 5.7168991538989253e-05, u'breed': 5.7168991538989253e-05, u'cross-cultur': 5.7168991538989253e-05, u'option': 0.11439515206951748, u'sourcesaf': 5.7168991538989253e-05, u'constraint': 0.057226160530528231, u'internation': 5.7168991538989253e-05, u'famou': 5.7168991538989253e-05, u'creator': 0.057226160530528231, u'descript': 0.057226160530528231, u'claim': 5.7168991538989253e-05, u'meta': 0.057226160530528231, u'dictionari': 0.17156414360850672, u'breakpoint': 5.7168991538989253e-05, u'good/bett': 5.7168991538989253e-05, u'phpeclips': 5.7168991538989253e-05, u'data': 0.28590212668648524, u'grow': 5.7168991538989253e-05}, 'malloc': {u'hip': 0.025346905702420743, u'absolut': 0.025346905702420743, u'rtf': 0.025346905702420743, u'help': 0.22791957864884027, u'especi': 0.025346905702420743, u'webpag': 0.025346905702420743, u'nowaday': 0.025346905702420743, u'bottom': 0.025346905702420743, u'everyon': 0.025346905702420743, u'caus': 0.050668489820723188, u'network': 0.075990073939025626, u'run"': 0.025346905702420743, u'"no': 0.025346905702420743, u'reader': 0.025346905702420743, u'robust': 0.025346905702420743, u'pdf': 0.025346905702420743, u'extract': 0.025346905702420743, u'setup': 0.025346905702420743, u'special': 0.025346905702420743, u'unus': 0.050668489820723188}, 'oledb': {u'compact': 0.040870488322717613, u'claim': 4.0829658664053568e-05, u'undocu': 0.040870488322717613, u'network': 0.16335946431487833, u'first-class': 0.040870488322717613, u'internation': 4.0829658664053568e-05, u'famou': 4.0829658664053568e-05, u'price': 0.040870488322717613, u'breed': 4.0829658664053568e-05, u'drive': 0.040870488322717613, u'trial': 0.040870488322717613, u'blackfish': 0.040870488322717613, u'integr': 0.08170014698677118, u'caveat': 0.040870488322717613, u'breakpoint': 4.0829658664053568e-05, u'good/bett': 4.0829658664053568e-05, u'embed': 0.08170014698677118, u'phpeclips': 4.0829658664053568e-05, u'net': 0.20418912297893191, u'firebird': 0.040870488322717613}, 'data-structures': {u'claim': 4.0829658664053568e-05, u'alloc': 0.08170014698677118, u'set': 4.0829658664053568e-05, u'cross-cultur': 4.0829658664053568e-05, u'string': 0.32667809897109257, u'space': 0.040870488322717613, u'internation': 4.0829658664053568e-05, u'realloc': 0.040870488322717613, u'doubl': 0.08170014698677118, u'well': 0.040870488322717613, u'initi': 0.040870488322717613, u'shorter': 0.040870488322717613, u'minimum': 4.0829658664053568e-05, u'instanc': 0.040870488322717613, u'free': 0.08170014698677118, u'breakpoint': 4.0829658664053568e-05, u'breed': 4.0829658664053568e-05, u'famou': 4.0829658664053568e-05, u'phpeclips': 4.0829658664053568e-05, u'help': 0.08170014698677118}, 'vi': {u'//': 0.042610250297973773, u'est': 0.042610250297973773, u'good': 0.042610250297973773, u'recogn': 0.042610250297973773, u'monkey': 0.042610250297973773, u'sourcesaf': 4.2567682615358418e-05, u'nonetheless': 0.042610250297973773, u'classpatchclass': 0.042610250297973773, u'object': 0.042610250297973773, u'punch': 0.042610250297973773, u'ryan': 0.042610250297973773, u'patch': 0.1277456155286906, u'attach': 0.042610250297973773, u'ident': 0.042610250297973773, u'downey': 0.042610250297973773, u'treat': 0.042610250297973773, u'duck': 0.042610250297973773, u'callabl': 0.042610250297973773, u'exactli': 0.042610250297973773, u'automag': 0.042610250297973773}, 'io': {u'beta': 0.033941407839414073, u'uncomfort': 0.033941407839414073, u'term': 0.033941407839414073, u'short': 0.033941407839414073, u'cruisecontrolnet': 0.033941407839414073, u'especi': 0.06784890817848907, u'almost': 0.033941407839414073, u'judg': 0.033941407839414073, u'"official"': 0.033941407839414073, u'happi': 0.033941407839414073, u'internation': 3.3907500339075e-05, u'unsupport': 0.033941407839414073, u'msbuild': 0.13566390885663909, u'build': 0.10175640851756408, u'stuff': 0.10175640851756408, u'front': 0.033941407839414073, u'/v': 0.033941407839414073, u'autom': 0.033941407839414073, u'prematur': 0.033941407839414073, u'suitabl': 0.033941407839414073}, 'mouse': {u'imagin': 9.5310712924132673e-05, u'breed': 9.5310712924132673e-05, u'hide': 0.095406023637056789, u'sourcesaf': 9.5310712924132673e-05, u'ternari': 0.095406023637056789, u'show': 0.095406023637056789, u'fals': 0.095406023637056789, u'famou': 9.5310712924132673e-05, u'express': 0.19071673656118945, u'element': 0.095406023637056789, u'claim': 9.5310712924132673e-05, u'internation': 9.5310712924132673e-05, u'flag': 0.095406023637056789, u'minimum': 9.5310712924132673e-05, u'breakpoint': 9.5310712924132673e-05, u'good/bett': 9.5310712924132673e-05, u'cross-cultur': 9.5310712924132673e-05, u'unicod': 9.5310712924132673e-05, u'grow': 9.5310712924132673e-05, u'excus': 9.5310712924132673e-05}, 'binary': {u'imagin': 8.0051232788984944e-05, u'claim': 8.0051232788984944e-05, u'excus': 8.0051232788984944e-05, u'set': 8.0051232788984944e-05, u'cross-cultur': 8.0051232788984944e-05, u'older': 0.080131284021773927, u'internation': 8.0051232788984944e-05, u'famou': 8.0051232788984944e-05, u'breed': 8.0051232788984944e-05, u'hex': 0.080131284021773927, u'good/bett': 8.0051232788984944e-05, u'beta': 0.24023374959974381, u'minimum': 8.0051232788984944e-05, u'letter': 0.080131284021773927, u'breakpoint': 8.0051232788984944e-05, u'octal': 0.24023374959974381, u'unicod': 8.0051232788984944e-05, u'phpeclips': 8.0051232788984944e-05, u'grow': 8.0051232788984944e-05, u'summari': 0.080131284021773927}, 'media': {u'excus': 5.4077438892494048e-05, u'iceberg': 5.4077438892494048e-05, u'unicod': 5.4077438892494048e-05, u'especi': 0.054131516331386534, u'internation': 5.4077438892494048e-05, u'media': 0.10820895522388059, u'emb': 0.054131516331386534, u'famou': 5.4077438892494048e-05, u'tip': 5.4077438892494048e-05, u'english': 5.4077438892494048e-05, u'ensur': 5.4077438892494048e-05, u'minimum': 5.4077438892494048e-05, u'video': 0.054131516331386534, u'explor': 0.16228639411637463, u'internet': 0.16228639411637463, u'good/bett': 5.4077438892494048e-05, u'cross-cultur': 5.4077438892494048e-05, u'embed': 0.10820895522388059, u'user-agent-detect': 0.054131516331386534, u'browser': 0.10820895522388059}, 'authentication': {u'teamciti': 0.048848331056021856, u'made': 0.048848331056021856, u'cross-cultur': 4.879953152449736e-05, u'good/bett': 4.879953152449736e-05, u'eleg': 0.048848331056021856, u'python': 0.14644739410501659, u'excus': 4.879953152449736e-05, u'minimum': 4.879953152449736e-05, u'send': 0.048848331056021856, u'test-cas': 0.048848331056021856, u'cruisecontrol': 0.048848331056021856, u'set': 4.879953152449736e-05, u'integr': 0.14644739410501659, u'codebas': 0.14644739410501659, u'e-mail': 0.048848331056021856, u'unicod': 4.879953152449736e-05, u'phpeclips': 4.879953152449736e-05, u'hook': 0.048848331056021856, u'batteri': 0.048848331056021856, u'internation': 4.879953152449736e-05}, 'html': {u'imagin': 0.00010535187526337968, u'convert': 0.10545722713864304, u'doubl': 0.21080910240202272, u'track-bar': 0.10545722713864304, u'sourcesaf': 0.00010535187526337968, u'form': 0.21080910240202272, u'claim': 0.00010535187526337968, u'internation': 0.00010535187526337968, u'breed': 0.00010535187526337968, u'decim': 0.10545722713864304, u'famou': 0.00010535187526337968, u'tester': 0.00010535187526337968, u'minimum': 0.00010535187526337968, u'set': 0.00010535187526337968, u'breakpoint': 0.00010535187526337968, u'cross-cultur': 0.00010535187526337968, u'unicod': 0.00010535187526337968, u'phpeclips': 0.00010535187526337968, u'grow': 0.00010535187526337968, u'excus': 0.00010535187526337968}, 'subdomain': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'passwords': {u'claim': 8.0051232788984944e-05, u'breed': 8.0051232788984944e-05, u'php': 0.080131284021773927, u'word': 0.080131284021773927, u'phpeclips': 8.0051232788984944e-05, u'internation': 8.0051232788984944e-05, u'ever': 0.080131284021773927, u'breakpoint': 8.0051232788984944e-05, u'famou': 8.0051232788984944e-05, u'minimum': 8.0051232788984944e-05, u'glyph': 8.0051232788984944e-05, u'fastest': 0.080131284021773927, u'cross-cultur': 8.0051232788984944e-05, u'choos': 0.080131284021773927, u'good/bett': 8.0051232788984944e-05, u'sandcastl': 8.0051232788984944e-05, u'unicod': 8.0051232788984944e-05, u'password': 0.24023374959974381, u'exact': 0.080131284021773927, u'told': 0.080131284021773927}, 'http': {u'systemexit': 0.069001195622183389, u'fix': 0.046008461326220912, u'http': 0.046008461326220912, u'gareth': 0.023015727030258438, u'`readuntil': 0.023015727030258438, u'track': 0.046008461326220912, u'rubi': 0.023015727030258438, u'make': 0.046008461326220912, u'rail': 0.16097213280603331, u'sign': 0.023015727030258438, u'websit': 0.046008461326220912, u'/usr/local/lib/ruby//timeoutrbin': 0.046008461326220912, u'version': 0.023015727030258438, u'caus': 0.046008461326220912, u'`read_new': 0.023015727030258438, u'error': 0.091993929918145881, u'surpris': 0.023015727030258438, u'exact': 0.023015727030258438, u'`get_respons': 0.046008461326220912, u'`readlin': 0.023015727030258438}, 'wing-ide': {u'x': 0.069072591774772277, u'tester': 6.9003588186585703e-05, u'inlin': 0.069072591774772277, u'cross-cultur': 6.9003588186585703e-05, u'sourcesaf': 6.9003588186585703e-05, u'keyword': 0.069072591774772277, u'imagin': 6.9003588186585703e-05, u'internation': 6.9003588186585703e-05, u'famou': 6.9003588186585703e-05, u'phpeclips': 6.9003588186585703e-05, u'breed': 6.9003588186585703e-05, u'reach': 0.069072591774772277, u'grow': 6.9003588186585703e-05, u'time': 0.069072591774772277, u'good/bett': 6.9003588186585703e-05, u'drastic': 0.069072591774772277, u'ide': 0.13807617996135799, u'speed': 0.069072591774772277, u'spend': 0.069072591774772277, u'complet': 0.13807617996135799}, 'user': {u'claim': 8.0051232788984944e-05, u'account': 0.080131284021773927, u'toy': 0.080131284021773927, u'basecamp': 0.080131284021773927, u'phpeclips': 8.0051232788984944e-05, u'tester': 8.0051232788984944e-05, u'internation': 8.0051232788984944e-05, u'famou': 8.0051232788984944e-05, u'breed': 8.0051232788984944e-05, u'rule': 0.080131284021773927, u'instantli': 0.080131284021773927, u'imagin': 8.0051232788984944e-05, u'minimum': 8.0051232788984944e-05, u'lookup': 0.080131284021773927, u'cross-cultur': 8.0051232788984944e-05, u'breakpoint': 8.0051232788984944e-05, u'good/bett': 8.0051232788984944e-05, u'webapp': 0.080131284021773927, u'subdomain': 0.24023374959974381, u'grow': 8.0051232788984944e-05}, 'wpf': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'php': {u'applic': 0.080043742665101891, u'tick': 0.053371385895657743, u'consol': 0.053371385895657743, u'parent': 0.026699029126213591, u'thread': 0.18673316974287851, u'chosen': 0.026699029126213591, u'famou': 2.6672356769444152e-05, u'phpeclips': 2.6672356769444152e-05, u'drift': 0.026699029126213591, u'three': 0.026699029126213591, u'timer': 0.24007788328176677, u'minimum': 2.6672356769444152e-05, u'internation': 2.6672356769444152e-05, u'busi': 0.080043742665101891, u'rule': 2.6672356769444152e-05, u'breakpoint': 2.6672356769444152e-05, u'good/bett': 2.6672356769444152e-05, u'patient': 2.6672356769444152e-05, u'along': 0.053371385895657743, u'main': 0.080043742665101891}, 'ternary-operator': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'polymorphism': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'filelock': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'reflection': {u'case': 0.23563353744700891, u'imagin': 0.00011775788977861516, u'tester': 0.00011775788977861516, u'set': 0.00011775788977861516, u'cross-cultur': 0.00011775788977861516, u'sourcesaf': 0.00011775788977861516, u'format': 0.23563353744700891, u'internation': 0.00011775788977861516, u'titl': 0.23563353744700891, u'breed': 0.00011775788977861516, u'famou': 0.00011775788977861516, u'claim': 0.00011775788977861516, u'minimum': 0.00011775788977861516, u'breakpoint': 0.00011775788977861516, u'good/bett': 0.00011775788977861516, u'unicod': 0.00011775788977861516, u'phpeclips': 0.00011775788977861516, u'baggag': 0.00011775788977861516, u'grow': 0.00011775788977861516, u'excus': 0.00011775788977861516}, 'database': {u'properitari': 0.019824922760041195, u'scale': 0.019824922760041195, u'pain': 0.019824922760041195, u'clean': 0.019824922760041195, u'relationship': 0.019824922760041195, u'matter': 0.039630040402439989, u'perform': 0.059435158044838787, u'replac': 0.019824922760041195, u'mysql': 0.19807098154163036, u'incred': 0.019824922760041195, u'roughli': 0.019824922760041195, u'emul': 0.019824922760041195, u'record': 0.039630040402439989, u'safe': 0.019824922760041195, u'statement': 0.039630040402439989, u'lose': 0.039630040402439989, u'django': 0.019824922760041195, u'degrad': 0.039630040402439989, u'anybodi': 0.039630040402439989, u'physic': 0.019824922760041195}, 'audio': {u'claim': 3.7747244451155068e-05, u'none': 0.037784991695606215, u'"animated"': 0.037784991695606215, u'bar': 0.11327948059791634, u'monitor': 0.037784991695606215, u'thread': 0.037784991695606215, u'breed': 3.7747244451155068e-05, u'explain': 0.037784991695606215, u'perfectli': 0.037784991695606215, u'statu': 0.037784991695606215, u'curs': 0.037784991695606215, u'progressbar': 0.037784991695606215, u'tutori': 0.037784991695606215, u'particularli': 0.037784991695606215, u'termin': 0.037784991695606215, u'alreadi': 0.037784991695606215, u'progress': 0.15102672504907141, u'straight-forward': 0.037784991695606215, u'ncurs': 0.075532236146761286, u'proceed': 0.037784991695606215}, 'parallel-processing': {u'claim': 7.4117995849392229e-05, u'domain': 0.074192113845241617, u'scale': 0.074192113845241617, u'cross-cultur': 7.4117995849392229e-05, u'phpeclips': 7.4117995849392229e-05, u'internation': 7.4117995849392229e-05, u'famou': 7.4117995849392229e-05, u'setup': 0.074192113845241617, u'breed': 7.4117995849392229e-05, u'subson': 0.14831010969463385, u'term': 0.074192113845241617, u'set': 7.4117995849392229e-05, u'breakpoint': 7.4117995849392229e-05, u'advers': 0.074192113845241617, u'good/bett': 7.4117995849392229e-05, u'tie': 0.074192113845241617, u'unrel': 0.074192113845241617, u'come': 0.074192113845241617, u'grow': 7.4117995849392229e-05, u'quickli': 0.074192113845241617}, 'dependency-properties': {u'imagin': 0.00013347570742124932, u'silverlight': 0.26708489054991991, u'cross-cultur': 0.00013347570742124932, u'aw': 0.00013347570742124932, u'sourcesaf': 0.00013347570742124932, u'tester': 0.00013347570742124932, u'internation': 0.00013347570742124932, u'confus': 0.13360918312867057, u'breed': 0.00013347570742124932, u'famou': 0.00013347570742124932, u'usabl': 0.13360918312867057, u'claim': 0.00013347570742124932, u'minimum': 0.00013347570742124932, u'breakpoint': 0.00013347570742124932, u'baggag': 0.00013347570742124932, u'good/bett': 0.00013347570742124932, u'wpf': 0.13360918312867057, u'phpeclips': 0.00013347570742124932, u'smell': 0.00013347570742124932, u'grow': 0.00013347570742124932}, 'rest': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'quotes': {u'imagin': 2.8992230082337937e-05, u'repeat': 0.058013452394758205, u'cross-cultur': 2.8992230082337937e-05, u'delimit': 0.087005682477096141, u'sourcesaf': 2.8992230082337937e-05, u'quot': 0.17398237272410996, u'internation': 2.8992230082337937e-05, u'famou': 2.8992230082337937e-05, u'phpeclips': 2.8992230082337937e-05, u'knowledg': 0.02902122231242027, u'grow': 2.8992230082337937e-05, u'claim': 2.8992230082337937e-05, u'cheer': 0.02902122231242027, u'onclick': 0.087005682477096141, u'breakpoint': 2.8992230082337937e-05, u'good/bett': 2.8992230082337937e-05, u'input': 0.11599791255943409, u'popul': 0.02902122231242027, u'anchor': 0.11599791255943409, u'string': 0.20297460280644788}, 'unicode': {u'delta': 0.021530585907252861, u'behaviuor': 0.021530585907252861, u'elimin': 0.021530585907252861, u'desper': 0.043039662737675302, u'identifi': 0.043039662737675302, u'g': 0.021530585907252861, u'rub': 0.021530585907252861, u'transfer': 0.021530585907252861, u'prompt': 0.021530585907252861, u'ram': 0.043039662737675302, u'sophist': 0.021530585907252861, u'hyper-thread': 0.021530585907252861, u'multithread': 0.021530585907252861, u'rate': 0.021530585907252861, u'gb': 0.021530585907252861, u'persist': 0.021530585907252861, u'debug': 0.17209412372020993, u'locat': 0.064548739568097743, u'phpeclips': 0.021530585907252861, u'bug': 0.021530585907252861}, 'mysql': {u'altern': 0.080131284021773927, u'via': 0.080131284021773927, u'return': 0.080131284021773927, u'serializ': 0.080131284021773927, u'expos': 0.16018251681075887, u'excus': 8.0051232788984944e-05, u'internation': 8.0051232788984944e-05, u'famou': 8.0051232788984944e-05, u'breed': 8.0051232788984944e-05, u'minimum': 8.0051232788984944e-05, u'ensur': 8.0051232788984944e-05, u'busi': 0.080131284021773927, u'breakpoint': 8.0051232788984944e-05, u'tier': 0.080131284021773927, u'cross-cultur': 8.0051232788984944e-05, u'serial': 0.080131284021773927, u'phpeclips': 8.0051232788984944e-05, u'unicod': 8.0051232788984944e-05, u'good/bett': 8.0051232788984944e-05, u'fill': 0.080131284021773927}, 'internationalization': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'winhelp': {u'canceled"': 0.029887734384330584, u'"navig': 0.029887734384330584, u'cross-cultur': 2.9857876507822766e-05, u'directli': 0.029887734384330584, u'adob': 0.029887734384330584, u'hardli': 0.029887734384330584, u'anymor': 0.059745610892153349, u'future-proof': 0.029887734384330584, u'chm': 0.20903499343126719, u'jump': 0.059745610892153349, u'reliabl': 0.029887734384330584, u'minimum': 2.9857876507822766e-05, u'page/anchor': 0.029887734384330584, u'breakpoint': 2.9857876507822766e-05, u'disadvantag': 0.029887734384330584, u'browser': 0.029887734384330584, u'winhelp': 0.14931924041562167, u'fan': 0.029887734384330584, u'css': 0.029887734384330584, u'nice': 0.089603487399976117}, 'ruby-on-rails': {u'imagin': 4.879953152449736e-05, u'guess': 0.048848331056021856, u'phpeclips': 4.879953152449736e-05, u'cross-cultur': 4.879953152449736e-05, u'sourcesaf': 4.879953152449736e-05, u'internation': 4.879953152449736e-05, u'gotten': 0.048848331056021856, u'clearer': 0.048848331056021856, u'supersed': 0.048848331056021856, u'famou': 4.879953152449736e-05, u'amount': 0.048848331056021856, u'happi': 0.048848331056021856, u'heavi': 0.048848331056021856, u'window': 0.39044505172750338, u'period': 0.048848331056021856, u'breakpoint': 4.879953152449736e-05, u'good/bett': 4.879953152449736e-05, u'toward': 0.048848331056021856, u'grow': 4.879953152449736e-05, u'steer': 0.048848331056021856}, 'exception-handling': {u'oper': 0.079240275687237599, u'thrown': 0.019824922760041195, u'timeout': 0.039630040402439989, u'somewher': 0.019824922760041195, u'lot': 0.039630040402439989, u'succeed': 0.019824922760041195, u'handl': 0.039630040402439989, u'peopl': 0.019824922760041195, u'caught': 0.059435158044838787, u'reach': 0.019824922760041195, u'except': 0.099045393329636383, u'elaps': 0.019824922760041195, u'break': 0.019824922760041195, u'fail': 0.039630040402439989, u'thread': 0.019824922760041195, u'retri': 0.079240275687237599, u'io': 0.079240275687237599, u'wrap': 0.039630040402439989, u'block': 0.039630040402439989, u'duplic': 0.039630040402439989}, 'tuples': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'decompression': {u'imagin': 6.4549444874774072e-05, u'patient': 6.4549444874774072e-05, u'way': 0.064613994319648843, u'sourcesaf': 6.4549444874774072e-05, u'&': 0.064613994319648843, u'internation': 6.4549444874774072e-05, u'famou': 6.4549444874774072e-05, u'breed': 6.4549444874774072e-05, u'compress': 0.12916343919442291, u'decompress': 0.12916343919442291, u'claim': 6.4549444874774072e-05, u'minimum': 6.4549444874774072e-05, u'necessari': 0.064613994319648843, u'file': 0.19371288406919698, u'breakpoint': 6.4549444874774072e-05, u'cross-cultur': 6.4549444874774072e-05, u'folder': 0.12916343919442291, u'glyph': 6.4549444874774072e-05, u'grow': 6.4549444874774072e-05, u'quickli': 0.064613994319648843}, 'css3': {u'iceberg': 4.2567682615358418e-05, u'cross-cultur': 4.2567682615358418e-05, u'string': 0.042610250297973773, u'parent': 0.085177932913332186, u'percentage-bas': 0.042610250297973773, u'pixel-bas': 0.042610250297973773, u'tip': 4.2567682615358418e-05, u'fix': 0.042610250297973773, u'safari': 0.042610250297973773, u'width': 0.29801634599012428, u'minimum': 4.2567682615358418e-05, u'rel': 0.085177932913332186, u'child': 0.17031329814404905, u'ensur': 4.2567682615358418e-05, u'famou': 4.2567682615358418e-05, u'unicod': 4.2567682615358418e-05, u'phpeclips': 4.2567682615358418e-05, u'children': 0.042610250297973773, u'good/bett': 4.2567682615358418e-05, u'internation': 4.2567682615358418e-05}, 'casting': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'format': {u'imagin': 0.00011775788977861516, u'tester': 0.00011775788977861516, u'set': 0.00011775788977861516, u'cross-cultur': 0.00011775788977861516, u'keyword': 0.11787564766839376, u'sourcesaf': 0.00011775788977861516, u'claim': 0.00011775788977861516, u'internation': 0.00011775788977861516, u'famou': 0.00011775788977861516, u'breed': 0.00011775788977861516, u'baggag': 0.00011775788977861516, u'minimum': 0.00011775788977861516, u'aw': 0.00011775788977861516, u'breakpoint': 0.00011775788977861516, u'good/bett': 0.00011775788977861516, u'unicod': 0.00011775788977861516, u'phpeclips': 0.00011775788977861516, u'similar': 0.23563353744700891, u'grow': 0.00011775788977861516, u'view': 0.35339142722562406}, 'deployment': {u'java/c#': 0.069072591774772277, u'dismay': 0.069072591774772277, u'sip': 0.069072591774772277, u'cross-cultur': 6.9003588186585703e-05, u'non-invit': 6.9003588186585703e-05, u'int': 0.13807617996135799, u'border': 0.069072591774772277, u'minimum': 6.9003588186585703e-05, u'coffe': 0.069072591774772277, u'across': 0.069072591774772277, u'spolski': 0.069072591774772277, u'excus': 6.9003588186585703e-05, u'free': 0.069072591774772277, u'famou': 6.9003588186585703e-05, u'ensur': 6.9003588186585703e-05, u'unicod': 6.9003588186585703e-05, u'local': 0.069072591774772277, u'orient': 0.069072591774772277, u'good/bett': 6.9003588186585703e-05, u'internation': 6.9003588186585703e-05}, 'integer': {u'claim': 4.879953152449736e-05, u'style': 4.879953152449736e-05, u'medium': 0.14644739410501659, u'phpeclips': 4.879953152449736e-05, u'admin': 0.048848331056021856, u'webconfig': 0.097647862580519218, u'mitig': 0.048848331056021856, u'sleepless': 0.048848331056021856, u'breed': 4.879953152449736e-05, u'host': 0.048848331056021856, u'clue': 0.048848331056021856, u'connectionstr': 0.048848331056021856, u'certainli': 0.048848331056021856, u'breakpoint': 4.879953152449736e-05, u'night': 0.048848331056021856, u'good/bett': 4.879953152449736e-05, u'fresh': 0.048848331056021856, u'grow': 4.879953152449736e-05, u'trust': 0.14644739410501659, u'decent': 0.048848331056021856}, 'xss': {u'msdn-style': 6.9003588186585703e-05, u'comparison': 0.069072591774772277, u'unicod': 6.9003588186585703e-05, u'excus': 6.9003588186585703e-05, u'eye': 6.9003588186585703e-05, u'advantag': 0.13807617996135799, u'java/eclips': 0.069072591774772277, u'xslt': 6.9003588186585703e-05, u'tip': 6.9003588186585703e-05, u'unfamiliar': 0.069072591774772277, u'de-facto': 0.069072591774772277, u'benefit': 0.069072591774772277, u'minimum': 6.9003588186585703e-05, u'patient': 6.9003588186585703e-05, u'succinct': 0.069072591774772277, u'output': 6.9003588186585703e-05, u'ensur': 6.9003588186585703e-05, u'disadvantag': 0.069072591774772277, u'compani': 0.069072591774772277, u'cv': 0.13807617996135799}, 'silverlight': {u'oper': 0.064613994319648843, u'imagin': 6.4549444874774072e-05, u'breed': 6.4549444874774072e-05, u'depend': 0.064613994319648843, u'sourcesaf': 6.4549444874774072e-05, u'claim': 6.4549444874774072e-05, u'internation': 6.4549444874774072e-05, u'famou': 6.4549444874774072e-05, u'properti': 0.12916343919442291, u'grow': 6.4549444874774072e-05, u'issu': 0.12916343919442291, u'minimum': 6.4549444874774072e-05, u'common': 0.064613994319648843, u'breakpoint': 6.4549444874774072e-05, u'good/bett': 6.4549444874774072e-05, u'cross-cultur': 6.4549444874774072e-05, u'unicod': 6.4549444874774072e-05, u'ancestor': 0.064613994319648843, u'class': 0.32281177381874515, u'excus': 6.4549444874774072e-05}, 'sorting': {u'tester': 4.879953152449736e-05, u'claim': 4.879953152449736e-05, u'ultim': 0.048848331056021856, u'cours': 0.048848331056021856, u'practic': 0.097647862580519218, u'multi-develop': 0.048848331056021856, u'restor': 0.048848331056021856, u'directori': 0.14644739410501659, u'setup': 0.048848331056021856, u'dnn': 0.048848331056021856, u'exclud': 0.048848331056021856, u'collabor': 0.097647862580519218, u'breakpoint': 4.879953152449736e-05, u'trunk': 0.048848331056021856, u'good/bett': 4.879953152449736e-05, u'dotnetnuk': 0.048848331056021856, u'grow': 4.879953152449736e-05, u'phpeclips': 4.879953152449736e-05, u'checkout': 0.048848331056021856, u'easier': 0.048848331056021856}, 'windows-server-2003': {u'imagin': 8.0051232788984944e-05, u'given': 0.080131284021773927, u'simpli': 0.080131284021773927, u'newlin': 0.080131284021773927, u'sourcesaf': 8.0051232788984944e-05, u'claim': 8.0051232788984944e-05, u'internation': 8.0051232788984944e-05, u'famou': 8.0051232788984944e-05, u'hyphen': 0.080131284021773927, u'breed': 8.0051232788984944e-05, u'stay': 0.080131284021773927, u'dash': 0.080131284021773927, u'good/bett': 8.0051232788984944e-05, u'cross-cultur': 8.0051232788984944e-05, u'breakpoint': 8.0051232788984944e-05, u'wrap': 0.080131284021773927, u'word-break': 0.080131284021773927, u'constrain': 0.080131284021773927, u'phpeclips': 8.0051232788984944e-05, u'simpl': 0.080131284021773927}, 'xsd': {u'claim': 9.5310712924132673e-05, u'assumpt': 0.095406023637056789, u'set': 9.5310712924132673e-05, u'cross-cultur': 9.5310712924132673e-05, u'nearli': 0.095406023637056789, u'famou': 9.5310712924132673e-05, u'breed': 9.5310712924132673e-05, u'minimum': 9.5310712924132673e-05, u'integ': 0.095406023637056789, u'unfortun': 0.095406023637056789, u'bang': 0.095406023637056789, u'valid': 0.095406023637056789, u'arent': 0.095406023637056789, u'breakpoint': 9.5310712924132673e-05, u'good/bett': 9.5310712924132673e-05, u'unicod': 9.5310712924132673e-05, u'phpeclips': 9.5310712924132673e-05, u'bcp': 0.095406023637056789, u'grow': 9.5310712924132673e-05, u'excus': 9.5310712924132673e-05}, 'memory-leaks': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'windows-server-2008': {u'imagin': 0.00015403573629081948, u'set': 0.00015403573629081948, u'cpu-intens': 0.15418977202711026, u'aw': 0.00015403573629081948, u'sourcesaf': 0.00015403573629081948, u'tester': 0.00015403573629081948, u'internation': 0.00015403573629081948, u'famou': 0.00015403573629081948, u'breed': 0.00015403573629081948, u'grow': 0.00015403573629081948, u'claim': 0.00015403573629081948, u'consid': 0.15418977202711026, u'baggag': 0.00015403573629081948, u'good/bett': 0.00015403573629081948, u'cross-cultur': 0.00015403573629081948, u'phpeclips': 0.00015403573629081948, u'open-end': 0.15418977202711026, u'parallel': 0.15418977202711026, u'simpler': 0.00015403573629081948, u'breakpoint': 0.00015403573629081948}, 'sitemap': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'code-review': {u'"softwar': 0.035132668819317696, u'softwar': 0.10532781131545696, u'term': 0.035132668819317696, u'"defined"': 0.035132668819317696, u'engin': 0.10532781131545696, u'often': 0.070230240067387331, u'how': 0.035132668819317696, u'disciplin': 0.035132668819317696, u'profession': 0.035132668819317696, u'young': 0.035132668819317696, u'least': 0.035132668819317696, u'arguabl': 0.035132668819317696, u'understood': 0.035132668819317696, u'matur': 0.035132668819317696, u'definit': 0.035132668819317696, u'field': 0.035132668819317696, u'fulli': 0.035132668819317696, u'still': 0.035132668819317696, u'ieee': 0.035132668819317696, u'popul': 0.035132668819317696}, 'fonts': {u'imagin': 6.9003588186585703e-05, u'carbonemac': 0.069072591774772277, u'cross-cultur': 6.9003588186585703e-05, u'sourcesaf': 6.9003588186585703e-05, u'fair': 0.069072591774772277, u'claim': 6.9003588186585703e-05, u'internation': 6.9003588186585703e-05, u'breed': 6.9003588186585703e-05, u'tool': 0.069072591774772277, u'daddi': 0.069072591774772277, u'famou': 6.9003588186585703e-05, u'grow': 6.9003588186585703e-05, u'minimum': 6.9003588186585703e-05, u'mac': 0.27608335633452941, u'set': 6.9003588186585703e-05, u'breakpoint': 6.9003588186585703e-05, u'good/bett': 6.9003588186585703e-05, u'phpeclips': 6.9003588186585703e-05, u'os': 0.20707976814794368, u'worth': 0.069072591774772277}, 'macromedia': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'literals': {u'claim': 4.4460252534234394e-05, u'liter': 0.22234572292370622, u'cross-cultur': 4.4460252534234394e-05, u'sourcesaf': 4.4460252534234394e-05, u'imagin': 4.4460252534234394e-05, u'python': 0.35572648052640937, u'breed': 4.4460252534234394e-05, u'earlier': 0.088964965321003009, u'famou': 4.4460252534234394e-05, u'integ': 0.044504712786768624, u'easili': 0.044504712786768624, u'second': 0.044504712786768624, u'minimum': 4.4460252534234394e-05, u'allow': 0.088964965321003009, u'breakpoint': 4.4460252534234394e-05, u'good/bett': 4.4460252534234394e-05, u'unicod': 4.4460252534234394e-05, u'phpeclips': 4.4460252534234394e-05, u'grow': 4.4460252534234394e-05, u'internation': 4.4460252534234394e-05}, 'warnings': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'proc': {u'claim': 7.4117995849392229e-05, u'differ': 0.22242810554402606, u'set': 7.4117995849392229e-05, u'cross-cultur': 7.4117995849392229e-05, u'deal': 0.074192113845241617, u'excus': 7.4117995849392229e-05, u'internation': 7.4117995849392229e-05, u'guidelin': 0.074192113845241617, u'breed': 7.4117995849392229e-05, u'famou': 7.4117995849392229e-05, u'proc/lambda': 0.074192113845241617, u'minimum': 7.4117995849392229e-05, u'procnew': 0.074192113845241617, u'breakpoint': 7.4117995849392229e-05, u'good/bett': 7.4117995849392229e-05, u'unicod': 7.4117995849392229e-05, u'subtl': 0.074192113845241617, u'proc': 0.074192113845241617, u'grow': 7.4117995849392229e-05, u'lambda': 0.14831010969463385}, 'internet-explorer-7': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'address-bar': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'bpgsql': {u'psql': 0.074192113845241617, u'insert': 0.074192113845241617, u'unicod': 7.4117995849392229e-05, u'liter': 0.074192113845241617, u'postgresql': 0.074192113845241617, u'cross-cultur': 7.4117995849392229e-05, u'sourcesaf': 7.4117995849392229e-05, u'imagin': 7.4117995849392229e-05, u'internation': 7.4117995849392229e-05, u'famou': 7.4117995849392229e-05, u'breed': 7.4117995849392229e-05, u'charact': 0.14831010969463385, u'warn': 0.14831010969463385, u'minimum': 7.4117995849392229e-05, u'breakpoint': 7.4117995849392229e-05, u'claim': 7.4117995849392229e-05, u'good/bett': 7.4117995849392229e-05, u'escap': 0.14831010969463385, u'grow': 7.4117995849392229e-05, u'produc': 0.074192113845241617}, 'data-conversion': {u'head': 0.078495214184842302, u'long': 0.03926722108896908, u'string': 0.11772320728071552, u'lib': 0.03926722108896908, u'row': 0.078495214184842302, u'breed': 3.9227993095873212e-05, u'phpeclips': 3.9227993095873212e-05, u'float': 0.03926722108896908, u'anyway': 0.03926722108896908, u'fast': 0.03926722108896908, u'turn': 0.03926722108896908, u'obvious': 0.03926722108896908, u'initi': 0.03926722108896908, u'key': 0.03926722108896908, u'breakpoint': 3.9227993095873212e-05, u'good/bett': 3.9227993095873212e-05, u'export': 0.03926722108896908, u'csv': 0.15695120037658875, u'delimit': 0.03926722108896908, u'view': 0.03926722108896908}, 'security': {u'cross-cultur': 8.0051232788984944e-05, u'map': 0.080131284021773927, u'tester': 8.0051232788984944e-05, u'breed': 8.0051232788984944e-05, u'hash': 0.080131284021773927, u'key': 0.16018251681075887, u'sourcesaf': 8.0051232788984944e-05, u'&': 0.080131284021773927, u'internation': 8.0051232788984944e-05, u'baggag': 8.0051232788984944e-05, u'claim': 8.0051232788984944e-05, u'imagin': 8.0051232788984944e-05, u'suppos': 8.0051232788984944e-05, u'dictionari': 0.16018251681075887, u'output': 8.0051232788984944e-05, u'good/bett': 8.0051232788984944e-05, u'word': 0.16018251681075887, u'phpeclips': 8.0051232788984944e-05, u'grow': 8.0051232788984944e-05, u'sorteddictionari': 0.080131284021773927}, 'postgresql': {u'annoy': 0.046575469942304104, u'yyyi': 0.046575469942304104, u'breed': 4.6528941001302811e-05, u'claim': 4.6528941001302811e-05, u'dd/m/yyi': 0.046575469942304104, u'especi': 0.046575469942304104, u'imagin': 4.6528941001302811e-05, u'hint': 0.046575469942304104, u'format': 0.23269123394751537, u'breakpoint': 4.6528941001302811e-05, u'yy': 0.046575469942304104, u'good/bett': 4.6528941001302811e-05, u'localis': 0.13963335194490972, u'letter': 0.046575469942304104, u'enter': 0.046575469942304104, u'distinguish': 0.046575469942304104, u'sens': 0.046575469942304104, u'phpeclips': 4.6528941001302811e-05, u'confus': 0.046575469942304104, u'grow': 4.6528941001302811e-05}, 'vb6-migration': {u'sell': 0.022498426683448709, u'absolut': 0.022498426683448709, u'easy-to-manag': 0.022498426683448709, u'asp': 0.022498426683448709, u'classic': 0.022498426683448709, u'fundament': 0.022498426683448709, u'where': 0.022498426683448709, u'needless': 0.022498426683448709, u'none': 0.022498426683448709, u're-design': 0.022498426683448709, u'avail': 0.067450328148880703, u'suit': 0.022498426683448709, u'visual': 0.17983008181246066, u'heck': 0.022498426683448709, u'dealership': 0.022498426683448709, u'basic': 0.1348781803470287, u'posit': 0.022498426683448709, u'ado': 0.022498426683448709, u'dial-up': 0.022498426683448709, u'vbnet': 0.044974377416164703}, 'spss': {u'imagin': 0.00011775788977861516, u'tester': 0.00011775788977861516, u'set': 0.00011775788977861516, u'cross-cultur': 0.00011775788977861516, u'sourcesaf': 0.00011775788977861516, u'claim': 0.00011775788977861516, u'internation': 0.00011775788977861516, u'f-test': 0.11787564766839376, u'breed': 0.00011775788977861516, u'famou': 0.00011775788977861516, u'spss': 0.11787564766839376, u'minimum': 0.00011775788977861516, u'statist': 0.35339142722562406, u'aw': 0.00011775788977861516, u'breakpoint': 0.00011775788977861516, u'good/bett': 0.00011775788977861516, u'phpeclips': 0.00011775788977861516, u'baggag': 0.00011775788977861516, u'grow': 0.00011775788977861516, u'facilit': 0.11787564766839376}, 'branching-and-merging': {u'claim': 0.00013347570742124932, u'good': 0.26708489054991991, u'cross-cultur': 0.00013347570742124932, u'specif': 0.13360918312867057, u'imagin': 0.00013347570742124932, u'internation': 0.00013347570742124932, u'famou': 0.00013347570742124932, u'explain': 0.13360918312867057, u'breed': 0.00013347570742124932, u'apach': 0.13360918312867057, u'set': 0.00013347570742124932, u'tip': 0.00013347570742124932, u'ensur': 0.00013347570742124932, u'minimum': 0.00013347570742124932, u'breakpoint': 0.00013347570742124932, u'good/bett': 0.00013347570742124932, u'unicod': 0.00013347570742124932, u'phpeclips': 0.00013347570742124932, u'grow': 0.00013347570742124932, u'excus': 0.00013347570742124932}, 'powerpoint': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'curses': {u'imagin': 0.00011775788977861516, u'tester': 0.00011775788977861516, u'right': 0.11787564766839376, u'cross-cultur': 0.00011775788977861516, u'sourcesaf': 0.00011775788977861516, u'claim': 0.00011775788977861516, u'internation': 0.00011775788977861516, u'famou': 0.00011775788977861516, u'breed': 0.00011775788977861516, u'minimum': 0.00011775788977861516, u'grow': 0.00011775788977861516, u'set': 0.00011775788977861516, u'file': 0.35339142722562406, u'breakpoint': 0.00011775788977861516, u'good/bett': 0.00011775788977861516, u'unicod': 0.00011775788977861516, u'phpeclips': 0.00011775788977861516, u'baggag': 0.00011775788977861516, u'check': 0.23563353744700891, u'excus': 0.00011775788977861516}, 'active-directory': {u'server-sid': 0.029887734384330584, u'differenti': 0.029887734384330584, u'throttl': 0.11946136390779889, u'non-techn': 0.029887734384330584, u'/aspnet': 0.029887734384330584, u'ip': 0.089603487399976117, u'henc': 0.029887734384330584, u'anyway': 0.029887734384330584, u'air': 0.029887734384330584, u'ii': 0.17917711692344443, u'matter': 0.029887734384330584, u'rang': 0.029887734384330584, u'connect': 0.029887734384330584, u'background': 0.029887734384330584, u'manner': 0.029887734384330584, u'rig': 0.029887734384330584, u'kbp': 0.029887734384330584, u'vpn': 0.059745610892153349, u'across': 0.029887734384330584, u'bandwith': 0.029887734384330584}, 'image': {u'box': 0.20418912297893191, u'claim': 4.0829658664053568e-05, u'c': 0.040870488322717613, u'breed': 4.0829658664053568e-05, u'desir': 0.040870488322717613, u'combobox': 0.040870488322717613, u'sourcesaf': 4.0829658664053568e-05, u'imagin': 4.0829658664053568e-05, u'bind': 0.040870488322717613, u'develop': 0.040870488322717613, u'bound': 0.040870488322717613, u'dabbl': 0.040870488322717613, u'achiev': 0.040870488322717613, u'set': 4.0829658664053568e-05, u'breakpoint': 4.0829658664053568e-05, u'combo': 0.20418912297893191, u'follow': 0.08170014698677118, u'grow': 4.0829658664053568e-05, u'select': 0.040870488322717613, u'funcion': 0.040870488322717613}, 'parsing': {u'claim': 5.7168991538989253e-05, u'breed': 5.7168991538989253e-05, u'cross-cultur': 5.7168991538989253e-05, u'specif': 0.11439515206951748, u'hyperlink': 0.057226160530528231, u'goe': 0.057226160530528231, u'famou': 5.7168991538989253e-05, u'straight': 0.057226160530528231, u'breakpoint': 5.7168991538989253e-05, u'click': 0.057226160530528231, u'nth': 0.057226160530528231, u'slide': 0.17156414360850672, u'link': 0.11439515206951748, u'ppt': 0.057226160530528231, u'good/bett': 5.7168991538989253e-05, u'onlin': 0.057226160530528231, u'patient': 5.7168991538989253e-05, u'phpeclips': 5.7168991538989253e-05, u'powerpoint': 0.057226160530528231, u'internation': 5.7168991538989253e-05}, 'git': {u'mailto': 0.064613994319648843, u'excus': 6.4549444874774072e-05, u'protocol': 0.064613994319648843, u'unicod': 6.4549444874774072e-05, u'internation': 6.4549444874774072e-05, u'famou': 6.4549444874774072e-05, u'delphi': 0.064613994319648843, u'independ': 0.064613994319648843, u'minimum': 6.4549444874774072e-05, u'click': 0.064613994319648843, u'ensur': 6.4549444874774072e-05, u'program': 0.19371288406919698, u'patient': 6.4549444874774072e-05, u'button': 0.064613994319648843, u'cross-cultur': 6.4549444874774072e-05, u'quick': 0.064613994319648843, u'sandcastl': 6.4549444874774072e-05, u'client"': 0.064613994319648843, u'email': 0.064613994319648843, u'"set': 0.064613994319648843}, 'gis': {u'map': 0.44916707496325325, u'excus': 4.0829658664053568e-05, u'cross-cultur': 4.0829658664053568e-05, u'plot': 0.040870488322717613, u'internation': 4.0829658664053568e-05, u'famou': 4.0829658664053568e-05, u'globe--flat-earth': 0.040870488322717613, u'breed': 4.0829658664053568e-05, u'latitude/longitud': 0.12252980565082475, u'grow': 4.0829658664053568e-05, u'arbitrarili': 0.040870488322717613, u'good/bett': 4.0829658664053568e-05, u'minimum': 4.0829658664053568e-05, u'equat': 0.040870488322717613, u'breakpoint': 4.0829658664053568e-05, u'pair': 0.08170014698677118, u'affect': 0.040870488322717613, u'claim': 4.0829658664053568e-05, u'phpeclips': 4.0829658664053568e-05, u'think': 0.040870488322717613}, 'compression': {u'thrown': 0.030807583405145882, u'unicod': 3.0776806598547338e-05, u'guess': 0.030807583405145882, u'dont': 0.030807583405145882, u'consist': 0.030807583405145882, u'auto': 0.030807583405145882, u'setup': 0.061584390003693215, u'sql': 0.030807583405145882, u'remov': 0.030807583405145882, u'contract': 0.061584390003693215, u'dataset': 0.030807583405145882, u'broken': 0.030807583405145882, u'ignor': 0.030807583405145882, u'detects/cr': 0.030807583405145882, u'xsd': 0.15391480979933525, u'tabl': 0.15391480979933525, u'typic': 0.030807583405145882, u'fill': 0.030807583405145882, u'fix': 0.030807583405145882, u'schema': 0.061584390003693215}, 'xml-comments': {u'xml': 0.13807617996135799, u'claim': 6.9003588186585703e-05, u'made': 0.13807617996135799, u'patient': 0.069072591774772277, u'internation': 6.9003588186585703e-05, u'breed': 6.9003588186585703e-05, u'minimum': 6.9003588186585703e-05, u'cross': 6.9003588186585703e-05, u'realli': 0.27608335633452941, u'ensur': 6.9003588186585703e-05, u'much': 0.069072591774772277, u'aw': 0.069072591774772277, u'breakpoint': 6.9003588186585703e-05, u'good/bett': 6.9003588186585703e-05, u'cross-cultur': 6.9003588186585703e-05, u'unicod': 6.9003588186585703e-05, u'phpeclips': 6.9003588186585703e-05, u'glyph': 6.9003588186585703e-05, u'non-invit': 0.069072591774772277, u'excus': 6.9003588186585703e-05}, 'opengl': {u'imagin': 6.0635459616783893e-05, u'tester': 6.0635459616783893e-05, u'breed': 6.0635459616783893e-05, u'sourcesaf': 6.0635459616783893e-05, u'e\u03c0i': 0.060696095076400669, u'mathematician': 0.060696095076400669, u'+': 0.060696095076400669, u'float': 0.12133155469318456, u'method': 0.060696095076400669, u'keen': 0.060696095076400669, u'zero': 0.060696095076400669, u'settl': 0.060696095076400669, u'breakpoint': 6.0635459616783893e-05, u'imprecis': 0.060696095076400669, u'disput': 0.060696095076400669, u'grow': 6.0635459616783893e-05, u'agre': 0.060696095076400669, u'=': 0.060696095076400669, u'divers': 0.060696095076400669, u'good/bett': 6.0635459616783893e-05}, 'hex': {u'c': 0.025346905702420743, u'balk': 0.025346905702420743, u'c#/vbnet': 0.025346905702420743, u'decla': 0.025346905702420743, u'array': 0.025346905702420743, u'littl': 0.050668489820723188, u'sinc': 0.075990073939025626, u'binari': 0.025346905702420743, u'delug': 0.025346905702420743, u'without': 0.050668489820723188, u'pars': 0.075990073939025626, u'asprox': 0.025346905702420743, u'debugg': 0.025346905702420743, u'sql': 0.12663324217563052, u'pair': 0.050668489820723188, u'byte': 0.10131165805732809, u'luck': 0.025346905702420743, u'ascii': 0.025346905702420743, u'anybodi': 0.050668489820723188, u'touch': 0.025346905702420743}, 'emacs': {u'claim': 5.7168991538989253e-05, u'graphic': 0.057226160530528231, u'cross-cultur': 5.7168991538989253e-05, u'grow': 5.7168991538989253e-05, u'imagin': 5.7168991538989253e-05, u'internation': 5.7168991538989253e-05, u'famou': 5.7168991538989253e-05, u'breed': 5.7168991538989253e-05, u'minimum': 5.7168991538989253e-05, u'us': 0.057226160530528231, u'vim': 0.28590212668648524, u'version': 0.22873313514749602, u'editor': 0.057226160530528231, u'breakpoint': 5.7168991538989253e-05, u'prime-tim': 0.057226160530528231, u'unicod': 5.7168991538989253e-05, u'phpeclips': 5.7168991538989253e-05, u'discuss': 0.057226160530528231, u'aquamac': 0.057226160530528231, u'excus': 5.7168991538989253e-05}, 'editor': {u'comparison': 0.069072591774772277, u'trade-off': 0.069072591774772277, u'tough': 0.069072591774772277, u'cross-cultur': 6.9003588186585703e-05, u'sourcesaf': 6.9003588186585703e-05, u'tester': 6.9003588186585703e-05, u'imagin': 6.9003588186585703e-05, u'breed': 6.9003588186585703e-05, u'phpeclips': 6.9003588186585703e-05, u'baggag': 6.9003588186585703e-05, u'famou': 6.9003588186585703e-05, u'vim-cocoa': 0.069072591774772277, u'claim': 6.9003588186585703e-05, u'internation': 6.9003588186585703e-05, u'emac': 0.27608335633452941, u'breakpoint': 6.9003588186585703e-05, u'macvim': 0.069072591774772277, u'good/bett': 6.9003588186585703e-05, u'x': 0.13807617996135799, u'xemac': 0.069072591774772277}, 'linq': {u'imagin': 0.00013347570742124932, u'differ': 0.26708489054991991, u'set': 0.00013347570742124932, u'cross-cultur': 0.00013347570742124932, u'unicod': 0.00013347570742124932, u'sourcesaf': 0.00013347570742124932, u'claim': 0.00013347570742124932, u'internation': 0.00013347570742124932, u'famou': 0.00013347570742124932, u'breed': 0.00013347570742124932, u'mathtrunc': 0.13360918312867057, u'tester': 0.00013347570742124932, u'minimum': 0.00013347570742124932, u'breakpoint': 0.00013347570742124932, u'good/bett': 0.00013347570742124932, u'mathfloor': 0.13360918312867057, u'net': 0.13360918312867057, u'phpeclips': 0.00013347570742124932, u'grow': 0.00013347570742124932, u'excus': 0.00013347570742124932}, 'data-storage': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'com-interop': {u'leak"': 0.069072591774772277, u'applic': 0.20707976814794368, u'and/or': 0.069072591774772277, u'cross-cultur': 6.9003588186585703e-05, u'phpeclips': 6.9003588186585703e-05, u'prevent': 0.069072591774772277, u'interop': 0.069072591774772277, u'breed': 6.9003588186585703e-05, u'minimum': 6.9003588186585703e-05, u'ensur': 6.9003588186585703e-05, u'kill': 0.069072591774772277, u'internation': 6.9003588186585703e-05, u'via': 0.069072591774772277, u'unmanag': 0.069072591774772277, u'breakpoint': 6.9003588186585703e-05, u'good/bett': 6.9003588186585703e-05, u'affect': 0.069072591774772277, u'unicod': 6.9003588186585703e-05, u'exit': 0.069072591774772277, u'excus': 6.9003588186585703e-05}, 'podcast': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'arrays': {u'oledb': 0.022003868812098829, u'execut': 0.065967642662446146, u'script': 0.043985755737272487, u'track': 0.043985755737272487, u'welcom': 0.022003868812098829, u'hundr': 0.022003868812098829, u'rinse/repeat': 0.022003868812098829, u'upgrad': 0.10993141651279348, u'auto-upd': 0.022003868812098829, u'autom': 0.043985755737272487, u'direct': 0.022003868812098829, u'packag': 0.043985755737272487, u'second': 0.043985755737272487, u'mechan': 0.022003868812098829, u'scratch': 0.022003868812098829, u'otherwis': 0.022003868812098829, u'went': 0.022003868812098829, u'amaz': 0.022003868812098829, u'dt': 0.022003868812098829, u'schema': 0.043985755737272487}, 'windows': {u'anatomi': 0.046575469942304104, u'p/invok': 0.046575469942304104, u'excus': 4.6528941001302811e-05, u'cross-cultur': 4.6528941001302811e-05, u'minimum': 4.6528941001302811e-05, u'memori': 0.18616229294621256, u'process': 0.13963335194490972, u'leak': 0.23269123394751537, u'away': 0.046575469942304104, u'tip': 4.6528941001302811e-05, u'"memori': 0.046575469942304104, u'ensur': 4.6528941001302811e-05, u'internation': 4.6528941001302811e-05, u'incomplet': 0.046575469942304104, u'paragraph': 4.6528941001302811e-05, u'com': 0.046575469942304104, u'famou': 4.6528941001302811e-05, u'good/bett': 4.6528941001302811e-05, u'unicod': 4.6528941001302811e-05, u'perspect': 0.046575469942304104}, 'embed': {u'assembl': 0.02902122231242027, u'explan': 0.02902122231242027, u'earlier': 0.02902122231242027, u'cross-cultur': 2.8992230082337937e-05, u'enabl': 0.02902122231242027, u'bottom': 0.02902122231242027, u'short': 0.02902122231242027, u'famou': 2.8992230082337937e-05, u'site/appl': 0.02902122231242027, u'*': 0.02902122231242027, u'number': 0.14499014264177201, u'upgrad': 0.02902122231242027, u'sync': 0.02902122231242027, u'version': 0.23196683288878581, u'reset': 0.02902122231242027, u'increment': 0.058013452394758205, u'wpd': 0.02902122231242027, u'revis': 0.087005682477096141, u'assemblyvers': 0.02902122231242027, u'log': 0.02902122231242027}, 'int': {u'claim': 6.9003588186585703e-05, u'excus': 6.9003588186585703e-05, u'cross-cultur': 6.9003588186585703e-05, u'sit': 0.069072591774772277, u'internation': 6.9003588186585703e-05, u'famou': 6.9003588186585703e-05, u'joel': 0.13807617996135799, u'breed': 6.9003588186585703e-05, u'minimum': 6.9003588186585703e-05, u'integ': 0.13807617996135799, u'brain': 0.069072591774772277, u'say': 0.069072591774772277, u'realiz': 0.069072591774772277, u'ensur': 6.9003588186585703e-05, u'know': 0.13807617996135799, u'breakpoint': 6.9003588186585703e-05, u'quick': 0.069072591774772277, u'unicod': 6.9003588186585703e-05, u'phpeclips': 6.9003588186585703e-05, u'came': 0.069072591774772277}, 'youtube': {u'count': 0.080131284021773927, u'redirect': 0.080131284021773927, u'excus': 8.0051232788984944e-05, u'mess': 0.080131284021773927, u'claim': 8.0051232788984944e-05, u'front': 0.080131284021773927, u'famou': 8.0051232788984944e-05, u'programat': 0.080131284021773927, u'breed': 8.0051232788984944e-05, u'ensur': 8.0051232788984944e-05, u'player': 0.16018251681075887, u'minimum': 8.0051232788984944e-05, u'breakpoint': 8.0051232788984944e-05, u'download': 0.16018251681075887, u'cross-cultur': 8.0051232788984944e-05, u'unicod': 8.0051232788984944e-05, u'phpeclips': 8.0051232788984944e-05, u'overlaid': 0.080131284021773927, u'grow': 8.0051232788984944e-05, u'good/bett': 8.0051232788984944e-05}, 'office-2007': {u'imagin': 6.4549444874774072e-05, u'tester': 6.4549444874774072e-05, u'breed': 6.4549444874774072e-05, u'multipl': 0.064613994319648843, u'sourcesaf': 6.4549444874774072e-05, u'claim': 6.4549444874774072e-05, u'internation': 6.4549444874774072e-05, u'famou': 6.4549444874774072e-05, u'markup': 0.064613994319648843, u'previou': 0.064613994319648843, u'back': 0.12916343919442291, u'grow': 6.4549444874774072e-05, u'minimum': 6.4549444874774072e-05, u'breakpoint': 6.4549444874774072e-05, u'enter': 0.19371288406919698, u'cross-cultur': 6.4549444874774072e-05, u'unicod': 6.4549444874774072e-05, u'phpeclips': 6.4549444874774072e-05, u'button': 0.32281177381874515, u'excus': 6.4549444874774072e-05}, 'full-text-search': {u'imagin': 6.9003588186585703e-05, u'#': 0.13807617996135799, u'set': 6.9003588186585703e-05, u'return': 0.13807617996135799, u'sourcesaf': 6.9003588186585703e-05, u'freetext': 0.069072591774772277, u'internation': 6.9003588186585703e-05, u'queri': 0.13807617996135799, u'searchfield': 0.069072591774772277, u'famou': 6.9003588186585703e-05, u'breakpoint': 6.9003588186585703e-05, u'claim': 6.9003588186585703e-05, u'minimum': 6.9003588186585703e-05, u'instanc': 0.069072591774772277, u'letter': 0.069072591774772277, u'text': 0.13807617996135799, u'good/bett': 6.9003588186585703e-05, u'cross-cultur': 6.9003588186585703e-05, u'phpeclips': 6.9003588186585703e-05, u'grow': 6.9003588186585703e-05}, 'file': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'timezone': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'sql-server': {u'tri': 0.11787564766839376, u'claim': 0.00011775788977861516, u'tester': 0.00011775788977861516, u'breed': 0.00011775788977861516, u'cross-cultur': 0.00011775788977861516, u'sourcesaf': 0.00011775788977861516, u'memori': 0.11787564766839376, u'forc': 0.11787564766839376, u'famou': 0.00011775788977861516, u'unload': 0.23563353744700891, u'minimum': 0.00011775788977861516, u'grow': 0.00011775788977861516, u'internation': 0.00011775788977861516, u'imagin': 0.00011775788977861516, u'without': 0.11787564766839376, u'breakpoint': 0.00011775788977861516, u'good/bett': 0.00011775788977861516, u'unicod': 0.00011775788977861516, u'phpeclips': 0.00011775788977861516, u'baggag': 0.00011775788977861516}, 'subsonic': {u'claim': 0.00040128410914927769, u'set': 0.00040128410914927769, u'cross-cultur': 0.00040128410914927769, u'sourcesaf': 0.00040128410914927769, u'imagin': 0.00040128410914927769, u'internation': 0.00040128410914927769, u'breed': 0.00040128410914927769, u'tester': 0.00040128410914927769, u'famou': 0.00040128410914927769, u'non-invit': 0.00040128410914927769, u'ensur': 0.00040128410914927769, u'minimum': 0.00040128410914927769, u'aw': 0.00040128410914927769, u'breakpoint': 0.00040128410914927769, u'good/bett': 0.00040128410914927769, u'unicod': 0.00040128410914927769, u'phpeclips': 0.00040128410914927769, u'baggag': 0.00040128410914927769, u'grow': 0.00040128410914927769, u'excus': 0.00040128410914927769}, 'lookup': {u'imagin': 0.00013347570742124932, u'accord': 0.13360918312867057, u'set': 0.00013347570742124932, u'cross-cultur': 0.00013347570742124932, u'sourcesaf': 0.00013347570742124932, u'tester': 0.00013347570742124932, u'internation': 0.00013347570742124932, u'ip': 0.26708489054991991, u'breed': 0.00013347570742124932, u'famou': 0.00013347570742124932, u'claim': 0.00013347570742124932, u'minimum': 0.00013347570742124932, u'aw': 0.00013347570742124932, u'breakpoint': 0.00013347570742124932, u'good/bett': 0.00013347570742124932, u'countri': 0.26708489054991991, u'phpeclips': 0.00013347570742124932, u'baggag': 0.00013347570742124932, u'grow': 0.00013347570742124932, u'simpler': 0.00013347570742124932}, 'branch': {u'claim': 0.00011775788977861516, u'excus': 0.00011775788977861516, u'set': 0.00011775788977861516, u'cross-cultur': 0.00011775788977861516, u'sourcesaf': 0.00011775788977861516, u'imagin': 0.00011775788977861516, u'internation': 0.00011775788977861516, u'famou': 0.00011775788977861516, u'breed': 0.00011775788977861516, u'minimum': 0.00011775788977861516, u'tip': 0.00011775788977861516, u'good/bett': 0.00011775788977861516, u'tutori': 0.23563353744700891, u'branch': 0.23563353744700891, u'breakpoint': 0.00011775788977861516, u'ensur': 0.00011775788977861516, u'unicod': 0.00011775788977861516, u'phpeclips': 0.00011775788977861516, u'grow': 0.00011775788977861516, u'merg': 0.23563353744700891}, 'unit-testing': {u'index': 0.17031329814404905, u'protect': 0.042610250297973773, u'excus': 4.2567682615358418e-05, u'resourc': 0.042610250297973773, u'unicod': 4.2567682615358418e-05, u'form': 0.1277456155286906, u'openid': 0.042610250297973773, u'databas': 0.085177932913332186, u'agnost': 0.042610250297973773, u'breed': 4.2567682615358418e-05, u'http//stackoverflowcom/questions//how-do-i-index-a-database-field': 0.042610250297973773, u'submit': 0.042610250297973773, u'dataset': 0.042610250297973773, u'claim': 4.2567682615358418e-05, u'role': 0.042610250297973773, u'queri': 0.042610250297973773, u'increas': 0.042610250297973773, u'grow': 4.2567682615358418e-05, u'guid': 0.042610250297973773, u'explain': 0.042610250297973773}, 'type-conversion': {u'load': 0.057226160530528231, u'confound': 0.057226160530528231, u'set': 5.7168991538989253e-05, u'cross-cultur': 5.7168991538989253e-05, u'd': 0.057226160530528231, u'claim': 5.7168991538989253e-05, u'internation': 5.7168991538989253e-05, u'famou': 5.7168991538989253e-05, u'gotcha': 0.057226160530528231, u'breed': 5.7168991538989253e-05, u'content': 0.22873313514749602, u'minimum': 5.7168991538989253e-05, u'breakpoint': 5.7168991538989253e-05, u'good/bett': 5.7168991538989253e-05, u'earth': 0.057226160530528231, u'array': 0.11439515206951748, u'phpeclips': 5.7168991538989253e-05, u'assum': 0.057226160530528231, u'grow': 5.7168991538989253e-05, u'winform': 0.17156414360850672}, 'opacity': {u'imagin': 0.00013347570742124932, u'set': 0.00013347570742124932, u'cross-cultur': 0.00013347570742124932, u'opac': 0.26708489054991991, u'sourcesaf': 0.00013347570742124932, u'claim': 0.00013347570742124932, u'internation': 0.00013347570742124932, u'breed': 0.00013347570742124932, u'decim': 0.13360918312867057, u'famou': 0.00013347570742124932, u'ensur': 0.00013347570742124932, u'minimum': 0.00013347570742124932, u'unicod': 0.00013347570742124932, u'breakpoint': 0.00013347570742124932, u'good/bett': 0.00013347570742124932, u'implicitli': 0.13360918312867057, u'phpeclips': 0.00013347570742124932, u'vbnet': 0.13360918312867057, u'grow': 0.00013347570742124932, u'excus': 0.00013347570742124932}, 'utilities': {u'unexpectedli': 0.048848331056021856, u'wont': 0.048848331056021856, u'"someon': 0.048848331056021856, u'breed': 4.879953152449736e-05, u'too"': 0.048848331056021856, u'sourcesaf': 4.879953152449736e-05, u'claim': 4.879953152449736e-05, u'imagin': 4.879953152449736e-05, u'sata-onli': 0.048848331056021856, u'files"': 0.048848331056021856, u'it"': 0.048848331056021856, u'sure"': 0.097647862580519218, u'set': 4.879953152449736e-05, u'file': 0.2928459886785087, u'breakpoint': 4.879953152449736e-05, u'"sure': 0.048848331056021856, u'leeri': 0.048848331056021856, u'phpeclips': 4.879953152449736e-05, u'grow': 4.879953152449736e-05, u'loop': 0.048848331056021856}, 'registry': {u'imagin': 0.00011775788977861516, u'tester': 0.00011775788977861516, u'asp': 0.35339142722562406, u'cross-cultur': 0.00011775788977861516, u'sourcesaf': 0.00011775788977861516, u'claim': 0.00011775788977861516, u'internation': 0.00011775788977861516, u'famou': 0.00011775788977861516, u'breed': 0.00011775788977861516, u'ensur': 0.00011775788977861516, u'server': 0.11787564766839376, u'download': 0.11787564766839376, u'minimum': 0.00011775788977861516, u'breakpoint': 0.00011775788977861516, u'oppos': 0.11787564766839376, u'unicod': 0.00011775788977861516, u'phpeclips': 0.00011775788977861516, u'grow': 0.00011775788977861516, u'good/bett': 0.00011775788977861516, u'excus': 0.00011775788977861516}, 'sql': {u'imagin': 4.879953152449736e-05, u'multipl': 0.097647862580519218, u'breakpoint': 4.879953152449736e-05, u'execut': 0.097647862580519218, u'claim': 4.879953152449736e-05, u'adob': 0.14644739410501659, u'queri': 0.19524692562951396, u'semicolon-': 0.048848331056021856, u'breed': 4.879953152449736e-05, u'famou': 4.879953152449736e-05, u'air': 0.14644739410501659, u'internation': 4.879953152449736e-05, u'minimum': 4.879953152449736e-05, u'sourcesaf': 4.879953152449736e-05, u'statement': 0.097647862580519218, u'sqlstatementexecut': 0.048848331056021856, u'cross-cultur': 4.879953152449736e-05, u'unicod': 4.879953152449736e-05, u'phpeclips': 4.879953152449736e-05, u'grow': 4.879953152449736e-05}, 'linq-to-sql': {u'reflect': 0.03926722108896908, u'tester': 3.9227993095873212e-05, u'viewdata': 0.03926722108896908, u'sourcesaf': 3.9227993095873212e-05, u'form': 0.15695120037658875, u'imagin': 3.9227993095873212e-05, u'url': 0.078495214184842302, u'request': 0.03926722108896908, u'breed': 3.9227993095873212e-05, u'situat': 0.078495214184842302, u'submit': 0.03926722108896908, u'redirecttoact': 0.078495214184842302, u'ed': 0.03926722108896908, u'valid': 0.078495214184842302, u'breakpoint': 3.9227993095873212e-05, u'lose': 0.03926722108896908, u'submiss': 0.078495214184842302, u'phpeclips': 3.9227993095873212e-05, u'grow': 3.9227993095873212e-05, u'paramet': 0.11772320728071552}, 'svn': {u'imagin': 6.4549444874774072e-05, u'breed': 6.4549444874774072e-05, u'excus': 6.4549444874774072e-05, u'unicod': 6.4549444874774072e-05, u'claim': 6.4549444874774072e-05, u'internation': 6.4549444874774072e-05, u'famou': 6.4549444874774072e-05, u'configur': 0.064613994319648843, u'minimum': 6.4549444874774072e-05, u'grow': 6.4549444874774072e-05, u'"programming"': 0.064613994319648843, u'necessarili': 0.064613994319648843, u'anyon': 0.064613994319648843, u'breakpoint': 6.4549444874774072e-05, u'good/bett': 6.4549444874774072e-05, u'cross-cultur': 6.4549444874774072e-05, u'mylyn': 0.12916343919442291, u'eclips': 0.19371288406919698, u'automag': 0.064613994319648843, u'updat': 0.19371288406919698}, 'algorithm': {u'load': 0.060696095076400669, u'imagin': 6.0635459616783893e-05, u'phpeclips': 6.0635459616783893e-05, u'effici': 0.18196701430996845, u'sourcesaf': 6.0635459616783893e-05, u'tester': 6.0635459616783893e-05, u'internation': 6.0635459616783893e-05, u'abstract': 0.060696095076400669, u'breed': 6.0635459616783893e-05, u'visit': 0.060696095076400669, u'structur': 0.12133155469318456, u'claim': 6.0635459616783893e-05, u'dict': 0.18196701430996845, u'breakpoint': 6.0635459616783893e-05, u'baggag': 6.0635459616783893e-05, u'good/bett': 6.0635459616783893e-05, u'overhead': 0.060696095076400669, u'flexibl': 0.060696095076400669, u'grow': 6.0635459616783893e-05, u'^': 0.060696095076400669}, 'virtualhost': {u'&': 0.032828282828282825, u'what': 0.032828282828282825, u'revers': 0.032828282828282825, u'iceberg': 3.2795487340941887e-05, u'consist': 0.032828282828282825, u'normal': 0.032828282828282825, u'color': 0.032828282828282825, u'frame': 0.032828282828282825, u'flash': 0.065623770169224716, u'tip': 3.2795487340941887e-05, u'annot': 0.16401023219205038, u'rectangl': 0.032828282828282825, u'somehow': 0.032828282828282825, u'vs': 3.2795487340941887e-05, u'video': 0.0984192575101666, u'hijack': 0.032828282828282825, u'youtub': 0.16401023219205038, u'ew': 0.032828282828282825, u'greasemonkey': 0.032828282828282825, u'plugin': 0.032828282828282825}, 'sftp': {u'claim': 9.5310712924132673e-05, u'set': 9.5310712924132673e-05, u'cross-cultur': 9.5310712924132673e-05, u'classic': 0.19071673656118945, u'internation': 9.5310712924132673e-05, u'famou': 9.5310712924132673e-05, u'breed': 9.5310712924132673e-05, u'purchas': 0.095406023637056789, u'ensur': 9.5310712924132673e-05, u'sftp': 0.28602744948532211, u'minimum': 9.5310712924132673e-05, u'imagin': 9.5310712924132673e-05, u'necessarili': 0.095406023637056789, u'breakpoint': 9.5310712924132673e-05, u'good/bett': 9.5310712924132673e-05, u'unicod': 9.5310712924132673e-05, u'phpeclips': 9.5310712924132673e-05, u'grow': 9.5310712924132673e-05, u'kick': 0.095406023637056789, u'excus': 9.5310712924132673e-05}, 'visual-c++': {u'excit': 0.064613994319648843, u'imagin': 6.4549444874774072e-05, u'claim': 6.4549444874774072e-05, u'breakpoint': 6.4549444874774072e-05, u'sourcesaf': 6.4549444874774072e-05, u'tester': 6.4549444874774072e-05, u'phpeclips': 6.4549444874774072e-05, u'delphi': 0.064613994319648843, u'breed': 6.4549444874774072e-05, u'situat': 0.064613994319648843, u'vaniti': 0.064613994319648843, u'resist': 0.064613994319648843, u'slide': 0.064613994319648843, u'train': 0.064613994319648843, u'aw': 6.4549444874774072e-05, u'baggag': 6.4549444874774072e-05, u'oppos': 0.064613994319648843, u'minut': 0.12916343919442291, u'strong': 0.064613994319648843, u'noth': 0.12916343919442291}, 'oracle': {u'excus': 5.4077438892494048e-05, u'silli': 0.054131516331386534, u'webdav': 0.16228639411637463, u'internation': 5.4077438892494048e-05, u'exchang': 0.10820895522388059, u'away': 0.054131516331386534, u'tip': 5.4077438892494048e-05, u'good/bett': 5.4077438892494048e-05, u'soon': 0.054131516331386534, u'send': 0.054131516331386534, u'convinc': 0.054131516331386534, u'ii': 0.054131516331386534, u'deprec': 0.054131516331386534, u'famou': 5.4077438892494048e-05, u'ensur': 5.4077438892494048e-05, u'cross-cultur': 5.4077438892494048e-05, u'unicod': 5.4077438892494048e-05, u'john': 0.054131516331386534, u'requir': 0.10820895522388059, u'justin': 0.054131516331386534}, 'file-type': {u'set': 5.7168991538989253e-05, u'identifi': 0.17156414360850672, u'form': 0.057226160530528231, u'internation': 5.7168991538989253e-05, u'famou': 5.7168991538989253e-05, u'tip': 5.7168991538989253e-05, u'upload': 0.11439515206951748, u'ensur': 5.7168991538989253e-05, u'charact': 0.17156414360850672, u'restrict': 0.057226160530528231, u'extens': 0.057226160530528231, u'minimum': 5.7168991538989253e-05, u'file': 0.11439515206951748, u'breakpoint': 5.7168991538989253e-05, u'good/bett': 5.7168991538989253e-05, u'cross-cultur': 5.7168991538989253e-05, u'unicod': 5.7168991538989253e-05, u'phpeclips': 5.7168991538989253e-05, u'find': 0.11439515206951748, u'excus': 5.7168991538989253e-05}, 'validation': {u'sound': 0.060696095076400669, u'sku': 0.060696095076400669, u'barebon': 0.060696095076400669, u'eye': 6.0635459616783893e-05, u'core"': 0.12133155469318456, u'clear': 0.060696095076400669, u'paragraph': 6.0635459616783893e-05, u'pare': 0.060696095076400669, u'reiter': 6.0635459616783893e-05, u'window': 0.060696095076400669, u'instanc': 0.060696095076400669, u'somebodi': 6.0635459616783893e-05, u'aw': 6.0635459616783893e-05, u'fulli': 0.060696095076400669, u'dedic': 0.060696095076400669, u'appeal': 0.060696095076400669, u'"server': 0.12133155469318456, u'non-invit': 6.0635459616783893e-05, u'dead-simpl': 6.0635459616783893e-05, u'simpler': 6.0635459616783893e-05}, 'bcp': {u'setter': 0.048848331056021856, u'protect': 0.048848331056021856, u'forgiv': 0.048848331056021856, u'excus': 4.879953152449736e-05, u'goe': 0.048848331056021856, u'tip': 4.879953152449736e-05, u'told': 0.048848331056021856, u'iceberg': 4.879953152449736e-05, u'getters/sett': 0.048848331056021856, u'privat': 0.048848331056021856, u'topic': 0.048848331056021856, u'variable/properti': 0.048848331056021856, u'taught': 0.097647862580519218, u'english': 4.879953152449736e-05, u'ensur': 4.879953152449736e-05, u'`timeout': 4.879953152449736e-05, u'"correct"': 0.048848331056021856, u'php': 0.097647862580519218, u'public': 0.14644739410501659, u'probabl': 0.048848331056021856}, 'cpu': {u'claim': 8.0051232788984944e-05, u'excus': 8.0051232788984944e-05, u'postgresql': 0.16018251681075887, u'unicod': 8.0051232788984944e-05, u'imagin': 8.0051232788984944e-05, u'internation': 8.0051232788984944e-05, u'famou': 8.0051232788984944e-05, u'breed': 8.0051232788984944e-05, u'minimum': 8.0051232788984944e-05, u'procedur': 0.16018251681075887, u'good/bett': 8.0051232788984944e-05, u'trigger': 0.080131284021773927, u'store': 0.16018251681075887, u'breakpoint': 8.0051232788984944e-05, u'ensur': 8.0051232788984944e-05, u'cross-cultur': 8.0051232788984944e-05, u'languag': 0.16018251681075887, u'phpeclips': 8.0051232788984944e-05, u'grow': 8.0051232788984944e-05, u'sever': 0.080131284021773927}}


In [54]:
# tot_bic_op = bic_output(diction, tot_so)
# tot_fic_op = fic_output(tot_probs, edge_wts)

In [55]:
def evalCrit(predTags, corrTags, tot_so):
    output = 0
    for i in xrange(len(tot_so)):
        p = set(predTags[i])
        c = set(corrTags[i])
        output += len(p.intersection(c)) / len(c)
    return output / len(tot_so)
    
    

In [56]:
def weightTuning(soNtags, tot_bic, tot_fic, k):
    final_eval = []
    for alpha in range(0, 10, 1):
        alpha = alpha / 10
        for beta in xrange(0, 10, 1):
            beta = beta / 10
#             print "Processing (%f, %f)" % (alpha, beta)
            eval_crit = 0
            for i in xrange(len(soNtags[0])):
                output = []
                
                bic = tot_bic_op[i]
                fic = tot_fic_op[i]
                
                tags = set(fic)
                tags.update(bic)
                
                for tag in tags:
                    value = alpha * bic.get(tag, 0) + beta * fic.get(tag, 0)
                    output.append((value, tag))
                    
                output.sort(reverse=True)
                
                so = soNtags[0][i]
                pred_tags = [o[1] for o in output[:k]]
                actual_tags = soNtags[1][i]
                evalVal = evalCrit([pred_tags], [actual_tags], [so])
                eval_crit +=  evalVal
            final_eval.append((eval_crit, (alpha, beta)))
    final_eval.sort(reverse=True)
    return final_eval[0]

In [58]:
# p = weightTuning(soNtags, tot_bic_op, tot_fic_op, 1)

In [73]:
v = {u'unit test saw mani question ask unit test specif languag question ask use use also common pitfal misconcept': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'country': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'air': 0.74701492537313419, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'connect php as/ ive got upcom project wherein need connect websit php/apach /openbsd back-end system run iseri os vr access tabl store ive done check around run roadblock ive seen db extens db softwar ibm run linux ive tri compil extens softwar ibm even tri precompil ibm_db extens luck ibm support linux turn linux emul kernel didnt seem help anyth anyon run across get everyth run nativ openbsd would great think may set second server run cento db instal like via zendcor ibm sinc seem driver set small transact server post get json represent db data need second option seem overkil anyon els better idea': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'arrays': 0.74701492537313419, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'index databas column hope get answer databas server outlin index work check http//stackoverflowcom/questions//how-does-database-indexing-work': {'debugging': 0.00074626865671641792, 'malloc': 0.74701492537313419, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'effici get sort sum sort list ascend list number effici algorithm think get ascend list sum everi two number list duplic result list irrelev remov avoid like clear im interest algorithm feel free post code languag paradigm like': {'variables': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'setup': 0.42777777777777776, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'registry': 0.42777777777777776, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'country': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'mechan track db schema chang best method track and/or autom db schema chang team use subvers version control weve abl autom task way push build stage server deploy test code product server still databas updat manual would like find creat solut allow us work effici across server differ environ continu use subvers backend code db updat push around variou server mani popular softwar packag includ auto-upd script detect db version appli necessari chang best way even larger scale across multipl project sometim multipl environ languag exist code simplifi process best roll solut anyon implement someth similar integr subvers post-commit hook bad idea solut support multipl platform would prefer definit need support linux/apache/mysql/php stack major work platform': {'sql-server-2008': 0.74701492537313419, 'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'text': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'winform combobox data bind gotcha ok im web develop find dabbl window form project everi one thing confound day follow assum someth like follow combo box bound array everyth work fine chang valu one combo box chang combo box one select know array alway pass refer learn learn c d earth would combo box chang togeth dont believ combo box control modifi collect work around dont would achiev funcion expect / desir': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'web-services': 0.74701492537313419, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'use nest class case work collect class use video playback record one main class act like public interfac method like etc workhors class video decod video encod learn exist nest class c++ im curiou know programm think use littl wari realli sure benefits/drawback seem accord book im read use case mine book suggest scenario like mine good solut would nest workhors class insid interfac class separ file class client meant use avoid possibl name conflict dont know justif nest class new concept want see programm think issu': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'security': 0.74701492537313419, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'use match attribut python object array dont rememb whether dream seem recal function allow someth like ive look doc kind thing doesnt fall obviou list header': {'ip': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'passwords': 0.42777777777777776, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'encryption': 0.42777777777777776, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'country': 0.0004273504273504274, 'text': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'anatomi "memori leak" net perspect memori leak determin whether applic leak effect prevent memori leak applic memori leak go away process exit kill memori leak applic affect process system even process complet unmanag code access via com interop and/or p/invok answer question incomplet think': {'vi': 0.00029940119760479042, 'monkeypatching': 0.00029940119760479042, 'exception': 0.00029940119760479042, 'unicode': 0.00029940119760479042, 'localization': 0.00029940119760479042, 'country': 0.00029940119760479042, 'pass-by-value': 0.00029940119760479042, 'flash': 0.00029940119760479042, 'exception-handling': 0.00029940119760479042, 'windows-server-2003': 0.00029940119760479042, 'connection': 0.00029940119760479042, 'lookup': 0.00029940119760479042, 'debugging': 0.00029940119760479042, 'branch': 0.29970059880239519, 'ip': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'tortoisesvn': 0.29970059880239519, 'text': 0.00029940119760479042, 'pass-by-reference': 0.00029940119760479042, 'branching-and-merging': 0.29970059880239519}, u'integr visual studio test project cruis control im look use visual studio built unit test project instead nunit wonder anyon experi tri integr type unit test project cruis controlnet': {'exception': 0.00029940119760479042, 'xml-comments': 0.00029940119760479042, 'localization': 0.00029940119760479042, 'sandcastle': 0.00029940119760479042, 'text': 0.00029940119760479042, 'pass-by-value': 0.00029940119760479042, 'documentation': 0.00029940119760479042, 'exception-handling': 0.00029940119760479042, 'vim': 0.29970059880239519, 'windows-server-2003': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'connection': 0.00029940119760479042, 'emacs': 0.29970059880239519, 'editor': 0.29970059880239519, 'unicode': 0.00029940119760479042, 'bayesian': 0.00029940119760479042, 'variables': 0.00029940119760479042, 'pass-by-reference': 0.00029940119760479042, 'xslt': 0.00029940119760479042, 'monkeypatching': 0.00029940119760479042}, u'make menu requir user press enter make select ive got menu python part easi im use get select user problem input requir user press enter make select way make program act immedi upon keystrok here ive got far would great someth like': {'debugging': 0.0029411764705882353, 'exception': 0.0029411764705882353, 'xml-comments': 0.0029411764705882353, 'localization': 0.0029411764705882353, 'c#': 0.0029411764705882353, 'ip': 0.0029411764705882353, 'pass-by-value': 0.0029411764705882353, 'exception-handling': 0.0029411764705882353, 'windows-server-2003': 0.0029411764705882353, 'sandcastle': 0.0029411764705882353, 'connection': 0.0029411764705882353, 'lookup': 0.0029411764705882353, 'unicode': 0.0029411764705882353, 'vi': 0.0029411764705882353, 'loops': 0.0029411764705882353, 'variables': 0.0029411764705882353, 'country': 0.0029411764705882353, 'text': 0.0029411764705882353, 'pass-by-reference': 0.0029411764705882353, 'monkeypatching': 0.0029411764705882353}, u'aspnet visual studio subvers - integr use ankhsvn visual studio one thing bug ankh realli work aspnet site cannot add properli repositori wont detect chang especi site remot server access frontpag extens file => open site altern better plug-in exist manual download file ftp use tortoisesvn svnex realli level integr want want stay within visual studio ide possibl also control remot server instal anyth mean whole chang tracking/comparison repositori done machin': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'collaboration': 0.74701492537313419, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'ide provid code complet python find type charact given keyword drastic speed code spend time write python code find reach ctrl+space ide support code complet python python develop mac os x tool would prefer seem like doc string properti python method perfect match inlin api discoveri': {'debugging': 0.00029940119760479042, 'exception': 0.00029940119760479042, 'localization': 0.00029940119760479042, 'c#': 0.00029940119760479042, 'ip': 0.00029940119760479042, 'pass-by-value': 0.00029940119760479042, 'odbc': 0.29970059880239519, 'exception-handling': 0.00029940119760479042, 'windows-server-2003': 0.00029940119760479042, 'ibm-midrange': 0.29970059880239519, 'connection': 0.00029940119760479042, 'lookup': 0.00029940119760479042, 'unicode': 0.00029940119760479042, 'vi': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'variables': 0.00029940119760479042, 'db2': 0.29970059880239519, 'text': 0.00029940119760479042, 'pass-by-reference': 0.00029940119760479042, 'monkeypatching': 0.00029940119760479042}, u'print html document web servic want print html c# web servic web browser control overkil function well servic environ function well system tight secur constraint sort free net librari support print basic html page code far run properli work fine call ui-typ thread noth happen call service-typ thread chang yield follow ie script error error dialogarguments___ie_printtyp null object url res//ieframedll/previewdlg small empti print preview dialog appear': {'debugging': 0.0029411764705882353, 'exception': 0.0029411764705882353, 'xml-comments': 0.0029411764705882353, 'localization': 0.0029411764705882353, 'c#': 0.0029411764705882353, 'ip': 0.0029411764705882353, 'pass-by-value': 0.0029411764705882353, 'exception-handling': 0.0029411764705882353, 'windows-server-2003': 0.0029411764705882353, 'sandcastle': 0.0029411764705882353, 'connection': 0.0029411764705882353, 'lookup': 0.0029411764705882353, 'unicode': 0.0029411764705882353, 'vi': 0.0029411764705882353, 'loops': 0.0029411764705882353, 'variables': 0.0029411764705882353, 'country': 0.0029411764705882353, 'text': 0.0029411764705882353, 'pass-by-reference': 0.0029411764705882353, 'monkeypatching': 0.0029411764705882353}, u'databas index work given index import dataset increas size someon explain index work databas agnost level inform queri index field check http//stackoverflowcom/questions//how-do-i-index-a-database-field': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'try-catch': 0.74701492537313419, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'best practic collabor environ bin directori svn best practic check bin directori collabor develop environ use svn project level refer exclud checkin easier add bin directori develop lot dotnetnuk site seem multi-develop environ alway huge task get environ setup correctli ultim goal cours new develop checkout trunk svn restor dnn databas work': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'c++': 0.42777777777777776, 'country': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'berkeley-db': 0.42777777777777776, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'get preview jpeg pdf window cross-platform python applic need gener jpeg preview first page pdf mac spawn sip someth similarli simpl window': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'winforms': 0.74701492537313419, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'longtim window user know start use linux weve final move websit decent host first time shell access know littl use linux navig file system read file vim im awar man command abl work solut problem show eventu know im unawar lot edit current use host hold live site im sure use effect im sure start web develop mind essenti command everi linux user know use command look': {'ip': 0.00074626865671641792, 'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'text': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'code-review': 0.74701492537313419, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'flat file databas best practic around creat flat file databas structur php lot matur php flat file framework see attempt implement sql-like queri syntax top purpos case would use databas point eleg trick get good perform featur small code overhead one would want take problem first place': {'ip': 0.00074626865671641792, 'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'actionscript-3': 0.74701492537313419, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'python mysql get python work postgresql cannot get work mysql main problem share host account abil instal thing django pysql gener fail instal comput mayb good cant instal host found bpgsql realli good requir instal singl file look read call function anybodi know someth like mysql': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'date': 0.42777777777777776, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'internationalization': 0.42777777777777776, 'variables': 0.0004273504273504274, 'text': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'best subvers client mac os window tortoisesvn daddi question mac noth simpl integr fair choic tool gui client subvers svn worth check': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.74701492537313419, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'cpu throttl c++ wonder eleg way set maximum cpu load particular thread intens calcul right locat time consum loop thread compress use hardcod valu make sure loop continu certain period time sleep certain minim time less job ie guarante thread use % cpu howev behavior depend number cpu core huge disadvantag simpli ugli smaller disadvantag idea': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'sql-injection': 0.74701492537313419, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'make subdomain user account webapp look allow user control subdomain app toy much like basecamp requir end allow creat dynam avail instantli recommend deal logic site rule lookup subdomain': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'open-source': 0.74701492537313419, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'build window nt use visual studio mfc applic im tri migrat use caus get set caus error set pleas use /md switch _afxdll build research date indic imposs build applic execut window nt use visual studio c++ case realli true workaround avail': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'random': 0.74701492537313419, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'big mysql databas get perform start degrad point mysql databas start lose perform physic databas size matter number record matter perform degrad linear exponenti believ larg databas roughli m record take almost gb base number incent clean data safe allow continu scale year': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'country': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'cvs': 0.74701492537313419, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'rockbox audio format specifi callback': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'curses': 0.74701492537313419, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'csv file import net realiz newbi question im look simpl solut - seem like one what best way import csv file strongly-typ data structur simpl = better': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'quotes': 0.42777777777777776, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'sandcastle': 0.0004273504273504274, 'anchor': 0.42777777777777776, 'monkeypatching': 0.0004273504273504274}, u'iphon app landscap mode system note question histor interest what best way creat iphon applic run landscap mode start regardless posit devic programmat use interfac builder': {'ip': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'vb6-migration': 0.42777777777777776, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'vb6': 0.42777777777777776, 'unicode': 0.0004273504273504274, 'text': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'check chang sql server tabl monitor sql server databas chang tabl without use trigger modifi structur databas way prefer program environ net c# id like abl support sql server sp newer applic bolt-on data visual anoth compani product custom base thousand dont want put requir modifi third-parti vendor tabl everi instal "chang table" mean chang tabl data chang tabl structur ultim would like chang trigger event applic instead check chang interv best cours action given requir trigger schema modif sql server seem use binary_checksum function t-sql way plan implement everi x second run follow queri compar store valu valu chang go tabl row row use queri compar return checksum store valu': {'debugging': 0.00029940119760479042, 'c': 0.29970059880239519, 'xml-comments': 0.00029940119760479042, 'localization': 0.00029940119760479042, 'c#': 0.00029940119760479042, 'vi': 0.00029940119760479042, 'pass-by-value': 0.00029940119760479042, 'mainframe': 0.29970059880239519, 'exception-handling': 0.00029940119760479042, 'windows-server-2003': 0.00029940119760479042, 'connection': 0.00029940119760479042, 'zos': 0.29970059880239519, 'text': 0.00029940119760479042, 'exception': 0.00029940119760479042, 'unicode': 0.00029940119760479042, 'ip': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'variables': 0.00029940119760479042, 'pass-by-reference': 0.00029940119760479042, 'monkeypatching': 0.00029940119760479042}, u'class view django django view point function problem want chang bit function ye could million keyword argument even statement function think object orient approach exampl page display user page similar page display group still similar use anoth data model group also member etc one way would point view class method extend class anyon tri approach idea': {'vi': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'gis': 0.42777777777777776, 'localization': 0.0004273504273504274, 'country': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'flash': 0.0004273504273504274, 'mapping': 0.42777777777777776, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'text': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'bandwith throttl ii ip address write applic download larg file background client log local vpn log local want throttl download howev would like limit download kbp user connect via vpn differenti user ip address rang sinc air applic figur throttl via server-sid sinc either server ii web servic aspnet / c# throttl ii seem work fine seem like done across entir web site anyway via ip rig net': {'ip': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xampp': 0.42777777777777776, 'server-configuration': 0.42777777777777776, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'text': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'string liter escap charact postgresql attempt insert escap charact tabl result warn exampl produc warn use psql anyon know get around': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'sandcastle': 0.0004273504273504274, 'connection': 0.42777777777777776, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.42777777777777776, 'variables': 0.0004273504273504274, 'country': 0.0004273504273504274, 'text': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'page collect linq page collect linq given': {'debugging': 0.0029411764705882353, 'exception': 0.0029411764705882353, 'xml-comments': 0.0029411764705882353, 'localization': 0.0029411764705882353, 'c#': 0.0029411764705882353, 'ip': 0.0029411764705882353, 'pass-by-value': 0.0029411764705882353, 'exception-handling': 0.0029411764705882353, 'windows-server-2003': 0.0029411764705882353, 'sandcastle': 0.0029411764705882353, 'connection': 0.0029411764705882353, 'lookup': 0.0029411764705882353, 'unicode': 0.0029411764705882353, 'vi': 0.0029411764705882353, 'loops': 0.0029411764705882353, 'variables': 0.0029411764705882353, 'country': 0.0029411764705882353, 'text': 0.0029411764705882353, 'pass-by-reference': 0.0029411764705882353, 'monkeypatching': 0.0029411764705882353}, u'embed databas net run network still look embed databas use net c# applic caveat applic least databas store network drive use user time first idea sql server compact edit realli nice integr run network firebird seem issu net integr seem realli first-class larg undocu blackfish sql look interest trial net version price also ok suggest someth work well net run network without need actual instal server softwar': {'debugging': 0.00029940119760479042, 'exception': 0.00029940119760479042, 'localization': 0.00029940119760479042, 'c#': 0.00029940119760479042, 'ip': 0.00029940119760479042, 'pass-by-value': 0.00029940119760479042, 'code-completion': 0.29970059880239519, 'exception-handling': 0.00029940119760479042, 'windows-server-2003': 0.00029940119760479042, 'wing-ide': 0.29970059880239519, 'connection': 0.00029940119760479042, 'lookup': 0.00029940119760479042, 'unicode': 0.00029940119760479042, 'vi': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'variables': 0.00029940119760479042, 'ide': 0.29970059880239519, 'text': 0.00029940119760479042, 'pass-by-reference': 0.00029940119760479042, 'monkeypatching': 0.00029940119760479042}, u'python os run need look see im window unix etc': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'image-processing': 0.74701492537313419, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'get malloc doubl free error realloc ive tri write string replac function c work alloc use littl differ find replac string rather charact start string trivial search replac string length replac string shorter search string sinc enough space alloc tri use get error tell doubl free - dont see sinc use perhap littl code help program work tri instanc replac string longer initi string still kind work spit error well result help call code look like': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'chm': 0.42777777777777776, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'winhelp': 0.42777777777777776, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'best way copi databa alway creat new empti databas backup restor exist databas realli best way seem error prone complic': {'ip': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'apple-touch-icon': 0.42777777777777776, 'exception-handling': 0.0004273504273504274, 'favicon': 0.42777777777777776, 'windows-server-2003': 0.0004273504273504274, 'sandcastle': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'text': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'error_log per virtual host one linux server run apach php got multipl virtual host separ logfil everyth thing cannot seem separ virtual host php overrid set seem anyth overlook someth way separ php virtual host': {'debugging': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'youtube': 0.42777777777777776, 'exception-handling': 0.0004273504273504274, 'country': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'reverse-engineering': 0.42777777777777776, 'text': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'function creat color wheel someth ive pseudo-solv mani time never quit found solut that stuck problem come way gener n color distinguish possibl n paramet': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'virtualhost': 0.74701492537313419, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'ensur except alway caught except c++ dont need caught compil time error call function develop judgment whether catch use try/catch unlik java way one ensur except thrown alway caught use try/catch call function': {'email-spam': 0.29970059880239519, 'debugging': 0.00029940119760479042, 'exception': 0.00029940119760479042, 'xml-comments': 0.00029940119760479042, 'artificial-intelligence': 0.29970059880239519, 'c#': 0.00029940119760479042, 'text': 0.00029940119760479042, 'pass-by-value': 0.00029940119760479042, 'localization': 0.00029940119760479042, 'exception-handling': 0.00029940119760479042, 'windows-server-2003': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'connection': 0.00029940119760479042, 'unicode': 0.00029940119760479042, 'vi': 0.00029940119760479042, 'bayesian': 0.29970059880239519, 'variables': 0.00029940119760479042, 'sandcastle': 0.00029940119760479042, 'pass-by-reference': 0.00029940119760479042, 'monkeypatching': 0.00029940119760479042}, u'much web build process you/should autom much web build process you/should autom system choic top head would say real one-step web build follow step would take place take sourc snapshot chang config file releas compress css javascript run test take databas snapshot without test data move sourc server import clean current version databas server automat confirm build success system step': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'string': 0.42777777777777776, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'sandcastle': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'cross-platform': 0.42777777777777776, 'chm': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'continu integr system python codebas im start work hobbi project python codebas would like set form continu integr ie run batteri test-cas time check-in made send nag e-mail respons person test fail similar cruisecontrol teamciti realiz could hook vcse requir test run machin version control server isnt eleg would like anyon suggest small user-friendli open-sourc continu integr system suitabl python codebas': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'crash': 0.42777777777777776, 'xml-comments': 0.0004273504273504274, 'ruby-on-rails': 0.42777777777777776, 'c#': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'text': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'best way handl multipl permiss type often encount follow scenario need offer mani differ type permiss primarili use aspnet / vbnet sql server scenario want offer dynam permiss system work differ paramet let say want give either depart specif person access applic pretend number applic keep grow past chosen one follow two way know use singl permiss tabl special column use determin appli paramet special column exampl typeid typeauxid sql would look someth like use map tabl type permiss join togeth thought hope exampl make sens cobbl togeth first exampl requir less work neither feel like best answer better way handl': {'debugging': 0.00029940119760479042, 'exception': 0.00029940119760479042, 'pass-by-reference': 0.00029940119760479042, 'localization': 0.00029940119760479042, 'c#': 0.00029940119760479042, 'vi': 0.00029940119760479042, 'http-authentication': 0.29970059880239519, 'exception-handling': 0.00029940119760479042, 'windows-server-2003': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'authentication': 0.29970059880239519, 'connection': 0.00029940119760479042, 'text': 0.00029940119760479042, 'unicode': 0.00029940119760479042, 'ip': 0.00029940119760479042, 'apache': 0.29970059880239519, 'variables': 0.00029940119760479042, 'lookup': 0.00029940119760479042, 'pass-by-value': 0.00029940119760479042, 'monkeypatching': 0.00029940119760479042}, u'give web site icon iphon set icon appear iphon web site creat': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'cruisecontrol.net': 0.74701492537313419, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'robust random number gener im look perform reason robust rng use special hardwar use mathemat method mersenn twister etc "collect entropy" machin whatev linux/etc drand gener random bit id like similar function/class c++ c# gener bit random low-ord bit equal random high-ord bit doesnt cryptograph secur must use base c-languag rand net systemrandom sourc code link sourc etc would appreciated! fail type rng look': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'cx-oracle': 0.42777777777777776, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'oracle': 0.42777777777777776, 'variables': 0.0004273504273504274, 'country': 0.0004273504273504274, 'text': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'tool convert visual j# code c# convers tool port visual j# code c#': {'css3': 0.42777777777777776, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'internet-explorer-7': 0.42777777777777776, 'exception-handling': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'country': 0.0004273504273504274, 'text': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'redirecttoact aspnet mvc without lose request data use aspnet mvc situat form submiss may requir one situat encount valid error form submiss need redirect back form would like url reflect url form action page submit requir form contain origin ed data user conveni well valid purpos pass data redirecttoact use viewdata paramet paramet chang paramet': {'debugging': 0.0004273504273504274, 'latitude-longitude': 0.42777777777777776, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'sandcastle': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'text': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'math': 0.42777777777777776, 'monkeypatching': 0.0004273504273504274}, u'solv linear equat need programmat solv system linear equat c object c need c++ here exampl equat id like get best approxim b tx': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'bpgsql': 0.74701492537313419, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'doesnt percentag width child absolut posit parent work absolut posit contain sever children one rel posit use percentage-bas width child collaps width ie firefox safari use pixel width work parent rel posit percentag width child work someth im miss easi fix besid pixel-bas width child area specif cover': {'opacity': 0.42777777777777776, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'type-conversion': 0.42777777777777776, 'country': 0.0004273504273504274, 'text': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'programmat launch ie mobil favorit screen way launch iemobil favorit screen directli specifi command line paramet': {'ip': 0.00074626865671641792, 'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'text': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'com': 0.74701492537313419, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'upgrad sql server ye know exist run copi absurd stipul best way migrat direct path document ive found deal upgrad forget nativ upgrad util script object data tri recreat scratch go attempt upgrad weekend server issu push back till next idea would welcom cours week updat end back databas question master execut master allow oledb provid connect use standalon creat dt packag use connect success copi tabl new databas also use use enterpris manag script databas index trigger sql file execut sql file new copi databas manag studio use enterpris manag script store procedur execut file databas sever dozen sproc issu make incompat mainli correct issu re-execut file recreat login gave appropri permiss bit rinse/repeat correct store procedur hundr correct upgrad went great otherwis abl use manag studio instead amaz differ report queri took - second run - second without modif new index anyth didnt expect kind immedi improv': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'globalization': 0.74701492537313419, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'good branch merg tutori tortoisesvn realli good tutori explain branch merg apach subvers better specif tortoisesvn client': {'debugging': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'text': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'timer': 0.42777777777777776, 'visual-c++': 0.42777777777777776, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'winhelp': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'gtk implement messagebox tri implement win use gtk app use sdl/opengl isnt gtk app handl initialis sort stuff insid function follow mean experienc gtk programm realis im probabl someth horribl wrong howev problem last dialog pop function stay around process exit idea': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, '.net': 0.42777777777777776, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'country': 0.0004273504273504274, 'text': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'scripting': 0.42777777777777776, 'monkeypatching': 0.0004273504273504274}, u'cx_oracl - best way iter result set sever way iter result set way best': {'debugging': 0.00029940119760479042, 'exception': 0.00029940119760479042, 'webdav': 0.29970059880239519, 'exchange-server': 0.29970059880239519, 'localization': 0.00029940119760479042, 'sql-injection': 0.00029940119760479042, 'vi': 0.00029940119760479042, 'pass-by-value': 0.00029940119760479042, 'mapi': 0.29970059880239519, 'exception-handling': 0.00029940119760479042, 'windows-server-2003': 0.00029940119760479042, 'sandcastle': 0.00029940119760479042, 'connection': 0.00029940119760479042, 'xml-comments': 0.00029940119760479042, 'unicode': 0.00029940119760479042, 'text': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'variables': 0.00029940119760479042, 'pass-by-reference': 0.00029940119760479042, 'monkeypatching': 0.00029940119760479042}, u'followup find accur "distance" color origin question look function attempt quantifi "distant" distinct two color question realli two part color space best repres human vision distanc metric space best repres human vision euclidean': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'country': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'audio': 0.74701492537313419, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'migrat larg app visual basic vbnet compani make monolith piec softwar custom use run car dealership origin applic written visual basic grown substanti last year hasnt realli busi case switch net framework sure newer consider better certain thing use correctli speed develop time there noth cant visual basic need abl get thing done harder visual basic almost noth imposs recent though microsoft drop support visual basic seem obviou switch new platform inevit way vbnet work requir think fundament differ heck even way pass variabl sub chang far tell switch net would requir complet re-design re-writ applic question posit tool avail smooth transit one work avoid sell custom boss suit like say where roi @rob burk boatload winform absolut ado code db access ye one easy-to-manag place asp web page none strictli desktop app custom control ye written us ie third parti stuff instal base ~ machin last time call microsoft tech support would time troubl get msdn librari work machin back day msdn wasnt avail onlin internet connect dial-up support case elev "solution" gave format hard drive reinstal everyth needless say didnt consid solut': {'debugging': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'search': 0.42777777777777776, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'lucene': 0.42777777777777776, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'text': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'net unit test packag get back bit net few-year use full-tim wonder good unit test packag day im familiar nunit year ago play briefli around ironrubi goal get someth like rspec go dont know much beyond realis could googl call day believ im like get better inform respons ask question - suggest': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'html-parsing': 0.42777777777777776, 'preg-replace': 0.42777777777777776, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'sandcastle': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'internation project implement internation actual project youv work took interest make softwar cross-cultur read famou post joel absolut minimum everi softwar develop absolut posit must know unicod charact set excuses! howev yet abl take advantag real project besid make sure use unicod string possibl make string unicod ensur understand encod everyth work tip iceberg everyth work date use control set us english speak peopl wasnt someth time work push project live look tip war stori peopl make softwar local real world project': {'debugging': 0.0029411764705882353, 'exception': 0.0029411764705882353, 'xml-comments': 0.0029411764705882353, 'localization': 0.0029411764705882353, 'c#': 0.0029411764705882353, 'ip': 0.0029411764705882353, 'pass-by-value': 0.0029411764705882353, 'exception-handling': 0.0029411764705882353, 'windows-server-2003': 0.0029411764705882353, 'sandcastle': 0.0029411764705882353, 'connection': 0.0029411764705882353, 'lookup': 0.0029411764705882353, 'unicode': 0.0029411764705882353, 'vi': 0.0029411764705882353, 'loops': 0.0029411764705882353, 'variables': 0.0029411764705882353, 'country': 0.0029411764705882353, 'text': 0.0029411764705882353, 'pass-by-reference': 0.0029411764705882353, 'monkeypatching': 0.0029411764705882353}, u'swap uniqu index column valu databas databas tabl one field primari key uniqu index want swap valu column two row could done two hack know delet row re-insert updat row valu swap updat actual valu dont want go seem appropri solut problem could anyon help': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'text': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'country': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274, 'proc': 0.42777777777777776, 'pass-by-reference': 0.0004273504273504274, 'lambda': 0.42777777777777776}, u'throw error mysql trigger trigger updat tabl throw error prevent updat tabl': {'variables': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'performance': 0.42777777777777776, 'pi': 0.42777777777777776, 'text': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'creat sqlite databas base xsd data set anybodi know way creat sqlite databas base xsd past ive use basic sqlite manag want fuse thing bit net develop possibl': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'github': 0.74701492537313419, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'heap corrupt win; locat im work multithread c++ applic corrupt heap usual tool locat corrupt seem inapplic old build month old sourc code exhibit behaviour recent releas around long time wasnt noticed; downsid sourc delta cant use identifi bug introduc - lot code chang repositori prompt crash behaviuor gener throughput system - socket transfer data mung intern represent set test data period caus app except variou place variou caus - includ heap alloc fail thu heap corrupt behaviour seem relat cpu power memori bandwidth; machin easier crash disabl hyper-thread core dual-cor core reduc rate elimin corrupt suggest time relat issu here rub run lightweight debug environ say heap corrupt reason easi reproduc - ten fifteen minut pass someth fail horrend except like run sophist debug environ ration purifi even microsoft applic verifi system becom memory-spe bound doesnt crash memory-bound cpu get disk light program go fast box consum g ram ive got choic abl reproduc problem identifi caus abl idenifi caus problem cant reproduc current best guess next get insan grunti box replac current dev box gb ram ; make possibl repro crash caus mis-behaviour run power debug environment; rewrit oper use mark memori read-onli soon done run os catch bad-guy who write freed memori ye sign desper hell rewrit ! wonder go make slow purifi et al ship purifi instrument built option colleagu walk past ask "stack overflow get stack overflow now!" question locat heap corruptor updat balanc seem gotten long way toward solv problem instead min app goe two hour crash yet suggest heap corrupt persist updat releas build visual studio seem dramat better; current suspicion rest implement ship reproduc problem produc dump might help analysi ill take note im concern dr watson trip fact heap get stomp anoth tri might use debug tool quit power time also lightweight got go moment much help someth goe wrong want catch vandal act mayb tool allow least narrow problem certain compon dont hold much hope desper time call sure compon project correct runtim librari set code gener categori vs project set im ill spend coupl hour tomorrow go workspac project check theyr compil link appropri flag updat took second select project dialog unselect find project dont right set right set': {'debugging': 0.00029940119760479042, 'exception': 0.00029940119760479042, 'xml-comments': 0.00029940119760479042, 'opengl': 0.29970059880239519, 'c#': 0.00029940119760479042, 'vi': 0.00029940119760479042, 'pass-by-value': 0.00029940119760479042, 'localization': 0.00029940119760479042, 'exception-handling': 0.00029940119760479042, 'windows-server-2003': 0.00029940119760479042, 'connection': 0.00029940119760479042, 'text': 0.00029940119760479042, 'unicode': 0.00029940119760479042, 'ip': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'variables': 0.00029940119760479042, '3d': 0.29970059880239519, 'pass-by-reference': 0.00029940119760479042, 'directx': 0.29970059880239519, 'monkeypatching': 0.00029940119760479042}, u'nant still support suitabl net /v use msbuild build stuff want use cruisecontrolnet build server ccnet refer nant lot look ccnet stuff nant could project configur msbuild also nant seem bit unsupport beta releas almost year old short actual quit happi msbuild especi sinc "official" compil front end bit uncomfort nant want judg prematur would reason use nant msbuild especi ccnet seem overlap bit nant term featur ad autom build relat stuff': {'variables': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'queue': 0.42777777777777776, 'connection': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'collections': 0.42777777777777776, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'text': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'implement "rememb me" rail applic rails-app sign box "rememb me" checkbox user check box remain log even close browser im keep track whether user log store id user session session implement rail session cooki persist make persist seem like hack surpris common function better way edit gareth answer pretti good would still like answer someon familiar rail uniqu cookiesessionstor': {'windows-services': 0.42777777777777776, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'scheduled-tasks': 0.42777777777777776, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'country': 0.0004273504273504274, 'text': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'calcul statist im write app help facilit research part involv statist calcul right research use program call spss part output care look like theyr realli concern valu problem background statist cant figur test call calcul thought valu might result f-test follow step given wikipedia got result differ give': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'compiler': 0.74701492537313419, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'modifi address bar url ajax app match current state im write ajax app user move app id like url address bar updat despit lack page reload basic id like abl bookmark point therebi return current state peopl handl maintain rest ajax app': {'debugging': 0.00029940119760479042, 'exception': 0.00029940119760479042, 'pass-by-value': 0.00029940119760479042, 'error-handling': 0.29970059880239519, 'parameters': 0.29970059880239519, 'localization': 0.00029940119760479042, 'vi': 0.00029940119760479042, 'pass-by-reference': 0.00029940119760479042, 'exception-handling': 0.00029940119760479042, 'windows-server-2003': 0.00029940119760479042, 'connection': 0.00029940119760479042, 'xml-comments': 0.00029940119760479042, 'text': 0.00029940119760479042, 'unicode': 0.00029940119760479042, 'ip': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'lookup': 0.00029940119760479042, 'php': 0.29970059880239519, 'variables': 0.00029940119760479042, 'monkeypatching': 0.00029940119760479042}, u'git better subvers ive use subvers year use sourcesaf love subvers combin tortoisesvn cant realli imagin could better yet there grow number develop claim subvers problem move new breed distribut version control system git git improv upon subvers': {'debugging': 0.0002304147465437788, 'exception': 0.0002304147465437788, 'xml-comments': 0.23064516129032256, 'xslt': 0.23064516129032256, 'sandcastle': 0.23064516129032256, 'vi': 0.0002304147465437788, 'pass-by-value': 0.0002304147465437788, 'documentation': 0.23064516129032256, 'exception-handling': 0.0002304147465437788, 'localization': 0.0002304147465437788, 'windows-server-2003': 0.0002304147465437788, 'try-catch': 0.0002304147465437788, 'connection': 0.0002304147465437788, 'malloc': 0.0002304147465437788, 'unicode': 0.0002304147465437788, 'text': 0.0002304147465437788, 'loops': 0.0002304147465437788, 'variables': 0.0002304147465437788, 'pass-by-reference': 0.0002304147465437788, 'monkeypatching': 0.0002304147465437788}, u'version control system databas structur chang often run follow problem work chang project requir new tabl column databas make databas modif continu work usual rememb write chang replic live system howev dont alway rememb ive chang dont alway rememb write make push live system get big obviou error ugh regardless fact may best practic situat version control system databas dont care specif databas technolog want know one exist happen work ms sql server great': {'variables': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'sorting': 0.42777777777777776, 'xml-comments': 0.0004273504273504274, 'dictionary': 0.42777777777777776, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'find full path font display name mac use photoshop javascript api find font given psd given font name return api want find actual physic font file correspond disc happen python program run osx guess im look one photoshop javascript python function osx api call python': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'http': 0.42777777777777776, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'text': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'country': 0.0004273504273504274, 'ruby': 0.42777777777777776, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'differ mathfloor mathtrunc differ net': {'debugging': 0.00029940119760479042, 'windows-server-2003': 0.00029940119760479042, 'exception': 0.00029940119760479042, 'localization': 0.00029940119760479042, 'c#': 0.00029940119760479042, 'ip': 0.00029940119760479042, 'pass-by-value': 0.00029940119760479042, 'exception-handling': 0.00029940119760479042, 'country': 0.00029940119760479042, 'connection': 0.00029940119760479042, 'variables': 0.00029940119760479042, 'lookup': 0.00029940119760479042, 'unicode': 0.00029940119760479042, 'vi': 0.00029940119760479042, 'timezone': 0.29970059880239519, 'monkeypatching': 0.00029940119760479042, 'text': 0.00029940119760479042, 'pass-by-reference': 0.00029940119760479042, 'timezoneoffset': 0.29970059880239519, 'browser': 0.29970059880239519}, u'get distinct order list name datat use linq let say column want collect uniqu name order alphabet follow queri ignor order claus get enforc': {'debugging': 0.0002304147465437788, 'exception': 0.0002304147465437788, 'html-form': 0.23064516129032256, 'localization': 0.0002304147465437788, 'c#': 0.0002304147465437788, 'ip': 0.0002304147465437788, 'pass-by-value': 0.0002304147465437788, 'exception-handling': 0.0002304147465437788, 'loops': 0.0002304147465437788, 'windows-server-2003': 0.0002304147465437788, 'submit-button': 0.23064516129032256, 'connection': 0.0002304147465437788, 'html': 0.23064516129032256, 'lookup': 0.0002304147465437788, 'unicode': 0.0002304147465437788, 'vi': 0.0002304147465437788, 'form-submit': 0.23064516129032256, 'country': 0.0002304147465437788, 'text': 0.0002304147465437788, 'monkeypatching': 0.0002304147465437788}, u'locat text within imag current work project goal locat text imag ocr text intent yet want basic obtain bound text within imag use aforgenet imag compon manipul assist sens anoth updat // ive sinc went along anoth rout project howev attempt obtain text use modi microsoft offic document imag allow ocr imag pull text eas': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'winapi': 0.42777777777777776, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'text': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'mouse': 0.42777777777777776, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'automat updat version number would like version properti applic increment build im sure enabl function visual studio / tri specifi assemblyvers * doesnt get exactli want im also use set file earlier attempt assembl version chang set got reset default sinc applic look set file anoth directori would like abl display version number form user find problem log version use well tell upgrad old releas short explan version work would also appreci build revis number get increment': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'sql': 0.74701492537313419, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'compress / decompress folder & file anyon know good way compress decompress file folder c# quickli handl larg file might necessari': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'xsd': 0.74701492537313419, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'feder sync subvers server possibl creat "federated" subvers server one server locat anoth locat b sync local version repositori automat way someon either locat interact repositori access respect local server therefor faster respons time': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'ternary-operator': 0.74701492537313419, 'monkeypatching': 0.00074626865671641792}, u'linux shell equival ii lamp develop consid move net ii platform one concern loss product due lack shell anyon els experi possibl linux shell equival window': {'ip': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'shell': 0.42777777777777776, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'sandcastle': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'text': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'bash': 0.42777777777777776, 'monkeypatching': 0.0004273504273504274}, u'use unsign valu sign one appropri use unsign variabl sign one loop hear lot opinion want see anyth resembl consensu know java doesnt unsign valu must conciou decis sun microsystem part': {'variables': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'nested-class': 0.42777777777777776, 'class': 0.42777777777777776, 'monkeypatching': 0.0004273504273504274}, u'gener list possibl permut string would go gener list possibl permut string x y charact length contain variabl list charact languag would work portabl': {'xml': 0.74701492537313419, 'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'best way access exchang use php im write cm applic php one requir must abl interfac custom exchang server ive written function time alway use webdav im lean away run site ii apach prefer window server thing would need includ ad contact given user address book send email given user run report contact user pretti easi webdav better way doesnt requir function like deprec time soon idea updat justin love idea use com object worri maintain rd product make everyth work john write web servic c# interfac function access php app also littl bit way far im % convinc either better webdav anyon show im silli': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'deployment': 0.42777777777777776, 'release-management': 0.42777777777777776, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'display float cube use directx opengl id like display ~ float cube use im look either sampl sourc code descript techniqu know kind thing easi accomplish d guru enough troubl get even one cube display correctli ive comb net good seri tutori although talk primit cant find inform larg number primit - forth': {'debugging': 0.0004273504273504274, 'floating-point': 0.42777777777777776, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'language-agnostic': 0.42777777777777776, 'text': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'berkeleydb concurr what optim level concurr c++ implement berkeleydb reason support mani thread hammer away db throughput start suffer resourc content ive read manual know set number lock locker databas page size etc id like advic someon real-world experi bdb concurr applic pretti simpl ill get put record kb cursor delet': {'variables': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'gtk': 0.42777777777777776, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'x11': 0.42777777777777776, 'lookup': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'get root permiss file insid vi often edit config file ill open one vi go save realiz didnt type way give vi sudo privileg save file seem recal see someth look stuff vi ago cant find': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.74701492537313419}, u'php variabl pass valu refer php variabl pass valu refer': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'git': 0.74701492537313419, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'use c socket api c++ z/o ive issu get c socket api work properli c++ specif although includ sys/socketh still get compil time error tell af_inet defin miss someth obviou could relat fact im code z/o problem much complic ; oh dear upon investig discov #ifdef im hit appar z/o isnt happi unless defin type socket im use littl action person idea oe socket thingi z/o socket programm perhap could give rundown work': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'mysql': 0.74701492537313419, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'modifi xfdl file updat # xfdl file extens identifi xfdl format document file belong xml-base document templat format standard format exactli like xml file format howev contain level encrypt use secur commun know view xfdl file use file viewer found also modifi save file filesave/sav id like howev modifi file fli suggest even possibl updat # success decod unzip xfdl xml file edit look way re-encod modifi xml file back base-gzip use rubi command line': {'debugging': 0.0029411764705882353, 'exception': 0.0029411764705882353, 'xml-comments': 0.0029411764705882353, 'localization': 0.0029411764705882353, 'c#': 0.0029411764705882353, 'ip': 0.0029411764705882353, 'pass-by-value': 0.0029411764705882353, 'exception-handling': 0.0029411764705882353, 'windows-server-2003': 0.0029411764705882353, 'sandcastle': 0.0029411764705882353, 'connection': 0.0029411764705882353, 'lookup': 0.0029411764705882353, 'unicode': 0.0029411764705882353, 'vi': 0.0029411764705882353, 'loops': 0.0029411764705882353, 'variables': 0.0029411764705882353, 'country': 0.0029411764705882353, 'text': 0.0029411764705882353, 'pass-by-reference': 0.0029411764705882353, 'monkeypatching': 0.0029411764705882353}, u'tool peopl use creat data dictionari project work coupl databas tabl column databas descript set extend properti sql part document go client need produc data dictionari show tabl column along collect meta data data-typ option constraint anyon use tool automat creat kind document tool use use data dictionari creator awesom doesnt seem data type option unless want add custom field fill': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'python': 0.74701492537313419, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'get current log os user adob air need name current log user air/flex applic applic deploy window machin think could attain regex user directori open way': {'debugging': 0.00029940119760479042, 'exception': 0.00029940119760479042, 'unicode': 0.00029940119760479042, 'pass-by-reference': 0.00029940119760479042, 'localization': 0.00029940119760479042, 'vi': 0.00029940119760479042, 'pass-by-value': 0.00029940119760479042, 'utilities': 0.29970059880239519, 'filesystems': 0.29970059880239519, 'windows-server-2003': 0.00029940119760479042, 'connection': 0.00029940119760479042, 'xml-comments': 0.00029940119760479042, 'lookup': 0.00029940119760479042, 'file': 0.29970059880239519, 'ip': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'variables': 0.00029940119760479042, 'text': 0.00029940119760479042, 'exception-handling': 0.00029940119760479042, 'monkeypatching': 0.00029940119760479042}, u'tripl quot delimit databound javascript string paramet aspnet delimit javascript databound string paramet anchor onclick event anchor tag aspnet repeat control onclick event anchor contain call javascript function javascript funciton take string input paramet string paramet popul databound valu repeat need doubl quot containerdataitem need singl quot onclick still need one delimit tripl quot input string paramet javascript function call sinc cant use singl quot ensur javascript function know input paramet string integ without extra quot around input string paramet javascript function think im pass integ cheer advanc knowledg drop anchor javascript': {'debugging': 0.0029411764705882353, 'exception': 0.0029411764705882353, 'xml-comments': 0.0029411764705882353, 'localization': 0.0029411764705882353, 'c#': 0.0029411764705882353, 'ip': 0.0029411764705882353, 'pass-by-value': 0.0029411764705882353, 'exception-handling': 0.0029411764705882353, 'windows-server-2003': 0.0029411764705882353, 'sandcastle': 0.0029411764705882353, 'connection': 0.0029411764705882353, 'lookup': 0.0029411764705882353, 'unicode': 0.0029411764705882353, 'vi': 0.0029411764705882353, 'loops': 0.0029411764705882353, 'variables': 0.0029411764705882353, 'country': 0.0029411764705882353, 'text': 0.0029411764705882353, 'pass-by-reference': 0.0029411764705882353, 'monkeypatching': 0.0029411764705882353}, u'sql server implement mysql replac mysql incred use yet properitari sql command wonder easili emul sql server start new transact either alway littl bit pain especi applic therefor alway keep version statement wonder easi univers way implement function sql server': {'mailto': 0.74701492537313419, 'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'net xml comment api document easi way produc msdn-style document visual studio xml output im patient enough set good xslt know im first person cross bridg also tri set sandcastl recent realli made eye cross either miss someth import process way involv know somebodi realli nice dead-simpl solut im reiter think format made paragraph non-invit read gave sandcastl tri realli hard time get set realli mind someth much simpler unless dont understand sandcastl process seem like aw lot extra baggag produc someth nice tester work': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'css': 0.74701492537313419, 'monkeypatching': 0.00074626865671641792}, u'multipl datacontext class ever appropri order fulli use linqtosql aspnet applic necessari creat datacontext class usual done use design vs ui perspect datacontext design section databas would like expos linqtosql integr set orm featur linqtosql question set project use larg databas tabl interconnect way foreign key first inclin make one huge datacontext class model entir databas way could theori though dont know would need practic use foreign key connect gener linqtosql easili go relat object code insert relat object etc howev give thought think may make sens creat multipl datacontext class one relat specif namespac logic interrel section within databas main concern instanti dispos one huge datacontext class time individu oper relat specif area databas would impos unnecessari imposit applic resourc addit easier creat manag smaller datacontext file one big one thing would lose would distant section databas would navig linqtosql even though chain relationship connect actual databas addit would tabl class would exist one datacontext thought experi whether multipl datacontext correspond db namespac appropri place addit one larg datacontext class correspond whole db': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.42777777777777776, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'sandcastle': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'asp.net-mvc': 0.42777777777777776, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'ad method exist object ive read possibl add method exist object eg class definit python think call monkey patch case duck punch understand alway good decis might one dont know python languag choic updat // est look like good answer john downey tri appear end true method exampl defin new patch function argument self write actual code way patch class method ask argument name self doesnt automag recogn ident class would happen defin within class definit mean call classpatchclass instead classpatch want function true method look like python isnt realli treat method variabl happen function callabl way attach actual method class oh ryan isnt exactli look isnt builtin function quit cool nonetheless': {'debugging': 0.0029411764705882353, 'exception': 0.0029411764705882353, 'xml-comments': 0.0029411764705882353, 'localization': 0.0029411764705882353, 'c#': 0.0029411764705882353, 'ip': 0.0029411764705882353, 'pass-by-value': 0.0029411764705882353, 'exception-handling': 0.0029411764705882353, 'windows-server-2003': 0.0029411764705882353, 'sandcastle': 0.0029411764705882353, 'connection': 0.0029411764705882353, 'lookup': 0.0029411764705882353, 'unicode': 0.0029411764705882353, 'vi': 0.0029411764705882353, 'loops': 0.0029411764705882353, 'variables': 0.0029411764705882353, 'country': 0.0029411764705882353, 'text': 0.0029411764705882353, 'pass-by-reference': 0.0029411764705882353, 'monkeypatching': 0.0029411764705882353}, u'best subvers client window vista bit ive use tortoisesvn window environ quit time seem feature-complet nice integr window shell importantli fairli painless teach colleagu littl experi sourc control howev sinc move window vista bit tortois buggi seem caus lot explorerex abnorm crash happen older version softwar latest version build curiou anyon suggest subvers client run window specif vista bit develop use varieti text editor use visual studio dreamweav svn ideal heard great thing cornerston would love someth similar window exist im correl vista/explor problem tortois normal occur im use function tortois sometim bring "merge" screen caus gui start act strang eventu hang crash see -- im instal mayb fix issu': {'variables': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'com-interop': 0.42777777777777776, 'memory-leaks': 0.42777777777777776, 'country': 0.0004273504273504274, 'text': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'frequent systemexit rubi make http call rubi rail websit make http call extern web servic day get systemexit stacktrac error email call servic fail tri exact queri site moment later work fine happen sinc site went live ive luck track caus rubi version rail version anyon els problem error stacktrac systemexit occur /usr/local/lib/ruby/gems//gems/rails-/lib/fcgi_handlerrbin `exit /usr/local/lib/ruby/gems//gems/rails-/lib/fcgi_handlerrbin `exit_now_handl /usr/local/lib/ruby/gems//gems/activesupport-/lib/active_support/inflectorrbin `to_proc /usr/local/lib/ruby//net/protocolrbin `call /usr/local/lib/ruby//net/protocolrbin `sysread /usr/local/lib/ruby//net/protocolrbin `rbuf_fil /usr/local/lib/ruby//timeoutrbin `timeout /usr/local/lib/ruby//timeoutrbin `timeout /usr/local/lib/ruby//net/protocolrbin `rbuf_fil /usr/local/lib/ruby//net/protocolrbin `readuntil /usr/local/lib/ruby//net/protocolrbin `readlin /usr/local/lib/ruby//net/httprbin `read_status_lin /usr/local/lib/ruby//net/httprbin `read_new /usr/local/lib/ruby//net/httprbin `request /usr/local/lib/ruby//net/httprbin `request_get /usr/local/lib/ruby//net/httprbin `get_respons /usr/local/lib/ruby//net/httprbin `start /usr/local/lib/ruby//net/httprbin `get_respons': {'variables': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'image': 0.42777777777777776, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'pdf': 0.42777777777777776, 'text': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'sqlstatementexecut - multipl queri one statement ive written databas gener script sql want execut adob air applic execut adob air use follow method error gener howev exist seem look first queri semicolon- remov queri fail way call multipl queri one statement': {'ip': 0.00029940119760479042, 'debugging': 0.00029940119760479042, 'exception': 0.00029940119760479042, 'localization': 0.00029940119760479042, 'c#': 0.00029940119760479042, 'subclipse': 0.29970059880239519, 'pass-by-value': 0.00029940119760479042, 'eclipse': 0.29970059880239519, 'exception-handling': 0.00029940119760479042, 'windows-server-2003': 0.00029940119760479042, 'connection': 0.00029940119760479042, 'aptana': 0.29970059880239519, 'text': 0.00029940119760479042, 'unicode': 0.00029940119760479042, 'vi': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'lookup': 0.00029940119760479042, 'country': 0.00029940119760479042, 'pass-by-reference': 0.00029940119760479042, 'monkeypatching': 0.00029940119760479042}, u'net test framework advic im look introduc unit test framework mix job use visual studio though may move within next month work primarili c# framework kind ide integr would best im open framework dont integr still rel simpl get set im go get resist one way anoth make sure im push isnt pain neck would help case obviou choic research ive done far point nunit id like get impress someon who actual use recommend team anyon use nunit pitfal limit awar good option youv use nunit id greatli appreci idea strength weak': {'vi': 0.00074626865671641792, 'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'embedded-database': 0.74701492537313419, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'xsd dataset ignor foreign key pretti standard tabl setup current applic use net xsd featur tabl consist standard contract inform column column foreign key tabl store basic setup function sql server use xsd tool drag tabl auto detects/cr foreign key two tabl work great im main page view contract data howev go administr page modifi depart data typic someth like howev point except thrown say effect foreign key refer broken im guess sinc dont fill fix problem know simpli remov foreign key xsd make thing work fine addit integr check xsd schema match sql schema databas nice': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'flash': 0.00074626865671641792, 'text': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'bcp': 0.74701492537313419, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'schedul task window run situat everi coupl month need develop solut that schedul run period havent found way handl window im entir happi everi time end feel corral use window servic seem unaccept amount overhead someth could cron job unix miss someth use schedul applic window @karl ye clearer someth ive steer away schedul task peopl work toward servic never realli gotten clear reason havent heavi window user year recent start develop platform im rather unfamiliar up down certain part @sasb command new seem like may im look link suggest window supersed schtask im guess command line window schedul task piec window': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'ie-mobile': 0.42777777777777776, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'text': 0.0004273504273504274, 'pocketpc': 0.42777777777777776, 'monkeypatching': 0.0004273504273504274}, u'learn python good exampl code dabbl coupl month read onlin docs/tutori start play bit well feel like start leav form toddler year readi seriou code everyth see gener encapsul singl script larg unwieldi dont know start would realli like see excel exampl proper python code style organ etc ideal associ decis idea look next would prefer simpl consol app extra py file mayb one packag good measur oh one thing reason come quit bit experi java net gener prefer one class definit per sourc file rare seem case python world least limit exposur doesnt smell right mayb enterpris python hobbyist python someth': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.74701492537313419, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'determin web user time zone standard way web server determin time zone offset user perhap header part user-ag descript': {'ip': 0.00074626865671641792, 'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'datediff': 0.74701492537313419, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'effici graph data structur python need abl manipul larg ^ node graph python data correspond node/edg minim say small number string effici term memori speed way dict dict flexibl simpler implement intuit expect list list faster list option would also requir keep data separ structur dict would allow someth sort graphij"property"="value" would suggest ye bit clearer mean effici particular case mean term random access retriev load data memori isnt huge problem that done time consum part visit node extract inform measur metric im interest hadnt consid make node class properti node seem like would add extra layer overhead hope someon would direct experi similar case could share graph one common abstract cs': {'debugging': 0.0029411764705882353, 'exception': 0.0029411764705882353, 'xml-comments': 0.0029411764705882353, 'localization': 0.0029411764705882353, 'c#': 0.0029411764705882353, 'ip': 0.0029411764705882353, 'pass-by-value': 0.0029411764705882353, 'exception-handling': 0.0029411764705882353, 'windows-server-2003': 0.0029411764705882353, 'sandcastle': 0.0029411764705882353, 'connection': 0.0029411764705882353, 'lookup': 0.0029411764705882353, 'unicode': 0.0029411764705882353, 'vi': 0.0029411764705882353, 'loops': 0.0029411764705882353, 'variables': 0.0029411764705882353, 'country': 0.0029411764705882353, 'text': 0.0029411764705882353, 'pass-by-reference': 0.0029411764705882353, 'monkeypatching': 0.0029411764705882353}, u'fastest way get valu \u03c0 solut welcom languag - im look fastest way obtain valu \u03c0 person challeng specif im use way dont involv use d constant like hard-cod number program test variou way know inlin assembl version theori fastest option though clearli portable; ive includ baselin compar version test built-in version fastest gcc auto-fold constant specifi version fastest here main test program inlin assembl stuff note work x x system build script build configur im test apart test variou compil flag ive compar -bit -bit optimis differ ive also tri switch order test around version still come top everi time though': {'windows-server-2003': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'data-storage': 0.42777777777777776, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'binary-data': 0.42777777777777776, 'text': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'lucen score result lucen multipl index cover one partit search differ index return result differ score result differ server match exactli ie search name - john smith dob - // partit would return score partit would return score match exactli name dob': {'debugging': 0.0002304147465437788, 'exception': 0.0002304147465437788, 'casting': 0.23064516129032256, 'unicode': 0.0002304147465437788, 'xml-comments': 0.0002304147465437788, 'warnings': 0.23064516129032256, 'type-safety': 0.23064516129032256, 'text': 0.0002304147465437788, 'pass-by-value': 0.0002304147465437788, 'localization': 0.0002304147465437788, 'exception-handling': 0.0002304147465437788, 'windows-server-2003': 0.0002304147465437788, 'c#': 0.0002304147465437788, 'connection': 0.0002304147465437788, 'generics': 0.23064516129032256, 'vi': 0.0002304147465437788, 'loops': 0.0002304147465437788, 'variables': 0.0002304147465437788, 'pass-by-reference': 0.0002304147465437788, 'monkeypatching': 0.0002304147465437788}, u'experi protocol buffer look inform googl protocol buffer data interchang format anyon play around code even creat project around im current use xml python project structur content creat hand text editor wonder gener opinion protocol buffer user-fac input format speed breviti benefit definit seem mani factor come actual gener process data': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'resources': 0.74701492537313419, 'monkeypatching': 0.00074626865671641792}, u'object orient vs relat databas object orient databas seem like realli cool idea need worri map domain model databas model mess around sql orm tool way understand relat db offer advantag massiv amount data search index need done mind % websit massiv enterpris issu never need thought arent oo db wide use': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'iteration': 0.74701492537313419, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'deploy sql server databas test live wonder guy manag deploy databas sql server specif sql server develop live one part buildscript standard window batch even current complex script might switch powershel later enterpris manager/manag studio express count would copi mdf file attach alway bit care work binari data seem compatibl issu even though develop live run version server time - given lack "explain creat table" t-sql - someth export exist databas sql-script run target server ye tool automat dump given databas sql queri run command line enterpris manager/manag studio express count lastli - given fact live databas alreadi contain data deploy may involv creat tabl rather check differ structur alter tabl live one instead may also need data verification/convers exist field chang hear lot great stuff red gate product hobbi project price bit steep use automat deploy sql server databas test live': {'variables': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'int': 0.42777777777777776, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'integer': 0.42777777777777776, 'lookup': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'effici code first prime number want print first prime number anyon give effici code clarif matter code ineffici n > size code matter cannot hard code valu manner': {'variables': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'windows-nt': 0.42777777777777776, 'country': 0.0004273504273504274, 'text': 0.0004273504273504274, 'compatibility': 0.42777777777777776, 'monkeypatching': 0.0004273504273504274}, u'connect databas loop recordset c# what simplest way connect queri databas set record c#': {'debugging': 0.0029411764705882353, 'exception': 0.0029411764705882353, 'xml-comments': 0.0029411764705882353, 'localization': 0.0029411764705882353, 'c#': 0.0029411764705882353, 'ip': 0.0029411764705882353, 'pass-by-value': 0.0029411764705882353, 'exception-handling': 0.0029411764705882353, 'windows-server-2003': 0.0029411764705882353, 'sandcastle': 0.0029411764705882353, 'connection': 0.0029411764705882353, 'lookup': 0.0029411764705882353, 'unicode': 0.0029411764705882353, 'vi': 0.0029411764705882353, 'loops': 0.0029411764705882353, 'variables': 0.0029411764705882353, 'country': 0.0029411764705882353, 'text': 0.0029411764705882353, 'pass-by-reference': 0.0029411764705882353, 'monkeypatching': 0.0029411764705882353}, u'learn write compil prefer languag c/c++ java rubi look help books/tutori write compil simpli educ purpos familiar c/c++ java rubi prefer resourc involv one three good resourc accept': {'ip': 0.00074626865671641792, 'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'text': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'podcast': 0.74701492537313419, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'display flash content c# winform applic best way display content c# winform applic would like creat user control similar current abl display imag flash content would great abl load flash content stream sort rather file disk': {'debugging': 0.00029940119760479042, 'exception': 0.00029940119760479042, 'xml-comments': 0.00029940119760479042, 'localization': 0.00029940119760479042, 'c#': 0.00029940119760479042, 'ip': 0.29970059880239519, 'pass-by-value': 0.00029940119760479042, 'exception-handling': 0.00029940119760479042, 'windows-server-2003': 0.00029940119760479042, 'sandcastle': 0.00029940119760479042, 'connection': 0.00029940119760479042, 'lookup': 0.29970059880239519, 'unicode': 0.00029940119760479042, 'vi': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'variables': 0.00029940119760479042, 'country': 0.29970059880239519, 'text': 0.00029940119760479042, 'pass-by-reference': 0.00029940119760479042, 'monkeypatching': 0.00029940119760479042}, u'regex pull sub-str two tag string file follow format data data data start data want end data id like grab tag use regex anyon show might done': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'safari': 0.42777777777777776, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'viewstate': 0.42777777777777776, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'net migrat engin belief microsoft work offici ruby-lik migrat framework howev havent abl find addit inform even origin sourc belief anyon know inform offici migrat framework possibl open sourc one': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'version-control': 0.74701492537313419, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'localis date format descriptor best way localis date format descriptor anyon cultur use mm/dd/yyyy format know annoy enter date format net framework provid good localis support trivial pars date accord user cultur often want also display help hint format requir especi distinguish yy yyyi interchang cultur best way result make sens user ie dd/m/yyi confus chang case switch two one letter': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'reflection': 0.74701492537313419, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'fix unprocess view path found error exceptionnotifi plugin rail upgrad rail websit exceptionnotifi plugin longer work complain error caus fix': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'asp.net': 0.74701492537313419, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'get valu built encod viewstat need grab base-encod represent viewstat obvious would avail fairli late request lifecycl ok exampl output page includ need way server side get valu clarifi need valu page render postback eg need know viewstat valu sent client viewstat im get back': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.74701492537313419, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'correct pixel dimens apple-touch-icon im sure correct size mani site seem repeat apple-touch-icon x pixel cite broken link sourc hanselman playgroundbluess comment suggest differ size includ x x appl applecom icon x! see relat question give web site icon iphon': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'command-line': 0.42777777777777776, 'localization': 0.0004273504273504274, 'windows': 0.42777777777777776, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'fill dataset datat linq queri result set expos linq queri asmx web servic usual busi tier return type serial transport asmx linq queri way popul type via linq queri get resultset linq queri altern linq queri serializ expos asmx web servic': {'ip': 0.00074626865671641792, 'debugging': 0.00074626865671641792, 'msdn': 0.74701492537313419, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'country': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'detect font use web page suppos follow css rule html detect one defin font use user browser edit peopl wonder want font detect contain glyph avail font user want display link ask user download font use web applic current display download font link user display peopl correct font instal': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'msbuild': 0.74701492537313419, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'aspnet built user profil vs old stile user class/tabl look guidanc regard best practic around use profil featur aspnet decid kept built-in user profil creat db tabl add column desir field exampl user zip code save zip code tabl add webconfig xml profil access via user profil aspnet mechan pros/con think sinc dont know profil well bit matrix right probabl whatev want go tabl rout eg sql get user zip code current user; dont know use aspnet profil': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'statistics': 0.42777777777777776, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'spss': 0.42777777777777776, 'windows-server-2003': 0.0004273504273504274, 'sandcastle': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'prefer version vim emac mac os x us like use graphic version vim emac instead consol version version recommend vim there mac os x vim macvim vim-cocoa emac carbonemac xemac aquamac readi prime-tim tough call trade-off still maintain discuss vim vs emac dont mind comparison editor': {'ip': 0.00029940119760479042, 'binary': 0.29970059880239519, 'exception': 0.00029940119760479042, 'literals': 0.29970059880239519, 'xml-comments': 0.00029940119760479042, 'debugging': 0.00029940119760479042, 'c#': 0.00029940119760479042, 'text': 0.00029940119760479042, 'pass-by-value': 0.00029940119760479042, 'localization': 0.00029940119760479042, 'exception-handling': 0.00029940119760479042, 'syntax': 0.29970059880239519, 'windows-server-2003': 0.00029940119760479042, 'connection': 0.00029940119760479042, 'unicode': 0.00029940119760479042, 'vi': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'variables': 0.00029940119760479042, 'pass-by-reference': 0.00029940119760479042, 'monkeypatching': 0.00029940119760479042}, u'use mutex visual basic import kernel librari function avail quit sure variou paramet return valu classic visual basic visual basicnet probabl work either languag form answer': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.74701492537313419, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'make sure email send programmat automat mark spam tricki one ive alway reli techniqu permission-bas email ie send peopl permiss send use blatantli spamish terminolog late email send programmat start shuffl peopl spam folder automat im wonder despit fact particular email one human would mark spam specif email contain licens key peopl paid good money dont think theyr go consid spam figur big topic essenti ignor simpleton': {'windows-server-2003': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'debugging': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'documentation': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'ant': 0.74701492537313419, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'asp need use sftp asp classic net get way sftp server upload download coupl file kick user peopl use sftp asp classic necessarili oppos purchas control': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'object-database': 0.42777777777777776, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'oop': 0.42777777777777776, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'country': 0.0004273504273504274, 'text': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'binari data mysql store binari data mysql': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'flash': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'linq': 0.74701492537313419, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'sort dictionari valu often dictionari key & valu need sort valu exampl hash word frequenc want order frequenc there good singl valu frequenc want map back word sorteddictionari order key valu resort custom class what cleanest way': {'svn': 0.74701492537313419, 'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'wiggl mous ok bit vaniti app situat today work train class machin set lock everi minut well trainer got excit talk - oppos chang slide - machin would lock id like write teeni app noth taskbar icon noth move mous pixel everi minut way delphi strong languag im move c# work id like know path least resist': {'debugging': 0.0029411764705882353, 'exception': 0.0029411764705882353, 'xml-comments': 0.0029411764705882353, 'localization': 0.0029411764705882353, 'c#': 0.0029411764705882353, 'ip': 0.0029411764705882353, 'pass-by-value': 0.0029411764705882353, 'exception-handling': 0.0029411764705882353, 'windows-server-2003': 0.0029411764705882353, 'sandcastle': 0.0029411764705882353, 'connection': 0.0029411764705882353, 'lookup': 0.0029411764705882353, 'unicode': 0.0029411764705882353, 'vi': 0.0029411764705882353, 'loops': 0.0029411764705882353, 'variables': 0.0029411764705882353, 'country': 0.0029411764705882353, 'text': 0.0029411764705882353, 'pass-by-reference': 0.0029411764705882353, 'monkeypatching': 0.0029411764705882353}, u'homegrown consumpt web servic ive write web servic net app im readi consum ive seen numer exampl homegrown code consum servic oppos use auto gener method visual studio creat ad web refer advantag': {'debugging': 0.0004273504273504274, 'photoshop': 0.42777777777777776, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'fonts': 0.42777777777777776, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'text': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'gettimeofday guarante microsecond resolut find port game origin written win api linux well port os x port win port linux implement give usecond sinc process start coupl give constant frequenc work well machin give bit variabl contain sinc program start portabl dont want discov work differ kernel compil certain way anyth like fine non-port someth linux howev': {'ip': 0.00074626865671641792, 'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'flat-file': 0.74701492537313419, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'better file copi altern window default need copi hundr gig random file around comput pretti leeri use vanilla file copi built window dont want hang "are sure" "are realli sure" "even zip files" "sure read-onli file too!" loop soon step away dont want work hour stop unexpectedli "someon open file wont copi it!" cancel whole copi quit indic done work remain file manag program experi recommend question relat question use old pata hard disk drive newer sata-onli comput': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'colors': 0.74701492537313419, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'littl divers float point imprecis part mathematician agre e\u03c0i + = howev float point implement disagre well settl disput im keen hear differ languag implement variou method make result close zero possibl creative!': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'full-text-search': 0.74701492537313419, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'visual studio setup project - per user registri set im tri maintain setup project ye legaci applic problem moment need write registri entri everi user comput need rather default user set chang per user feel isnt possibl isnt someth instal someth applic happen user profil creat instal mind still want chang littl possibl applic question possibl add registri entri everi user setup project moment project list five registri root key user/machin hive dont realli know anyth user root key havent seen user/machin hive anyon enlighten perhap could solv problem': {'variables': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'asp-classic': 0.42777777777777776, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'sftp': 0.42777777777777776, 'windows-server-2003': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'country': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'java lib app convert csv xml file exist applic librari java allow convert csv data file xml file xml tag would provid possibl first row contain column head': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'text': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'membership': 0.42777777777777776, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'country': 0.0004273504273504274, 'navigation': 0.42777777777777776, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'aspnet site map anyon got experi creat sql-base aspnet site-map provid ive got default file work properli control ill need way user site creat modifi page dynam need tie page view permiss standard membership system well id like advic articl video thank': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'hex': 0.42777777777777776, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'text': 0.0004273504273504274, 'country': 0.0004273504273504274, 'ascii': 0.42777777777777776, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'followup "sorting" color distinct origin question given n maxim distant color associ distanc metric come way sort color order first m also reason close maxim distinct set word given bunch distinct color come order use mani color need start begin reason assur distinct nearbi color also distinct eg bluish red isnt next reddish blue random ok certainli optim clarif given larg visual distinct set color say want sort use first say get rel visual distinct subset color equival roughli say want sort list closer individu color visual farther apart list': {'debugging': 0.00074626865671641792, 'build-process': 0.74701492537313419, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'text': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'doesnt sql full text index return result word contain # instanc queri like follow use sql server full text index defin use column searchfield return result use believ # special letter allow freetext work correctli queri': {'regex': 0.74701492537313419, 'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'window help file - option back old day help trivial possibl gener funki rtf file special tag run compil got winhelp file hlp actual work realli well microsoft decid winhelp hip cool anymor switch chm point actual axe winhelp vista chm mayb nice everyon tri open chm file network know nice "navig webpag canceled" screen caus secur restrict way make chm work network hardli good choic user press help button want help make funki set bottom line find chm absolut unus winhelp option anymor either wonder altern especi come integr applic ie winhelp chm function allow directli jump topic pdf disadvantag requir adob reader one lightweight one mani peopl use could live see kind standard nowaday tell reliabl jump given page/anchor html file seem best choic deal differ browser css stuff edit look creat help file fan "no setup extract run" philosophi problem mani time past mani user run network caus exactli problem look robust future-proof way provid help user without code differ help system applic make chm realli nice format secur stuff make unus help system suppos provid help user gener even problem': {'debugging': 0.0029411764705882353, 'exception': 0.0029411764705882353, 'xml-comments': 0.0029411764705882353, 'localization': 0.0029411764705882353, 'c#': 0.0029411764705882353, 'ip': 0.0029411764705882353, 'pass-by-value': 0.0029411764705882353, 'exception-handling': 0.0029411764705882353, 'windows-server-2003': 0.0029411764705882353, 'sandcastle': 0.0029411764705882353, 'connection': 0.0029411764705882353, 'lookup': 0.0029411764705882353, 'unicode': 0.0029411764705882353, 'vi': 0.0029411764705882353, 'loops': 0.0029411764705882353, 'variables': 0.0029411764705882353, 'country': 0.0029411764705882353, 'text': 0.0029411764705882353, 'pass-by-reference': 0.0029411764705882353, 'monkeypatching': 0.0029411764705882353}, u'use ncurs rubi id like creat progress bar indic statu batch job rubi ive read tutori / librari use ncurs none particularli help explain creat "animated" progress bar termin use curs rubi im alreadi awar use separ thread monitor progress given job im sure proceed draw progress bar updat progressbar class incred straight-forward perfectli solv problem': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'filelock': 0.42777777777777776, 'unicode': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'io': 0.42777777777777776, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'text': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'get new object instanc type one may alway know type object compile-tim may need creat instanc type get new object instanc type': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'format': 0.42777777777777776, 'c#': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'country': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'title-case': 0.42777777777777776, 'monkeypatching': 0.0004273504273504274}, u'encrypt password fastest yet secur way encrypt password php prefer ever method choos portablein word later migrat websit differ server password continu workth method use told depend exact version librari instal server': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'active-directory': 0.74701492537313419, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'set form opac use decim doubl want use track-bar chang form opac code tri build get error cannot implicitli convert type decim doubl tri make control doesnt work code work fine vbnet past': {'windows-mobile': 0.065254237288135591, 'flex': 0.065254237288135591, 'rockbox': 0.065254237288135591, 'pass-by-reference': 6.5189048239895696e-05, 'localization': 6.5189048239895696e-05, 'url': 0.065254237288135591, 'variables': 6.5189048239895696e-05, 'exception-handling': 6.5189048239895696e-05, 'django': 0.065254237288135591, 'terminal': 0.065254237288135591, 'maps': 0.065254237288135591, 'data-dictionary': 0.065254237288135591, 'design': 0.065254237288135591, 'unicode': 0.065254237288135591, 'wpf': 0.065254237288135591, 'monkeypatching': 6.5189048239895696e-05, 'data-binding': 0.065254237288135591, 'annotations': 0.065254237288135591, 'sitemap': 0.065254237288135591, 'phing': 0.065254237288135591}, u'get authent user name apach use plain http authent php first let get secur consider way im use simpl authent apach one-off intern use non-internet connect lan php web app get http authent user name php': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'iis': 0.74701492537313419, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'client collat sql server upgrad exist program well purchas new program also use vendor say us host databas need get enterpris version softwar client use differ collat connect enterpris support cannot find anyth mss site support honestli dont want pay extra enterpris standard edit work miss talk featur sql server suspect vendor tri upsel': {'ip': 0.00074626865671641792, 'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'text': 0.74701492537313419, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'debug php script debug php script awar basic debug use error report breakpoint debug phpeclips also quit use good/bett techniqu': {'debugging': 0.00029940119760479042, 'exception': 0.00029940119760479042, 'xml-comments': 0.00029940119760479042, 'pass-by-reference': 0.29970059880239519, 'localization': 0.00029940119760479042, 'c#': 0.00029940119760479042, 'ip': 0.00029940119760479042, 'variables': 0.29970059880239519, 'exception-handling': 0.00029940119760479042, 'windows-server-2003': 0.00029940119760479042, 'sandcastle': 0.00029940119760479042, 'connection': 0.00029940119760479042, 'lookup': 0.00029940119760479042, 'unicode': 0.00029940119760479042, 'vi': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'country': 0.00029940119760479042, 'text': 0.00029940119760479042, 'pass-by-value': 0.29970059880239519, 'monkeypatching': 0.00029940119760479042}, u'regist window program mailto protocol programmat make link regist program would handl event program solut found quick googl search manual need automat user program click button "set default email client" edit remov refer delphi answer independ languag': {'vi': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'country': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'flash': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'tag-cloud': 0.42777777777777776, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'text': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'cloud': 0.42777777777777776, 'monkeypatching': 0.0004273504273504274}, u'differ int integ java c# sit local border sip coffe read joel softwar free came across joel spolski say someth particular type programm know differ int integ java/c# object orient program languag quick brain check realiz dismay didnt know answer': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'testing': 0.74701492537313419, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'subson vs nhibern concensu use one tool advers find subson use term get thing done quickli larg project tend scale tie domain model databas model nhibern come give lightweight poco unrel databas model setup time much longer': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'glossary': 0.74701492537313419, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'unload com control work vb ide part everyday work maintain extend legaci vb applic common engin written c/c++ vb use function order improv perform come asynchron program c interfac enough reli com control fire event vb problem regist control vb vb load control memori unload quit vb ide control load whole time unabl recompil vc dll file lock solut found enabl control vb use createobject full name control problem must declar control object vb know noth interfac use access intellisens pain idea tell vb unload control quit applic directli ide': {'ip': 0.00074626865671641792, 'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'objective-c': 0.74701492537313419, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'text': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'xml editing/view softwar softwar recommend work edit larg xml schema im look window linux softwar doesnt cross platform want suggest help deal huge xml file': {'ip': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'sandcastle': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'projects-and-solutions': 0.42777777777777776, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'visual-studio': 0.42777777777777776, 'monkeypatching': 0.0004273504273504274}, u'unhandl except handler net im maintain applic one thing ive task make sure user doesnt see unfriendli error notif ive ad handler get call problem standard error dialog still display except handler call jeff talk problem blog there solut standard way handl uncaught except display friendli dialog box edit jeff respons mark correct answer link provid complet inform what requir': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'postgresql': 0.74701492537313419, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'edit databas record multipl user design databas tabl normalis ms sql server creat standalon window front end applic use hand user add edit inform add web interfac allow search accross product area later date concern two user start edit record last commit updat would winner import inform may lost number solut come mind im sure go creat bigger headach noth hope two user never go edit record time - might never hap edit routin could store copi origin data well updat compar user finish edit differ show user comfirm updat - would requir two copi data store add last updat datetim column check match updat show differ - requir new column relev tabl creat edit tabl regist user start edit record check prevent user edit record - would requir car thought program flow prevent deadlock record becom lock user crash program better solut go one': {'debugging': 0.0029411764705882353, 'exception': 0.0029411764705882353, 'xml-comments': 0.0029411764705882353, 'localization': 0.0029411764705882353, 'c#': 0.0029411764705882353, 'ip': 0.0029411764705882353, 'pass-by-value': 0.0029411764705882353, 'exception-handling': 0.0029411764705882353, 'windows-server-2003': 0.0029411764705882353, 'sandcastle': 0.0029411764705882353, 'connection': 0.0029411764705882353, 'lookup': 0.0029411764705882353, 'unicode': 0.0029411764705882353, 'vi': 0.0029411764705882353, 'loops': 0.0029411764705882353, 'variables': 0.0029411764705882353, 'country': 0.0029411764705882353, 'text': 0.0029411764705882353, 'pass-by-reference': 0.0029411764705882353, 'monkeypatching': 0.0029411764705882353}, u'php session secur guidelin maintain respons session secur php there inform web time land one place!': {'debugging': 0.0029411764705882353, 'exception': 0.0029411764705882353, 'xml-comments': 0.0029411764705882353, 'localization': 0.0029411764705882353, 'c#': 0.0029411764705882353, 'ip': 0.0029411764705882353, 'pass-by-value': 0.0029411764705882353, 'exception-handling': 0.0029411764705882353, 'windows-server-2003': 0.0029411764705882353, 'sandcastle': 0.0029411764705882353, 'connection': 0.0029411764705882353, 'lookup': 0.0029411764705882353, 'unicode': 0.0029411764705882353, 'vi': 0.0029411764705882353, 'loops': 0.0029411764705882353, 'variables': 0.0029411764705882353, 'country': 0.0029411764705882353, 'text': 0.0029411764705882353, 'pass-by-reference': 0.0029411764705882353, 'monkeypatching': 0.0029411764705882353}, u'check file lock way check whether file lock without use try/catch block right way know open file catch systemioioexcept': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'nant': 0.74701492537313419, 'monkeypatching': 0.00074626865671641792}, u'make xampp/apach serv file outsid htdoc possibl configur xampp serv file outsid directori instanc say file xampp file normal serv that configur default way make apach recogn serv file without move prefer id like apach serv up/hav access entir content project directori dont want move project directori edit edit add apach question titl make q/a "searchable"': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'javascript': 0.42777777777777776, 'exception-handling': 0.0004273504273504274, 'sandcastle': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'validation': 0.42777777777777776, 'text': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'checklist ii /aspnet window authent ive troubl get aspnet applic automat log user intranet site im build matter googl experiment appli alway login box display ie ive got window authent mode set webconfig disabl anonym access configur correct default domain ii still ask user log annoyingli user requir provid domain domain\\aus caus problem non-techn visitor thank zeu password rememb function im network administr possibl someth activ directori set incorrectli could miss someth simpl pleas note dont want imperson user need know iprincipalnam properti match valid record user databas henc authent user applic end would use checklist configur requir ad aspnet ii work togeth manner refer debug hope reduc user friction': {'debugging': 0.0029411764705882353, 'exception': 0.0029411764705882353, 'xml-comments': 0.0029411764705882353, 'localization': 0.0029411764705882353, 'c#': 0.0029411764705882353, 'ip': 0.0029411764705882353, 'pass-by-value': 0.0029411764705882353, 'exception-handling': 0.0029411764705882353, 'windows-server-2003': 0.0029411764705882353, 'sandcastle': 0.0029411764705882353, 'connection': 0.0029411764705882353, 'lookup': 0.0029411764705882353, 'unicode': 0.0029411764705882353, 'vi': 0.0029411764705882353, 'loops': 0.0029411764705882353, 'variables': 0.0029411764705882353, 'country': 0.0029411764705882353, 'text': 0.0029411764705882353, 'pass-by-reference': 0.0029411764705882353, 'monkeypatching': 0.0029411764705882353}, u'mean type safeti warn certain java gener cast mean java warn type safeti cast object list actual check eras type list get warn tri cast object type gener inform follow code': {'ip': 0.00074626865671641792, 'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'text': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'email': 0.74701492537313419, 'monkeypatching': 0.00074626865671641792}, u'access remot form php want gather info user local php page control use info queri form anoth site dont control': {'debugging': 0.00074626865671641792, 'sqlite': 0.74701492537313419, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'use mstest cruisecontrolnet use cruisecontrol quit nunit nant recent project decid use test framework come visual studio far adequ im attempt get solut run cruisecontrol ive final got build work; howev unabl get test show cruisecontrol interfac despit ad custom build task compon design anyon definit link instruct get set': {'debugging': 0.0029411764705882353, 'exception': 0.0029411764705882353, 'xml-comments': 0.0029411764705882353, 'localization': 0.0029411764705882353, 'c#': 0.0029411764705882353, 'ip': 0.0029411764705882353, 'pass-by-value': 0.0029411764705882353, 'exception-handling': 0.0029411764705882353, 'windows-server-2003': 0.0029411764705882353, 'sandcastle': 0.0029411764705882353, 'connection': 0.0029411764705882353, 'lookup': 0.0029411764705882353, 'unicode': 0.0029411764705882353, 'vi': 0.0029411764705882353, 'loops': 0.0029411764705882353, 'variables': 0.0029411764705882353, 'country': 0.0029411764705882353, 'text': 0.0029411764705882353, 'pass-by-reference': 0.0029411764705882353, 'monkeypatching': 0.0029411764705882353}, u'asynchron multi-direct server-cli commun open socket client-serv app client window mobil devic written c++ server full window written c# origin need send messag client server server ever send back acknowledg receiv messag would like updat server actual send messag client request data current set client receiv mode send data server doesnt allow server send request time would wait client data first thought would creat anoth thread client separ open socket listen server requestsjust like server alreadi respect client way within thread use socket server send request time use someth affect pass receiv buffer event tell data sent': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'country': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'parsing': 0.74701492537313419, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'xml process python im build piec project need construct post xml document web servic id like python mean expand skill unfortun whilst know xml model fairli well net im uncertain pro con xml model python anyon experi xml process python would suggest start xml file ill build fairli simpl': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'types': 0.74701492537313419, 'monkeypatching': 0.00074626865671641792}, u'open sourc rubi project recent start studi rubi lieu jeff advic weekend stop theoriz write lot softwar learn mistak interest hone skill help open sourc commun process thought id ask anyon suggest cool/interest open sourc project written rubi know involv': {'ip': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'data-structures': 0.42777777777777776, 'graph': 0.42777777777777776, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'sandcastle': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'winhelp': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'reduc duplic error handl code c# ive never complet happi way except handl work there lot except try/catch bring tabl stack unwind etc seem break lot oo model process anyway here problem let say class wrap includ network file io oper eg read write file particular unc path somewher variou reason dont want io oper fail detect fail retri keep retri succeed reach timeout alreadi conveni retrytim class instanti use sleep current thread retri determin timeout period elaps etc problem bunch io oper sever method class need wrap try-catch / retri logic here exampl code snippet avoid duplic code everi file io oper throughout class solut use anonym deleg block singl method class execut deleg block pass allow thing like method like somewhat leav lot desir id like hear peopl would solv sort problem': {'polymorphism': 0.29970059880239519, 'silverlight': 0.29970059880239519, 'xml-comments': 0.00029940119760479042, 'dependency-properties': 0.29970059880239519, 'debugging': 0.00029940119760479042, 'c#': 0.00029940119760479042, 'text': 0.00029940119760479042, 'pass-by-value': 0.00029940119760479042, 'localization': 0.00029940119760479042, 'exception-handling': 0.00029940119760479042, 'windows-server-2003': 0.00029940119760479042, 'sandcastle': 0.00029940119760479042, 'connection': 0.00029940119760479042, 'exception': 0.00029940119760479042, 'unicode': 0.00029940119760479042, 'vi': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'variables': 0.00029940119760479042, 'pass-by-reference': 0.00029940119760479042, 'monkeypatching': 0.00029940119760479042}, u'get match use preg_replac php tri grab capit letter coupl word wrap span tag use preg_replac extract wrap purpos output anyth': {'debugging': 0.00074626865671641792, 'protocol-buffers': 0.74701492537313419, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'folder project visual studio solut splite solut logic layer best use separ project group folder': {'ip': 0.00074626865671641792, 'debugging': 0.00074626865671641792, 'xfdl': 0.74701492537313419, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'text': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'hide inherit member im look way effect hide inherit member librari class inherit common base class recent descend class inherit depend properti becom vestigi littl confus use intellisens use class visual design class control written compil either wpf silverlight know im pretti certain cant use silverlight much function issu usabl issu updat properti would realli like hide come ancestor specif tool im design cant member hide oper know ridicul': {'debugging': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'dvcs': 0.74701492537313419, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'reliabl timer consol applic awar net three timer type see compar timer class net framework class librari chosen thread timer type drift main thread busi need reliabl way timer work control timer put anoth thread alway tick along work begin complet parent thread busi issu timer consol applic timer tick along anoth thread main thread anyth applic close tri ad loop main thread busi timer go': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'rdbms': 0.42777777777777776, 'pass-by-reference': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'text': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'datatable': 0.42777777777777776, 'monkeypatching': 0.0004273504273504274}, u'window server "server core" appropri sql server instanc im set dedic sql server box window server week would like pare barebon possibl still fulli function end "server core" option sound appeal im clear whether run sql server sku sever servic address microsoft websit dont see indic sql server anyon know definit': {'ip': 0.00029940119760479042, 'debugging': 0.00029940119760479042, 'exception': 0.00029940119760479042, 'xml-comments': 0.00029940119760479042, 'localization': 0.00029940119760479042, 'c#': 0.00029940119760479042, 'vi': 0.00029940119760479042, 'pass-by-value': 0.00029940119760479042, 'exception-handling': 0.00029940119760479042, 'subsonic': 0.29970059880239519, 'windows-server-2003': 0.00029940119760479042, 'connection': 0.00029940119760479042, 'orm': 0.29970059880239519, 'unicode': 0.00029940119760479042, 'text': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'variables': 0.00029940119760479042, 'nhibernate': 0.29970059880239519, 'pass-by-reference': 0.00029940119760479042, 'monkeypatching': 0.00029940119760479042}, u'elegantli express left join aggreg sql linq queri sql': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'algorithm': 0.74701492537313419, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'easy-to-us regular express support c++ im look robust easy-to-us regular express evalu nativ c++ either platform-independ windows-specif requir cant use boost ace regex librari unfortun cant use net regex manag code main requir standalon open': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'file-locking': 0.74701492537313419, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'would access object properti within object method "purist" "correct" way access object properti within object method getter/sett method know outsid object use getter/sett within would java php would java php forgiv java littl year sinc program java edit seem peopl assum talk privat protect variables/properti learn oo taught use getters/sett everi singl properti even public actual told never make variable/properti public may start fals assumpt get go appear peopl answer question mayb say public properti dont need getter setter goe taught talk although mayb need discuss well that probabl good topic differ question though': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'java': 0.42777777777777776, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'data-conversion': 0.42777777777777776, 'country': 0.0004273504273504274, 'text': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'use python itertoolsgroupbi havent abl find understand explan actual use python function im tri take list - case children objectifi element divid group base criteria later iter group separ ive review document exampl ive troubl tri appli beyond simpl list number use anoth techniqu use pointer good "prerequisite" read would also appreci': {'debugging': 0.00029940119760479042, 'exception': 0.00029940119760479042, 'localization': 0.00029940119760479042, 'c#': 0.00029940119760479042, 'vi': 0.00029940119760479042, 'pass-by-value': 0.00029940119760479042, 'system': 0.29970059880239519, 'exception-handling': 0.00029940119760479042, 'windows-server-2003': 0.00029940119760479042, 'linear-equation': 0.29970059880239519, 'connection': 0.00029940119760479042, 'text': 0.00029940119760479042, 'unicode': 0.00029940119760479042, 'ip': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'variables': 0.00029940119760479042, 'linear-algebra': 0.29970059880239519, 'lookup': 0.00029940119760479042, 'pass-by-reference': 0.00029940119760479042, 'monkeypatching': 0.00029940119760479042}, u'export data sql server mysql ive bang head sql server tri get lot data ive given databas nearli tabl need turn mysql databas first call use bcp unfortun doesnt produc valid csv - string arent encapsul cant deal row string comma whatev use delimit would still hand write creat tabl statement obvious csv doesnt tell anyth data type would better tool could connect sql server mysql copi lose view store procedur trigger etc isnt hard copi tabl use base type one db anoth anybodi know tool dont mind mani assumpt make simplif occur long support integ float datetim string lot prune normalis etc anyway dont care keep key relationship anyth like need initi set data fast!': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'theory': 0.74701492537313419, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'get subclips aptana work newest releas subvers version subclips current avail aptana automag plugin manag work newest version subvers see subclips websit howev eclips ad new remot updat site updat manag tri instal told need mylyn much search found mylyn ad anoth new remot updat site updat manag tri instal told need orgeclipseui equival look configur detail aptana look like built eclips anyon know way upgrad version eclips aptana built way get subclips work newest version subvers know isnt necessarili "programming" question hope ok sinc highli relev program experi': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'rdoc': 0.74701492537313419, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'annot youtub video programmat want abl display normal youtub video overlaid annot consist color rectangl frame requir done programat youtub annot requir use front end creat hand want abl gener what best way idea build flash player ew somehow draw youtub flash player work revers engin & hijack youtub annot system either mess local file redirect attempt download annot use greasemonkey firefox plugin idea doesnt count download video': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'printing': 0.74701492537313419, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'viewstat invalid safari one site maintain reli heavili use isnt code howev certain page extra-blo safari throw error appear happen safari firefox ie opera load success scenario': {'debugging': 0.00018726591760299626, 'account': 0.18745318352059923, 'unicode': 0.00018726591760299626, 'localization': 0.00018726591760299626, 'c#': 0.00018726591760299626, 'ip': 0.00018726591760299626, 'pass-by-value': 0.00018726591760299626, 'exception-handling': 0.00018726591760299626, 'connection': 0.00018726591760299626, 'exception': 0.00018726591760299626, 'text': 0.00018726591760299626, 'user': 0.18745318352059923, 'dns': 0.18745318352059923, 'vi': 0.00018726591760299626, 'loops': 0.00018726591760299626, 'lookup': 0.00018726591760299626, 'web-applications': 0.18745318352059923, 'subdomain': 0.18745318352059923, 'pass-by-reference': 0.00018726591760299626, 'monkeypatching': 0.00018726591760299626}, u'logic reorder column tabl im ad column tabl microsoft sql server control column display logic queri dont want mess physic layout column disk would like logic group column togeth possibl tool like sql server manag studio list content tabl conveni way know sql manag studio go "design" mode tabl drag order column around id like abl raw sql perform order script command line': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'migration': 0.74701492537313419, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'differ distribut version control system work togeth offic central sourc safe instal use sourc control cant chang offic use server develop laptop would like differ local sourc control repositori sync central server avail regardless central provid reason request maintain local stabl branch/build client present continu develop without jump flame hoop also consult client may request use sourc control provid flexibl would make life easier exist distribut sourc control client handl': {'ip': 0.00074626865671641792, 'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'iphone': 0.74701492537313419, 'unicode': 0.00074626865671641792, 'text': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'use svn revis label build ccnet use ccnet sampl project svn sourc control ccnet configur creat build everi check ccnet use msbuild build sourc code would like use latest revis number gener assemblyinfoc compil retriev latest revis subvers use valu ccnet edit im use nant - msbuild': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'indexing': 0.74701492537313419, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'add exist comment rdoc rubi ive got comment want make rdoc comment format appropri view use anyon get start understand use rdoc': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'ip': 0.00074626865671641792, '.net-3.5': 0.74701492537313419, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'definit guid form base websit authent form base authent websit believ stack overflow resourc specif technic question also gener guidelin solv variat common problem "form base authent websites" fine topic experi includ topic log remain log store password use secret question forgotten username/password function openid "rememb me" checkbox browser autocomplet usernam password secret url public url protect digest check password strength e-mail valid much form base authent includ thing like role author http basic authent pleas help us suggest subtop submit good articl subject edit offici answer': {'debugging': 0.0004273504273504274, 'continuous-integration': 0.42777777777777776, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'text': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'extreme-programming': 0.42777777777777776, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'advantag use svn cv compani use cv de-facto standard sourc control howev ive heard lot peopl say svn better know svn newer im unfamiliar benefit im look good succinct comparison two system note advantag disadvantag java/eclips develop environ': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'sockets': 0.74701492537313419, 'monkeypatching': 0.00074626865671641792}, u'map latitude/longitud distort map bunch latitude/longitud pair map known x/i coordin geograph distort map one latitude/longitud pair want plot map best possibl go first decid creat system linear equat three nearest lat/long point comput transform doesnt work well sinc that linear system cant use nearbi point either cant assum north exist lat/long->x/i map edit mercat project anyth like arbitrarili distort readabl think subway map want use nearest map distort part map doesnt affect map im tri comput entir map small geograph area there need worri globe--flat-earth assumpt good enough': {'variables': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'import': 0.42777777777777776, 'csv': 0.42777777777777776, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'version sql server databas ive want get databas version control anyon advic recommend articl get start ill alway want least data alumb mention user type administr ill also often want larg collect gener test data perform measur': {'ip': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'media': 0.42777777777777776, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'embed': 0.42777777777777776, 'country': 0.0004273504273504274, 'text': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'calcul someon age c# given repres person birthday calcul age': {'vi': 0.00029940119760479042, 'debugging': 0.00029940119760479042, 'exception': 0.00029940119760479042, 'localization': 0.00029940119760479042, 'c#': 0.00029940119760479042, 'visualj#': 0.29970059880239519, 'pass-by-value': 0.00029940119760479042, 'exception-handling': 0.00029940119760479042, 'country': 0.00029940119760479042, 'connection': 0.00029940119760479042, 'lookup': 0.00029940119760479042, 'code-generation': 0.29970059880239519, 'unicode': 0.00029940119760479042, 'ip': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'variables': 0.00029940119760479042, 'monkeypatching': 0.00029940119760479042, 'text': 0.00029940119760479042, 'pass-by-reference': 0.00029940119760479042, 'j#': 0.29970059880239519}, u'pass enumer valu web servic dilemma basic share enumer two applic user upload document front-end applic web applic call web servic back-end applic pass document back-end app save document insert row document tabl document type possibl document type invoic contract etc pass paramet web servic uploaddocu method question type possibl valu paramet sinc need hardcod valu applic think ok use descript string invoic contract workord signedworkord mayb better approach creat documenttyp enumer first applic reproduc also second applic pass correspond integ valu web servic': {'profile': 0.74701492537313419, 'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'best way allow plugin php applic start new web applic php time around want creat someth peopl extend use plugin interfac one go write hook code plugin attach specif event': {'ip': 0.00074626865671641792, 'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vb.net': 0.74701492537313419, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'process size unix correct way get process size use someth els': {'variables': 0.00029940119760479042, 'debugging': 0.29970059880239519, 'exception': 0.00029940119760479042, 'xml-comments': 0.00029940119760479042, 'localization': 0.00029940119760479042, 'c#': 0.00029940119760479042, 'ip': 0.00029940119760479042, 'pass-by-value': 0.00029940119760479042, 'memory': 0.29970059880239519, 'exception-handling': 0.00029940119760479042, 'windows-server-2003': 0.00029940119760479042, 'connection': 0.00029940119760479042, 'text': 0.00029940119760479042, 'unicode': 0.00029940119760479042, 'vi': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'lookup': 0.00029940119760479042, 'multithreading': 0.29970059880239519, 'pass-by-reference': 0.00029940119760479042, 'monkeypatching': 0.00029940119760479042}, u'decent simpl gis/map compon aspnet need display map us state shown differ color accord underli data id also like map contain facil subscrib hover/click event state javascript presum use imagemap data wont chang kind ajax-i way ideal compon would gener static imag would cacheabl suggest': {'variables': 0.00074626865671641792, 'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'unit-testing': 0.74701492537313419, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'use object properti default method properti im tri produc unexpect t_variabl error dont want put magic number weight sinc object use "defaultweight" paramet new shipment get dont specifi weight cant put defaultweight shipment chang shipment group shipment group better way follow': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'permissions': 0.74701492537313419, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'best way avoid sql inject attack ive provid solut python pleas flesh exampl languag': {'ip': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'stored-procedures': 0.42777777777777776, 'xml-comments': 0.0004273504273504274, 'triggers': 0.42777777777777776, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'sandcastle': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'text': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'good technolog podcast ye podcast nice littl audiobook listen way work current amount podcast like search needl haystack except haystack happen internet fill mani "hot new gadgets" stuff even though mainli net develop nowaday mayb anyon know good podcast peopl regard whole softwar lifecycl unit test contin integr document deploy - guy gal listen pleas note categor somewhat subject may % accur mani podcast cover sever area categor made consid "main" area gener softwar engin / product stack overflow inact still good listen tekpub requir paid subscript softwar engin radio folder perspect dr dobb video feed pragmat podcast inact matter agil toolkit podcast stack trace inact parley techz startup success podcast berkeley cs class lectur floss weekli develop life net / visual studio / microsoft herd code hanselminut net rocks! deep fri byte altnet podcast inact polymorph podcast inconsist sparkl client silverlight podcast dnrtv! spaghetti code aspnet podcast channel radio tf powerscript podcast thirsti develop eleg code inact connectedshow crafti coder code qa jqueri yayqueri offici jqueri podcast java / groovi java poss grail podcast java technolog insid basement coder rubi / rail railscast rail envi rubi rail podcast rubivers rubi web design / javascript / ajax webdevradio boagworld rissington podcast ajaxian yui theater unix / linux / mac / iphon mac develop network hacker public radio linux outlaw mac os ken lugradio linux radio show inact linux action show! linux kernel mail list lkml summari podcast stanford iphon program class advanc iphon develop cours - madison area technic colleg wwdc session video requir appl develop registr system administr secur infrastructur runa radio secur now! crypto-gram secur podcast hak vmware vmtn window weekli pauldotcom secur regist - semi-coher comput feathercast gener tech / busi tekzilla week tech guardian tech weekli pcmag radio podcast inact entrepreneurship corner manag tool / misc / podcast network convers retrobit podcast agenda netcast cranki geek command line freelanc radio ibm developerwork regist - open season drunk retir technometria sod radionerd hacker medley': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'linux': 0.74701492537313419, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'multipl submit button html form let say creat wizard html form one button goe back one goe forward sinc back button appear first markup press enter use button submit form exampl would like get decid button use submit form user press enter way press enter wizard move next page previou use': {'variables': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'hook': 0.42777777777777776, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'architecture': 0.42777777777777776, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'country': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'decod t-sql cast c#/vbnet recent site delug resurg asprox bot sql inject attack without go detail attack attempt execut sql code encod t-sql command ascii encod binari string look someth like abl decod sql littl wari sinc didnt know exactli happen time tri write simpl decod tool could decod type text without even touch sql server main part need decod ive tri follow command luck anybodi suggest proper way translat encod without use sql server possibl ill take vbnet code sinc im familiar okay im sure im miss someth here im sinc input basic string start snippet encod portion - c translat decla - first attempt return exact thing put sinc convert charact byte realiz need pars two charact byte manual sinc dont know method yet littl decod look someth like thing look good first coupl pair loop balk get "c" pair say string incorrect format interestingli enough step debugg getstr method byte array abl pars point get "-+" result anybodi help figur im miss need "direct cast" byte instead attempt pars': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, '64bit': 0.74701492537313419, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'ad script function net applic littl game written c# use databas back-end trade card game want implement function card script mean essenti interfac card class implement contain function call game make thing maintainable/modd would like class card sourc code databas essenti compil first use add/chang card ill add databas tell applic refresh without need assembl deploy especi sinc would talk assembl per card mean hundr assembl possibl regist class sourc file instanti etc languag c# extra bonu possibl write script net languag': {'debugging': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'forms': 0.74701492537313419, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'use combin set test data would like test function tupl set fring case normal valu exampl test function return whenev given three length form valid triangl would specif case neg / small / larg number valu close-to overflow etc; main aim gener combin valu without repetit order get set test data note actual know answer might help other challeng peopl here! --will post answer later': {'variables': 0.00074626865671641792, 'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'article': 0.74701492537313419, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'use lambda use procnew rubi subtl differ proc/lambda one hand differ give guidelin decid one choos rubi proc lambda differ what deal': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'primes': 0.74701492537313419, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'delet file lock anoth process c# im look way delet file lock anoth process use c# suspect method must abl find process lock file perhap track handl although im sure c# close process abl complet file delet use': {'debugging': 0.00029940119760479042, 'exception': 0.00029940119760479042, 'adobe': 0.29970059880239519, 'xml-comments': 0.00029940119760479042, 'localization': 0.00029940119760479042, 'vi': 0.00029940119760479042, 'pass-by-value': 0.00029940119760479042, 'flash': 0.29970059880239519, 'exception-handling': 0.00029940119760479042, 'windows-server-2003': 0.00029940119760479042, 'macromedia': 0.29970059880239519, 'connection': 0.00029940119760479042, 'lookup': 0.00029940119760479042, 'unicode': 0.00029940119760479042, 'ip': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'variables': 0.00029940119760479042, 'text': 0.00029940119760479042, 'pass-by-reference': 0.00029940119760479042, 'monkeypatching': 0.00029940119760479042}, u'sync svn revis number aspnet web site stack overflow subvers version number bottom svn revis want use automat version net web site/appl window form wpd projects/solut implement': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'analytics': 0.42777777777777776, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'download': 0.42777777777777776, 'variables': 0.0004273504273504274, 'country': 0.0004273504273504274, 'text': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'aspnet custom client-sid valid custom valid function javascript user control net web site check see fee paid excess fee amount due ive place valid code ascx file also tri use pageclientscriptregisterclientscriptblock case valid fire cannot find javascript function output firefox error consol "feeamountcheck defined" function taken directli firefox->view sourc idea function isnt found remedi without add function master page consum page': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'windows-server-2008': 0.74701492537313419, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'languag use postgresql trigger store procedur postgresql interest support sever languag write store procedur one use': {'debugging': 0.0004273504273504274, 'datacontext': 0.42777777777777776, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'linq-to-sql': 0.42777777777777776, 'variables': 0.0004273504273504274, 'sandcastle': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'embed window media player browser use wmv video intern site embed web site work quit well internet explor firefox ive found way make work firefox stop work internet explor want use silverlight yet especi sinc cannot sure client run window xp window media player instal sort univers code emb wmp internet explor firefox need implement user-agent-detect deliv differ html differ browser': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'versioning': 0.74701492537313419, 'monkeypatching': 0.00074626865671641792}, u'track file download websit play mp flash player user click play flash player automat download mp start play easi way track mani time particular song clip binari file download play link link actual mp file javascript code pop player latter easili add log code track number hit former youll need someth track web server log make distinct host plan come webal nice javascript code answer howev would nice know track download use method without switch host': {'variables': 0.0004273504273504274, 'debugging': 0.0004273504273504274, 'malloc': 0.0004273504273504274, 'compression': 0.42777777777777776, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'decompression': 0.42777777777777776, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'text': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'express binari liter python express integ binari number python liter easili abl find answer hex octal use liter express binari python summari answer python earlier express binari use liter python earlier way express binari liter python beta like python beta also allow second charact letter o repres octal python beta longer allow older syntax octal': {'vi': 0.00029940119760479042, 'debugging': 0.00029940119760479042, 'exception': 0.00029940119760479042, 'address-bar': 0.29970059880239519, 'xml-comments': 0.00029940119760479042, 'localization': 0.00029940119760479042, 'c#': 0.00029940119760479042, 'connection': 0.00029940119760479042, 'pass-by-value': 0.00029940119760479042, 'exception-handling': 0.00029940119760479042, 'rest': 0.29970059880239519, 'windows-server-2003': 0.00029940119760479042, 'ajax': 0.29970059880239519, 'unicode': 0.00029940119760479042, 'text': 0.00029940119760479042, 'loops': 0.00029940119760479042, 'variables': 0.00029940119760479042, 'ip': 0.00029940119760479042, 'pass-by-reference': 0.00029940119760479042, 'monkeypatching': 0.00029940119760479042}, u'limit size queue<t> net queue<t> object initialis capac obvious capac keep expand add item alreadi object automat dequeu item limit reach best solut creat inherit class': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'database': 0.74701492537313419, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'singl influenti book everi programm read could go back time tell read specif book begin career develop book would expect list vari cover wide rang thing search use search box upper-right corner search answer current question use exampl': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'text': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'application-integration': 0.74701492537313419, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'python differ use mani place use interchang use one': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'throttle': 0.42777777777777776, 'xml-comments': 0.0004273504273504274, 'pass-by-reference': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'sandcastle': 0.0004273504273504274, 'connection': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'cpu': 0.42777777777777776, 'monkeypatching': 0.0004273504273504274}, u'html word-break dash given rel simpl css make string stay constrain width simpli wrap newlin hyphen': {'debugging': 0.00074626865671641792, 'osx': 0.74701492537313419, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'distribut sourc control option ive interest git last time look window support essenti "run linux" distribut sourc control tool run well window': {'debugging': 0.0029411764705882353, 'exception': 0.0029411764705882353, 'xml-comments': 0.0029411764705882353, 'localization': 0.0029411764705882353, 'c#': 0.0029411764705882353, 'ip': 0.0029411764705882353, 'pass-by-value': 0.0029411764705882353, 'exception-handling': 0.0029411764705882353, 'windows-server-2003': 0.0029411764705882353, 'sandcastle': 0.0029411764705882353, 'connection': 0.0029411764705882353, 'lookup': 0.0029411764705882353, 'unicode': 0.0029411764705882353, 'vi': 0.0029411764705882353, 'loops': 0.0029411764705882353, 'variables': 0.0029411764705882353, 'country': 0.0029411764705882353, 'text': 0.0029411764705882353, 'pass-by-reference': 0.0029411764705882353, 'monkeypatching': 0.0029411764705882353}, u'bayesian filter spam wonder good clean oo implement bayesian filter spam text classif learn purpos': {'debugging': 0.0004273504273504274, 'exception': 0.0004273504273504274, 'xml-comments': 0.0004273504273504274, 'localization': 0.0004273504273504274, 'c#': 0.0004273504273504274, 'ip': 0.0004273504273504274, 'pass-by-value': 0.0004273504273504274, 'exception-handling': 0.0004273504273504274, 'lookup': 0.0004273504273504274, 'windows-server-2003': 0.0004273504273504274, 'unix': 0.42777777777777776, 'connection': 0.0004273504273504274, 'text': 0.0004273504273504274, 'unicode': 0.0004273504273504274, 'vi': 0.0004273504273504274, 'loops': 0.0004273504273504274, 'variables': 0.0004273504273504274, 'size': 0.42777777777777776, 'pass-by-reference': 0.0004273504273504274, 'monkeypatching': 0.0004273504273504274}, u'cruisecontrolnet msbuild /poutputpath ccnetartifactdirectori im tri setup cruisecontrolnet moment far work nice problem msbuild task accord document pass ccnetartifactdirectori msbuild use tri work fact kill servic error thoughtworkscruisecontrolcoreconfigpreprocessorevaluationexcept refer unknown symbol ccnetartifactdirectori document rather spars googl und mainli offer modifi sln project file want avoid order abl manual build project later - would realli prefer': {'debugging': 0.0029411764705882353, 'exception': 0.0029411764705882353, 'xml-comments': 0.0029411764705882353, 'localization': 0.0029411764705882353, 'c#': 0.0029411764705882353, 'ip': 0.0029411764705882353, 'pass-by-value': 0.0029411764705882353, 'exception-handling': 0.0029411764705882353, 'windows-server-2003': 0.0029411764705882353, 'sandcastle': 0.0029411764705882353, 'connection': 0.0029411764705882353, 'lookup': 0.0029411764705882353, 'unicode': 0.0029411764705882353, 'vi': 0.0029411764705882353, 'loops': 0.0029411764705882353, 'variables': 0.0029411764705882353, 'country': 0.0029411764705882353, 'text': 0.0029411764705882353, 'pass-by-reference': 0.0029411764705882353, 'monkeypatching': 0.0029411764705882353}, u'float point number pars catch algorithm one fun part multi-cultur program number format american use german use french use first approach would take string pars backward encount separ use decim separ obviou flaw would interpret anoth approach string contain differ non-numer charact use last one decim separ discard other one check occur discard appear check digit ye discard otherwis use decim separ obviou "best solution" would detect user cultur browser work frenchman use en-u windows/brows net framework contain mythic black magic float point parser better tri auto-detect number format': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'color-space': 0.74701492537313419, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'what best way gener tag cloud array use h h size follow array want gener tag cloud artist higher number enclos tag lowest enclos tag': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'sql-server': 0.74701492537313419, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'get countri accord certain ip anyon know simpl way retriev countri given ip address prefer format': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'plugins': 0.74701492537313419, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'ternari express work tri set flag show hide page element alway display even express fals what': {'debugging': 0.0029411764705882353, 'exception': 0.0029411764705882353, 'xml-comments': 0.0029411764705882353, 'localization': 0.0029411764705882353, 'c#': 0.0029411764705882353, 'ip': 0.0029411764705882353, 'pass-by-value': 0.0029411764705882353, 'exception-handling': 0.0029411764705882353, 'windows-server-2003': 0.0029411764705882353, 'sandcastle': 0.0029411764705882353, 'connection': 0.0029411764705882353, 'lookup': 0.0029411764705882353, 'unicode': 0.0029411764705882353, 'vi': 0.0029411764705882353, 'loops': 0.0029411764705882353, 'variables': 0.0029411764705882353, 'country': 0.0029411764705882353, 'text': 0.0029411764705882353, 'pass-by-reference': 0.0029411764705882353, 'monkeypatching': 0.0029411764705882353}, u'onlin peer code review peopl work small team team far best develop need learn someon better onlin resources/method sort peer code review use pretti activ onlin art commun specif draw anim sort site could post pictur critiqu comment ive never seen anyth like programm im think somewher could post object architectur plan db design plan even straight code other look learn critiqu comment anyon know anyth like anywher prefer focus toward php anyth even non-fre site would ok note ye awar could secur implic secur obscur worst secur post code would actual help flush potenti secur issu': {'debugging': 0.00074626865671641792, 'exception': 0.74701492537313419, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'sandcastle': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'calcul rel time given specif valu display rel time like hour ago day ago month ago et cetera': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'datetime': 0.74701492537313419, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'country': 0.00074626865671641792, 'text': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}, u'format string titl case format string titl case': {'debugging': 0.00074626865671641792, 'exception': 0.00074626865671641792, 'xml-comments': 0.00074626865671641792, 'views': 0.74701492537313419, 'c#': 0.00074626865671641792, 'ip': 0.00074626865671641792, 'pass-by-value': 0.00074626865671641792, 'localization': 0.00074626865671641792, 'exception-handling': 0.00074626865671641792, 'country': 0.00074626865671641792, 'windows-server-2003': 0.00074626865671641792, 'connection': 0.00074626865671641792, 'text': 0.00074626865671641792, 'unicode': 0.00074626865671641792, 'vi': 0.00074626865671641792, 'loops': 0.00074626865671641792, 'variables': 0.00074626865671641792, 'lookup': 0.00074626865671641792, 'pass-by-reference': 0.00074626865671641792, 'monkeypatching': 0.00074626865671641792}}

In [74]:
v

{u'access remot form php want gather info user local php page control use info queri form anoth site dont control': {'c#': 0.0007462686567164179,
  'connection': 0.0007462686567164179,
  'debugging': 0.0007462686567164179,
  'exception': 0.0007462686567164179,
  'exception-handling': 0.0007462686567164179,
  'ip': 0.0007462686567164179,
  'localization': 0.0007462686567164179,
  'lookup': 0.0007462686567164179,
  'loops': 0.0007462686567164179,
  'monkeypatching': 0.0007462686567164179,
  'pass-by-reference': 0.0007462686567164179,
  'pass-by-value': 0.0007462686567164179,
  'sandcastle': 0.0007462686567164179,
  'sqlite': 0.7470149253731342,
  'text': 0.0007462686567164179,
  'unicode': 0.0007462686567164179,
  'variables': 0.0007462686567164179,
  'vi': 0.0007462686567164179,
  'windows-server-2003': 0.0007462686567164179,
  'xml-comments': 0.0007462686567164179},
 u'ad method exist object ive read possibl add method exist object eg class definit python think call monkey patch case d